# Analisis Simulacion

## Analisis General - Simulación Base

### Pre-procesamiento

In [1]:
import pandas as pd

# 1. Leer los archivos
arma_df = pd.read_excel("./datos/Simulacion/Base/resultados_140_ARMA_FINAL.xlsx")
arima_df = pd.read_excel("./datos/Simulacion/Base/resultados_140_ARIMA_FINAL.xlsx")
setar_df = pd.read_excel("./datos/Simulacion/Base/resultados_140_SETAR_FINAL.xlsx")

# 2. Asignar la columna ESCENARIO a cada dataframe
arma_df['ESCENARIO'] = "Lineal Estacionario"
arima_df['ESCENARIO'] = "Lineal No estacionario"
setar_df['ESCENARIO'] = "No lineal Estacionario"

# 3. Juntarlos uno bajo el otro (Concatenar)
df_total = pd.concat([arma_df, arima_df, setar_df], ignore_index=True)

# Seleccionar solo las columnas requeridas
columnas_deseadas = [
    "Paso", "Config", "Dist", "Var", "Block Bootstrapping", 
    "Sieve Bootstrap", "LSPM", "LSPMW", "AREPD", "MCPS", 
    "AV-MCPS", "DeepAR", "EnCQR-LSTM", "ESCENARIO"
]
df_total = df_total[columnas_deseadas]

# Definimos cuáles son las columnas que representan a los modelos predictivos
modelos = [
    "Block Bootstrapping", "Sieve Bootstrap", "LSPM", "LSPMW", 
    "AREPD", "MCPS", "AV-MCPS", "DeepAR", "EnCQR-LSTM"
]

# 4. Guardar el dataframe consolidado
df_total.to_excel("./datos/Simulacion/Base/dataframe_consolidado.xlsx", index=False)

# 5. Generar y mostrar las tablas (Media y Mediana)
metricas = {'MEDIA': 'mean', 'MEDIANA': 'median'}

for nombre_metrica, funcion in metricas.items():
    # Calculamos el valor general según la métrica (mean o median)
    if funcion == 'mean':
        resumen_general = df_total[modelos].mean()
        resumen_escenarios = df_total.groupby('ESCENARIO')[modelos].mean().T
    else:
        resumen_general = df_total[modelos].median()
        resumen_escenarios = df_total.groupby('ESCENARIO')[modelos].median().T

    # Construimos la tabla final para esta métrica
    tabla_resumen = pd.DataFrame(index=modelos)
    tabla_resumen['General'] = resumen_general
    tabla_resumen['ARMA'] = resumen_escenarios['Lineal Estacionario']
    tabla_resumen['ARIMA'] = resumen_escenarios['Lineal No estacionario']
    tabla_resumen['SETAR'] = resumen_escenarios['No lineal Estacionario']

    # Determinar el Mejor_Escenario (valor mínimo entre los tres escenarios)
    escenarios_cols = ['ARMA', 'ARIMA', 'SETAR']
    tabla_resumen['Mejor_Escenario'] = tabla_resumen[escenarios_cols].idxmin(axis=1)

    # Imprimir resultado
    print(f"\n--- Tabla Comparativa de Modelos (Basada en {nombre_metrica}) ---")
    print(tabla_resumen.reset_index().rename(columns={'index': 'Modelo'}).to_string(index=False))


--- Tabla Comparativa de Modelos (Basada en MEDIA) ---
             Modelo  General     ARMA     ARIMA    SETAR Mejor_Escenario
Block Bootstrapping 4.275512 0.947365 11.251901 0.627270           SETAR
    Sieve Bootstrap 0.570491 0.547725  0.547481 0.616268           ARIMA
               LSPM 0.845497 0.811287  1.064804 0.660398           SETAR
              LSPMW 1.572319 0.960743  3.079645 0.676570           SETAR
              AREPD 3.880255 0.936671 10.031183 0.672910           SETAR
               MCPS 1.562693 0.759453  3.231805 0.696822           SETAR
            AV-MCPS 1.588747 0.740911  3.341841 0.683490           SETAR
             DeepAR 1.836024 0.568693  4.329124 0.610255            ARMA
         EnCQR-LSTM 2.454851 0.843920  5.850490 0.670144           SETAR

--- Tabla Comparativa de Modelos (Basada en MEDIANA) ---
             Modelo  General     ARMA    ARIMA    SETAR Mejor_Escenario
Block Bootstrapping 0.989417 0.653853 5.275960 0.540499           SETAR
    Sieve Bo

### Analisis general

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import normaltest
import warnings
from pathlib import Path
import itertools

warnings.filterwarnings('ignore')

# Configuración general
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'
sns.set_palette("husl")

# Crear carpeta de resultados
output_dir = Path("./Resultados_analisis/Simulacion_base")
output_dir.mkdir(parents=True, exist_ok=True)

# Crear carpeta de interacciones
interactions_dir = output_dir / "Interacciones"
interactions_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
df = pd.read_excel("./datos/Simulacion/Base/dataframe_consolidado.xlsx")

# 1) CAMBIO DE NOMBRES DE ESCENARIOS
df['ESCENARIO'] = df['ESCENARIO'].replace({
    "Lineal Estacionario": "Lineal Estacionario (ARMA)",
    "Lineal No estacionario": "Lineal No Estacionario (ARIMA)",
    "No lineal Estacionario": "No lineal Estacionario (SETAR)"
})

# Identificar columnas de modelos
var_cols = ['Paso', 'Config', 'Dist', 'Var', 'ESCENARIO']
original_model_cols = [col for col in df.columns if col not in var_cols]

# 2) ORGANIZAR MODELOS POR RENDIMIENTO EN "Lineal No Estacionario (ARIMA)" (Menor a mayor)
target_scenario = "Lineal No Estacionario (ARIMA)"
model_order_scores = df[df['ESCENARIO'] == target_scenario][original_model_cols].mean().sort_values()
model_cols = list(model_order_scores.index)

print("Nuevo orden de modelos (basado en Lineal No Estacionario (ARIMA)):")
for i, m in enumerate(model_cols, 1):
    print(f"{i}. {m}: {model_order_scores[m]:.4f}")

# Mapeo de escenarios
escenarios_map = {
    'Lineal Estacionario (ARMA)': 'Lineal Estacionario (ARMA)',
    'Lineal No Estacionario (ARIMA)': 'Lineal No Estacionario (ARIMA)',
    'No lineal Estacionario (SETAR)': 'No lineal Estacionario (SETAR)'
}

# Definir colores para cada modelo
palette = sns.color_palette("husl", len(model_cols))
model_colors = {model: palette[i] for i, model in enumerate(model_cols)}

# ====================================================================================
# SECCIÓN 1: RENDIMIENTO POR ESCENARIOS
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 1: RENDIMIENTO POR ESCENARIOS")
print("="*80)

def plot_performance_by_scenario():
    fig, ax = plt.subplots(figsize=(14, 8))
    
    scenarios = ["Lineal Estacionario (ARMA)", "Lineal No Estacionario (ARIMA)", "No lineal Estacionario (SETAR)"]
    x = np.arange(len(model_cols))
    width = 0.25 
    
    scenario_colors = {
        'Lineal Estacionario (ARMA)': '#5D3FD3',    
        'Lineal No Estacionario (ARIMA)': '#808080', 
        'No lineal Estacionario (SETAR)': '#00A36C'  
    }
    
    for idx, scenario in enumerate(scenarios):
        means = [df[df['ESCENARIO'] == scenario][model].mean() for model in model_cols]
        position = x + (idx - 1) * width
        
        bars = ax.bar(position, means, width, label=scenario, 
                     color=scenario_colors[scenario], alpha=0.85, edgecolor='black', linewidth=0.5)
        
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2f}', ha='center', va='bottom', fontsize=7, rotation=0)
    
    ax.set_xlabel('Modelo (Ordenados por desempeño en ARIMA)', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title('Rendimiento de Modelos por Escenario (ECRPS)', fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(model_cols, rotation=45, ha='right', fontsize=9)
    ax.legend(loc='upper left', ncol=1, fontsize=10, framealpha=0.9)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '1.1_rendimiento_por_escenario.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 1.1 guardada")

plot_performance_by_scenario()

def plot_relative_performance():
    base_scenario = 'Lineal Estacionario (ARMA)'
    scenarios_compare = ['Lineal No Estacionario (ARIMA)', 'No lineal Estacionario (SETAR)']
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    y = np.arange(len(model_cols))
    height = 0.35  
    
    for idx, scenario in enumerate(scenarios_compare):
        changes = []
        for model in model_cols:
            base_value = df[df['ESCENARIO'] == base_scenario][model].mean()
            scenario_value = df[df['ESCENARIO'] == scenario][model].mean()
            pct_change = ((scenario_value - base_value) / base_value) * 100
            changes.append(pct_change)
        
        position = y + idx * height
        bars = ax.barh(position, changes, height, label=scenario, alpha=0.85, edgecolor='black', linewidth=0.5)
        
        for bar, val in zip(bars, changes):
            width = bar.get_width()
            ax.text(width + (1 if width > 0 else -1), bar.get_y() + bar.get_height()/2.,
                   f'{val:+.1f}%', ha='left' if val > 0 else 'right', 
                   va='center', fontsize=7)
    
    ax.set_yticks(y + height / 2)
    ax.set_yticklabels(model_cols, fontsize=10)
    ax.set_xlabel('Cambio Relativo (%)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    ax.set_title(f'Cambio Relativo en ECRPS vs. {base_scenario}', fontsize=14, fontweight='bold', pad=20)
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1.5)
    ax.legend(loc='best', fontsize=10, framealpha=0.9)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '1.2_cambio_relativo_escenario_base.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 1.2 guardada")

plot_relative_performance()

# ====================================================================================
# SECCIÓN 2: ANÁLISIS POR CONFIG
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 2: ANÁLISIS POR CONFIG")
print("="*80)

def plot_zscore_heatmap_config(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Z-scores de ECRPS por Configuración ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Z-scores de ECRPS por Configuración (General)'
    
    pivot_data = data_filtered.groupby('Config')[model_cols].mean()
    z_scores = pivot_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(z_scores.T, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, cbar_kws={'label': 'Z-score'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=12, fontweight='bold', pad=20)
    ax.set_xlabel('Configuración', fontsize=11)
    ax.set_ylabel('Modelo', fontsize=11)
    
    plt.tight_layout()
    filename = f'2.1{suffix}_zscore_config.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 2.1{suffix} guardada")

plot_zscore_heatmap_config()
plot_zscore_heatmap_config('Lineal Estacionario (ARMA)', '.a')
plot_zscore_heatmap_config('Lineal No Estacionario (ARIMA)', '.b')
plot_zscore_heatmap_config('No lineal Estacionario (SETAR)', '.c')

def plot_variability_config(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Configuración ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Configuración (General)'
    
    pivot_std = data_filtered.groupby('Config')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    fmt = '.2f' if suffix == '' else '.4f'
    sns.heatmap(pivot_std.T, annot=True, fmt=fmt, cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Configuración', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'2.2{suffix}_variabilidad_config.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 2.2{suffix} guardada")

plot_variability_config()
plot_variability_config('Lineal Estacionario (ARMA)', '.a')
plot_variability_config('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_config('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 3: ANÁLISIS POR DIST
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 3: ANÁLISIS POR DIST")
print("="*80)

def plot_zscore_heatmap_dist(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Z-scores de ECRPS por Distribución ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Z-scores de ECRPS por Distribución (General)'
    
    pivot_data = data_filtered.groupby('Dist')[model_cols].mean()
    z_scores = pivot_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(z_scores.T, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, cbar_kws={'label': 'Z-score'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=12, fontweight='bold', pad=20)
    ax.set_xlabel('Distribución', fontsize=11)
    ax.set_ylabel('Modelo', fontsize=11)
    
    plt.tight_layout()
    filename = f'3.1{suffix}_zscore_dist.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 3.1{suffix} guardada")

plot_zscore_heatmap_dist()
plot_zscore_heatmap_dist('Lineal Estacionario (ARMA)', '.a')
plot_zscore_heatmap_dist('Lineal No Estacionario (ARIMA)', '.b')
plot_zscore_heatmap_dist('No lineal Estacionario (SETAR)', '.c')

def plot_variability_dist(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Distribución ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Distribución (General)'
    
    pivot_std = data_filtered.groupby('Dist')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Distribución', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'3.2{suffix}_variabilidad_dist.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 3.2{suffix} guardada")

plot_variability_dist()
plot_variability_dist('Lineal Estacionario (ARMA)', '.a')
plot_variability_dist('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_dist('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 4: ANÁLISIS POR VAR
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 4: ANÁLISIS POR VAR")
print("="*80)

def plot_evolution_var(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Evolución de ECRPS por Varianza ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Evolución de ECRPS por Varianza (General)'
    
    fig, ax = plt.subplots(figsize=(12, 7))
    var_values = sorted(data_filtered['Var'].unique())
    
    for model in model_cols:
        means = []
        for var in var_values:
            mean_val = data_filtered[data_filtered['Var'] == var][model].mean()
            means.append(mean_val)
        
        ax.plot(var_values, means, marker='o', label=model, color=model_colors[model],
                linewidth=2.5, markersize=7, alpha=0.85)
    
    ax.set_xlabel('Varianza', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='best', fontsize=9, ncol=2, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    filename = f'4.1{suffix}_evolucion_var.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 4.1{suffix} guardada")

plot_evolution_var()
plot_evolution_var('Lineal Estacionario (ARMA)', '.a')
plot_evolution_var('Lineal No Estacionario (ARIMA)', '.b')
plot_evolution_var('No lineal Estacionario (SETAR)', '.c')

def plot_variability_var(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Varianza ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Varianza (General)'
    
    pivot_std = data_filtered.groupby('Var')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Varianza', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'4.2{suffix}_variabilidad_var.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 4.2{suffix} guardada")

plot_variability_var()
plot_variability_var('Lineal Estacionario (ARMA)', '.a')
plot_variability_var('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_var('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)")
print("="*80)

def plot_evolution_paso(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Evolución de ECRPS por Horizonte de Pronóstico ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Evolución de ECRPS por Horizonte de Pronóstico (General)'
    
    fig, ax = plt.subplots(figsize=(12, 7))
    pasos = sorted(data_filtered['Paso'].unique())
    
    for model in model_cols:
        means = []
        for paso in pasos:
            mean_val = data_filtered[data_filtered['Paso'] == paso][model].mean()
            means.append(mean_val)
        
        ax.plot(pasos, means, marker='o', label=model, color=model_colors[model],
                linewidth=2.5, markersize=7, alpha=0.85)
    
    ax.set_xlabel('Horizonte de Pronóstico', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='best', fontsize=9, ncol=2, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    filename = f'5.1{suffix}_evolucion_paso.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 5.1{suffix} guardada")

plot_evolution_paso()
plot_evolution_paso('Lineal Estacionario (ARMA)', '.a')
plot_evolution_paso('Lineal No Estacionario (ARIMA)', '.b')
plot_evolution_paso('No lineal Estacionario (SETAR)', '.c')

def plot_variability_paso(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Horizonte ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Horizonte (General)'
    
    pivot_std = data_filtered.groupby('Paso')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Horizonte de Pronóstico', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'5.2{suffix}_variabilidad_paso.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 5.2{suffix} guardada")

plot_variability_paso()
plot_variability_paso('Lineal Estacionario (ARMA)', '.a')
plot_variability_paso('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_paso('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 5.5: ANÁLISIS DE INTERACCIONES
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 5.5: ANÁLISIS DE INTERACCIONES")
print("="*80)

# Función auxiliar para configurar el grid de modelos
def get_model_grid_axes(n_models):
    n_cols = 3
    n_rows = (n_models + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows))
    return fig, axes.flatten(), n_rows, n_cols

# INTERACCIÓN 3 (Lista 8): DIST × PASO - Subplots por Modelo
def plot_interaction_dist_paso(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Dist × Paso por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, n_rows, n_cols = get_model_grid_axes(len(model_cols))
    dists = sorted(data_filtered['Dist'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_dist = sns.color_palette("viridis", len(dists))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for d_idx, dist in enumerate(dists):
            means = [data_filtered[(data_filtered['Dist'] == dist) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='o', label=dist, color=colors_dist[d_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Distribución", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.7{suffix}_interaccion_dist_paso_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 8 (5.7) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 4 (Lista 9): DIST × VAR - Subplots por Modelo
def plot_interaction_dist_var(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Dist × Var por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    dists = sorted(data_filtered['Dist'].unique())
    vars_val = sorted(data_filtered['Var'].unique())
    colors_dist = sns.color_palette("magma", len(dists))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for d_idx, dist in enumerate(dists):
            means = [data_filtered[(data_filtered['Dist'] == dist) & (data_filtered['Var'] == v)][model].mean() for v in vars_val]
            ax.plot(vars_val, means, marker='s', label=dist, color=colors_dist[d_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Varianza')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Distribución", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.8{suffix}_interaccion_dist_var_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 9 (5.8) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 5 (Lista 10): CONFIG × PASO - Subplots por Modelo
def plot_interaction_config_paso(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Config × Paso por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    configs = sorted(data_filtered['Config'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_conf = sns.color_palette("tab10", len(configs))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for c_idx, config in enumerate(configs):
            means = [data_filtered[(data_filtered['Config'] == config) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='^', label=config, color=colors_conf[c_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Config", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.9{suffix}_interaccion_config_paso_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 10 (5.9) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 6 (Lista 11): VAR × HORIZONTE - Subplots por Modelo
def plot_interaction_var_horizonte(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Var × Horizonte por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    vars_val = sorted(data_filtered['Var'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_var = sns.color_palette("rocket", len(vars_val))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for v_idx, var in enumerate(vars_val):
            means = [data_filtered[(data_filtered['Var'] == var) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='d', label=f'Var {var}', color=colors_var[v_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Varianza", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.10{suffix}_interaccion_var_horizonte_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 11 (5.10) {suffix} guardada: Subplots por Modelo")


# ====================================================================================
# EJECUCIÓN DE LAS NUEVAS FUNCIONES
# ====================================================================================

for sc_name, sc_suf in [ (None, ''), ('Lineal Estacionario (ARMA)', '.a'), 
                        ('Lineal No Estacionario (ARIMA)', '.b'), 
                        ('No lineal Estacionario (SETAR)', '.c') ]:
    plot_interaction_dist_paso(sc_name, sc_suf)
    plot_interaction_dist_var(sc_name, sc_suf)
    plot_interaction_config_paso(sc_name, sc_suf)
    plot_interaction_var_horizonte(sc_name, sc_suf)

# ====================================================================================
# SECCIÓN 6: ROBUSTEZ Y TEST DIEBOLD-MARIANO POR ESCENARIO
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 6: ROBUSTEZ Y TEST DIEBOLD-MARIANO POR ESCENARIO")
print("="*80)

def plot_robustness():
    fig, ax = plt.subplots(figsize=(12, 8))
    
    cv_data = []
    for model in model_cols:
        cv = df[model].std() / df[model].mean()
        cv_data.append((model, cv))
    
    cv_df = pd.DataFrame(cv_data, columns=['Modelo', 'CV'])
    
    # Ordenar de menor a mayor CV para coherencia
    cv_df = cv_df.sort_values('CV')
    
    colors_cv = ['#2ecc71' if cv < cv_df['CV'].median() else '#e74c3c' 
                 for cv in cv_df['CV']]
    
    bars = ax.barh(cv_df['Modelo'], cv_df['CV'], color=colors_cv, alpha=0.8, edgecolor='black')
    
    for bar, cv in zip(bars, cv_df['CV']):
        width = bar.get_width()
        ax.text(width + 0.001, bar.get_y() + bar.get_height()/2.,
               f'{cv:.4f}', ha='left', va='center', fontsize=9)
    
    ax.set_xlabel('Coeficiente de Variación', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    ax.set_title('Robustez: Coeficiente de Variación\n(Ordenado de menor a mayor - Menor valor indica mayor estabilidad)', 
                  fontsize=14, fontweight='bold', pad=20)
    ax.axvline(x=cv_df['CV'].median(), color='black', linestyle='--', linewidth=1.5, alpha=0.5, label='Mediana')
    ax.legend(loc='best', fontsize=10)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '6.1_robustez_coeficiente_variacion.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 6.1 guardada (ordenada de menor a mayor CV)")

plot_robustness()

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano con fixed-smoothing asymptotics (Coroneo & Iacone, 2020)
    """
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    if T < 2:
        return np.nan, np.nan, np.nan
    
    u = d - d_bar
    m = max(1, int(np.floor(T**(1/3))))
    
    from scipy.fft import fft
    fft_u = fft(u)
    periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
    
    if m >= len(periodogram) - 1:
        m = len(periodogram) - 2
    
    sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
    
    if sigma_hat_sq <= 0:
        sigma_hat_sq = np.var(d, ddof=1) / T
        if sigma_hat_sq <= 0:
            return 0, 1.0, 0
    
    dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
    df = 2 * m
    hln_dm_stat = dm_stat
    p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat


def create_scenario_description(row):
    """
    Crea una descripción del escenario basada en sus características
    """
    escenario = row['ESCENARIO']
    config = row['Config']
    dist = row['Dist']
    var = row['Var']
    
    # Mapear nombres completos
    escenario_map = {
        'Lineal Estacionario (ARMA)': 'ARMA',
        'Lineal No Estacionario (ARIMA)': 'ARIMA',
        'No lineal Estacionario (SETAR)': 'SETAR'
    }
    
    escenario_short = escenario_map.get(escenario, escenario)
    
    return f"{escenario_short}_Config{config}_Dist{dist}_Var{var}"


def dm_test_by_scenarios(scenario_filter=None, suffix=''):
    """
    Realiza el test DM por cada escenario individual con corrección de Bonferroni
    
    Parameters:
    -----------
    scenario_filter : str or None
        Si se especifica ('Lineal Estacionario (ARMA)', etc.), filtra solo esos escenarios
    suffix : str
        Sufijo para los archivos generados
    """
    print("\n" + "-"*80)
    if scenario_filter:
        print(f"ANÁLISIS DM PARA: {scenario_filter}")
    else:
        print("ANÁLISIS DM GENERAL (TODOS LOS ESCENARIOS)")
    print("-"*80)
    
    # Filtrar datos si es necesario
    if scenario_filter:
        data_filtered = df[df['ESCENARIO'] == scenario_filter].copy()
    else:
        data_filtered = df.copy()
    
    # Crear identificador único para cada escenario
    data_filtered['Escenario_ID'] = data_filtered.apply(create_scenario_description, axis=1)
    
    # Obtener lista de escenarios únicos
    unique_scenarios = data_filtered['Escenario_ID'].unique()
    n_scenarios = len(unique_scenarios)
    n_models = len(model_cols)
    
    print(f"\nTotal de escenarios únicos: {n_scenarios}")
    print(f"Total de modelos: {n_models}")
    print(f"Total de comparaciones por escenario: {n_models * (n_models - 1) // 2}")
    
    # Matriz para almacenar todos los p-valores
    all_pvalues = []
    scenario_names = []
    comparison_names = []
    
    # Matriz para contar cuántas veces la media de fila > columna
    all_mean_comparisons = []
    
    # Generar nombres de comparaciones
    for i in range(n_models):
        for j in range(i+1, n_models):
            comparison_names.append(f"{model_cols[i]} vs {model_cols[j]}")
    
    # Realizar test DM para cada escenario
    for scenario_id in unique_scenarios:
        scenario_data = data_filtered[data_filtered['Escenario_ID'] == scenario_id]
        
        # Extraer características del escenario
        scenario_info = scenario_data.iloc[0]
        
        pvalues_row = []
        mean_comp_row = []
        
        # Comparar todos los pares de modelos
        for i in range(n_models):
            for j in range(i+1, n_models):
                model1 = model_cols[i]
                model2 = model_cols[j]
                
                errors1 = scenario_data[model1].values
                errors2 = scenario_data[model2].values
                
                h_forecast = int(scenario_info['Paso'])
                
                _, p_val, _ = modified_diebold_mariano_test(errors1, errors2, h=h_forecast)
                pvalues_row.append(p_val)
                
                # Comparar medias (1 si media1 > media2, 0 si no)
                mean1 = np.mean(errors1)
                mean2 = np.mean(errors2)
                mean_comp_row.append(1 if mean1 > mean2 else 0)
        
        all_pvalues.append(pvalues_row)
        all_mean_comparisons.append(mean_comp_row)
        scenario_names.append(scenario_id)
    
    # Crear DataFrame con todos los p-valores
    pvalues_df = pd.DataFrame(all_pvalues, columns=comparison_names, index=scenario_names)
    mean_comp_df = pd.DataFrame(all_mean_comparisons, columns=comparison_names, index=scenario_names)
    
    # Guardar en Excel
    excel_filename = f'6.2{suffix}_dm_test_por_escenarios.xlsx'
    with pd.ExcelWriter(output_dir / excel_filename) as writer:
        pvalues_df.to_excel(writer, sheet_name='P_valores_por_escenario')
        
        # Agregar hoja con información de escenarios
        scenario_info_list = []
        for scenario_id in scenario_names:
            scenario_data = data_filtered[data_filtered['Escenario_ID'] == scenario_id].iloc[0]
            scenario_info_list.append({
                'Escenario_ID': scenario_id,
                'Tipo': scenario_data['ESCENARIO'],
                'Config': scenario_data['Config'],
                'Dist': scenario_data['Dist'],
                'Var': scenario_data['Var'],
                'Paso': scenario_data['Paso']
            })
        scenario_info_df = pd.DataFrame(scenario_info_list)
        scenario_info_df.to_excel(writer, sheet_name='Info_Escenarios', index=False)
    
    print(f"✓ Tabla de p-valores guardada: {n_scenarios} escenarios × {len(comparison_names)} comparaciones")
    
    # BOXPLOT DE LOS 3 MENORES Y 3 MAYORES P-VALORES
    plot_extreme_pvalues_boxplot(pvalues_df, comparison_names, suffix)
    
    # MATRIZ DE PORCENTAJE DE SIGNIFICANCIA CON BONFERRONI Y % MEDIA MAYOR
    create_significance_matrix(pvalues_df, mean_comp_df, comparison_names, n_scenarios, suffix)
    
    return pvalues_df, comparison_names


def plot_extreme_pvalues_boxplot(pvalues_df, comparison_names, suffix=''):
    """
    Crea boxplots de las 3 comparaciones con menores y mayores p-valores promedio
    """
    print("\n" + "-"*80)
    print("GENERANDO BOXPLOTS DE P-VALORES EXTREMOS")
    print("-"*80)
    
    # Determinar tipo de análisis
    analysis_type = {
        '': 'GENERAL',
        '.a': 'ARMA',
        '.b': 'ARIMA',
        '.c': 'SETAR'
    }
    tipo = analysis_type.get(suffix, 'GENERAL')
    
    # Calcular p-valor promedio para cada comparación
    mean_pvalues = pvalues_df.mean().sort_values()
    
    # Seleccionar 3 menores y 3 mayores
    lowest_3 = mean_pvalues.head(3)
    highest_3 = mean_pvalues.tail(3)
    
    selected_comparisons = list(lowest_3.index) + list(highest_3.index)
    
    print(f"\n3 Comparaciones con MENORES p-valores promedio:")
    for comp in lowest_3.index:
        print(f"  {comp}: {lowest_3[comp]:.4f}")
    
    print(f"\n3 Comparaciones con MAYORES p-valores promedio:")
    for comp in highest_3.index:
        print(f"  {comp}: {highest_3[comp]:.4f}")
    
    # Crear boxplot
    fig, ax = plt.subplots(figsize=(14, 8))
    
    data_to_plot = [pvalues_df[comp].values for comp in selected_comparisons]
    
    bp = ax.boxplot(data_to_plot, labels=selected_comparisons, patch_artist=True,
                    showmeans=True, meanline=True)
    
    # Colorear: menores en verde, mayores en rojo
    colors = ['#2ecc71'] * 3 + ['#e74c3c'] * 3
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
    
    # Calcular alpha de Bonferroni
    n_comparisons = len(comparison_names)
    alpha_bonferroni = 0.05 / n_comparisons
    
    # Línea de significancia
    ax.axhline(y=0.05, color='black', linestyle='--', linewidth=2, label='α = 0.05 (nominal)')
    ax.axhline(y=alpha_bonferroni, color='red', linestyle='--', linewidth=1.5, 
               label=f'α = {alpha_bonferroni:.5f} (Bonferroni)', alpha=0.7)
    
    ax.set_xlabel('Comparación entre Modelos', fontsize=12, fontweight='bold')
    ax.set_ylabel('P-valor', fontsize=12, fontweight='bold')
    ax.set_title(f'Distribución de P-valores [{tipo}]: 3 Menores y 3 Mayores Comparaciones\n(Menor p-valor = Mayor evidencia de diferencia significativa)', 
                 fontsize=14, fontweight='bold', pad=20)
    ax.set_xticklabels(selected_comparisons, rotation=45, ha='right', fontsize=9)
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Añadir anotaciones de media
    medians = [np.median(data) for data in data_to_plot]
    for i, (median, comp) in enumerate(zip(medians, selected_comparisons)):
        ax.text(i+1, median, f'{median:.3f}', ha='center', va='bottom', fontsize=8, fontweight='bold')
    
    plt.tight_layout()
    filename = f'6.3{suffix}_boxplot_pvalores_extremos.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Gráfica 6.3{suffix} guardada: Boxplot de p-valores extremos [{tipo}]")


def create_significance_matrix(pvalues_df, mean_comp_df, comparison_names, n_scenarios, suffix=''):
    """
    Crea matriz de % de veces que el p-valor es significativo con Bonferroni
    y % de veces que la media de fila < columna (F < C)
    """
    print("\n" + "-"*80)
    print("GENERANDO MATRIZ DE PORCENTAJE DE SIGNIFICANCIA CON BONFERRONI")
    print("-"*80)
    
    # Determinar tipo de análisis
    analysis_type = {
        '': 'GENERAL',
        '.a': 'ARMA',
        '.b': 'ARIMA',
        '.c': 'SETAR'
    }
    tipo = analysis_type.get(suffix, 'GENERAL')
    
    alpha = 0.05
    n_models = len(model_cols)
    n_comparisons = len(comparison_names)
    alpha_bonferroni = alpha / n_comparisons
    
    print(f"Tipo de análisis: {tipo}")
    print(f"α nominal: {alpha}")
    print(f"α Bonferroni: {alpha_bonferroni:.5f}")
    
    # Matriz de porcentaje de significancia
    significance_matrix = np.zeros((n_models, n_models))
    
    # Matriz de porcentaje de veces que media fila < media columna
    mean_lower_matrix = np.zeros((n_models, n_models))
    
    # Procesar cada comparación
    for comparison in comparison_names:
        # Extraer nombres de modelos
        parts = comparison.split(' vs ')
        model1 = parts[0]
        model2 = parts[1]
        
        idx1 = model_cols.index(model1)
        idx2 = model_cols.index(model2)
        
        # Calcular porcentaje de veces que es significativo (Bonferroni)
        pvalues = pvalues_df[comparison].values
        significant_count = np.sum(pvalues < alpha_bonferroni)
        percentage_sig = (significant_count / len(pvalues)) * 100
        
        # Calcular porcentaje de veces que media1 < media2
        # Asumiendo que mean_comp_df tiene 1 si m1 > m2 y 0 si m1 < m2
        mean_comparisons = mean_comp_df[comparison].values
        percentage_mean1_greater = (np.sum(mean_comparisons) / len(mean_comparisons)) * 100
        
        # Invertimos la lógica para obtener F < C
        percentage_mean1_lower = 100 - percentage_mean1_greater
        percentage_mean2_lower = percentage_mean1_greater # Porque si m1 > m2, entonces m2 < m1
        
        # Llenar matrices
        significance_matrix[idx1, idx2] = percentage_sig
        significance_matrix[idx2, idx1] = percentage_sig
        
        mean_lower_matrix[idx1, idx2] = percentage_mean1_lower
        mean_lower_matrix[idx2, idx1] = percentage_mean2_lower
    
    # Crear DataFrames
    significance_df = pd.DataFrame(significance_matrix, 
                                   index=model_cols, 
                                   columns=model_cols)
    
    mean_lower_df = pd.DataFrame(mean_lower_matrix,
                                 index=model_cols,
                                 columns=model_cols)
    
    # Crear matriz combinada para anotaciones
    combined_annot = np.empty((n_models, n_models), dtype=object)
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                combined_annot[i, j] = '-'
            else:
                sig_val = significance_matrix[i, j]
                mean_val = mean_lower_matrix[i, j]
                
                # Cambiado a F < C
                combined_annot[i, j] = f'Sig: {sig_val:.1f}%\nF<C: {mean_val:.1f}%'
    
    # Visualizar como heatmap
    fig, ax = plt.subplots(figsize=(16, 14))
    
    # Usamos RdYlGn (Rojo a Verde). En error, F < C alto es bueno (Verde)
    sns.heatmap(significance_df, annot=combined_annot, fmt='', cmap='RdYlGn', 
                center=50, vmin=0, vmax=100,
                cbar_kws={'label': '% Significancia (p < α Bonferroni)'},
                ax=ax, linewidths=0.5, linecolor='gray', annot_kws={'fontsize': 8})
    
    title_text = f'Matriz de Significancia Estadística [{tipo}]\n'
    title_text += f'Sig: % Significancia (α={alpha_bonferroni:.5f}) | F<C: % Fila < Columna (Mejor)'
    
    ax.set_title(title_text, fontsize=13, fontweight='bold', pad=20)
    ax.set_xlabel('Modelo (Columna)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo (Fila)', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'6.4{suffix}_matriz_significancia_porcentual.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    
    # Guardar en Excel
    excel_filename = f'6.4{suffix}_matriz_significancia.xlsx'
    with pd.ExcelWriter(output_dir / excel_filename) as writer:
        significance_df.to_excel(writer, sheet_name='Porcentaje_Significancia')
        mean_lower_df.to_excel(writer, sheet_name='Porcentaje_Media_Menor')
        
        # Resumen estadístico
        summary_stats = []
        for model in model_cols:
            idx = model_cols.index(model)
            sig_values = [significance_matrix[idx, j] for j in range(n_models) if j != idx]
            mean_values = [mean_lower_matrix[idx, j] for j in range(n_models) if j != idx]
            
            summary_stats.append({
                'Modelo': model,
                'Significancia_Promedio_%': np.mean(sig_values),
                'Media_Menor_Promedio_% (F<C)': np.mean(mean_values),
                'Media_Menor_Mediana_% (F<C)': np.median(mean_values),
                'ECRPS_Promedio': df[model].mean() if suffix == '' else df[df['ESCENARIO'] == {'': '', '.a': 'Lineal Estacionario (ARMA)', '.b': 'Lineal No Estacionario (ARIMA)', '.c': 'No lineal Estacionario (SETAR)'}[suffix]][model].mean()
            })
        
        summary_df = pd.DataFrame(summary_stats)
        summary_df = summary_df.sort_values('ECRPS_Promedio')
        summary_df.to_excel(writer, sheet_name='Resumen_por_Modelo', index=False)
        
        # Información metodológica
        method_info = pd.DataFrame({
            'Parámetro': ['Tipo de análisis', 'Alpha nominal', 'Alpha Bonferroni', 
                          'Número comparaciones', 'Número escenarios', 'Test estadístico', 'Métrica Comparativa'],
            'Valor': [tipo, alpha, alpha_bonferroni, n_comparisons, n_scenarios, 
                      'Diebold-Mariano modificado (HLN)', 'F < C (Fila menor que Columna)']
        })
        method_info.to_excel(writer, sheet_name='Metodologia', index=False)
    
    print(f"✓ Gráfica 6.4{suffix} guardada: Matriz de significancia (F<C) [{tipo}]")
    print(f"✓ Excel 6.4{suffix} guardado: Matriz y resumen estadístico [{tipo}]")
    
    return significance_df, mean_lower_df


# EJECUTAR ANÁLISIS DM: GENERAL Y POR FAMILIA
print("\n" + "="*80)
print("EJECUTANDO ANÁLISIS DM: GENERAL Y POR FAMILIA DE PROCESOS")
print("="*80)

# 1. Análisis General
dm_test_by_scenarios(scenario_filter=None, suffix='')

# 2. Análisis por ARMA
dm_test_by_scenarios(scenario_filter='Lineal Estacionario (ARMA)', suffix='.a')

# 3. Análisis por ARIMA
dm_test_by_scenarios(scenario_filter='Lineal No Estacionario (ARIMA)', suffix='.b')

# 4. Análisis por SETAR
dm_test_by_scenarios(scenario_filter='No lineal Estacionario (SETAR)', suffix='.c')

print("\n" + "="*80)
print("PROCESO COMPLETADO")
print("="*80)
print("\nMejoras implementadas:")
print("1. ✓ Orden consistente en gráficas 1.1 y 1.2")
print("2. ✓ Formato de 2 decimales en heatmap 2.2 general")
print("3. ✓ Solo Coeficiente de Variación en gráfica 6.1 (ordenado menor a mayor)")
print("4. ✓ Test Diebold-Mariano POR ESCENARIO INDIVIDUAL")
print("5. ✓ Tabla de p-valores por escenario (Escenarios × Comparaciones)")
print("6. ✓ Boxplot de 3 menores y 3 mayores p-valores")
print("7. ✓ Matriz de % de significancia estadística")
print("\n" + "="*80)
print("ANÁLISIS DE INTERACCIONES (Carpeta: Interacciones/)")
print("="*80)
print("8. ✓ Config × Var: Heatmaps por modelo (matriz 3×3)")
print("9. ✓ Config × Dist: Heatmaps por modelo (matriz 3×3)")
print("10. ✓ Dist × Paso: Gráficas de líneas por distribución")
print("11. ✓ Dist × Var: Gráficas de líneas por distribución")
print("12. ✓ Config × Paso: Gráficas de líneas por configuración")
print("13. ✓ Var × Horizonte: Gráficas de líneas por varianza")
print("="*80)
print(f"\nARCHIVOS GENERADOS:")
print(f"  - 6.2_dm_test_por_escenarios.xlsx: Tabla completa de p-valores")
print(f"  - 6.3_boxplot_pvalores_extremos.png: Distribución de p-valores extremos")
print(f"  - 6.4_matriz_significancia_porcentual.png: Heatmap de % significancia")
print(f"  - 6.4_matriz_significancia.xlsx: Matriz y resumen estadístico")
print("="*80)

Nuevo orden de modelos (basado en Lineal No Estacionario (ARIMA)):
1. Sieve Bootstrap: 0.5475
2. LSPMW: 1.0636
3. LSPM: 1.0648
4. MCPS: 3.2318
5. AV-MCPS: 3.3418
6. DeepAR: 4.3291
7. EnCQR-LSTM: 5.8505
8. AREPD: 10.0312
9. Block Bootstrapping: 11.2519

SECCIÓN 1: RENDIMIENTO POR ESCENARIOS
✓ Gráfica 1.1 guardada
✓ Gráfica 1.2 guardada

SECCIÓN 2: ANÁLISIS POR CONFIG
✓ Gráfica 2.1 guardada
✓ Gráfica 2.1.a guardada
✓ Gráfica 2.1.b guardada
✓ Gráfica 2.1.c guardada
✓ Gráfica 2.2 guardada
✓ Gráfica 2.2.a guardada
✓ Gráfica 2.2.b guardada
✓ Gráfica 2.2.c guardada

SECCIÓN 3: ANÁLISIS POR DIST
✓ Gráfica 3.1 guardada
✓ Gráfica 3.1.a guardada
✓ Gráfica 3.1.b guardada
✓ Gráfica 3.1.c guardada
✓ Gráfica 3.2 guardada
✓ Gráfica 3.2.a guardada
✓ Gráfica 3.2.b guardada
✓ Gráfica 3.2.c guardada

SECCIÓN 4: ANÁLISIS POR VAR
✓ Gráfica 4.1 guardada
✓ Gráfica 4.1.a guardada
✓ Gráfica 4.1.b guardada
✓ Gráfica 4.1.c guardada
✓ Gráfica 4.2 guardada
✓ Gráfica 4.2.a guardada
✓ Gráfica 4.2.b guardada
✓ Gráfica

## Analisis Diferenciado

### Pre-procesamiento

In [23]:
import pandas as pd
import numpy as np

# 1. Cargar los archivos
path_diff = "./datos/Simulacion/Diferenciado/resultados_140_ARIMA_CON_DIFERENCIACION.xlsx"
path_no_diff = "./datos/Simulacion/Diferenciado/resultados_140_ARIMA_FINAL.xlsx"

arima_Diff_df = pd.read_excel(path_diff)
arima_df = pd.read_excel(path_no_diff)

# 2. Agregar la columna 'Diferenciacion' al dataframe que no la tiene
arima_df['Diferenciacion'] = 'No'

# 3. Unir los dataframes (uno debajo del otro)
# El orden de las columnas se ajustará automáticamente
df_unido = pd.concat([arima_Diff_df, arima_df], ignore_index=True)

# 4. Guardar la unión en la misma carpeta
path_salida = "./datos/Simulacion/Diferenciado/resultados_UNION_ARIMA.xlsx"
df_unido.to_excel(path_salida, index=False)

# 5. Crear la tabla comparativa (Resumen)
# Definimos las métricas/modelos a comparar
modelos_metricas = [
    'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 
    'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap'
]

# Calculamos el promedio para cada caso
resumen_no_diff = arima_df[modelos_metricas].mean()
resumen_diff = arima_Diff_df[modelos_metricas].mean()

# Construir el DataFrame comparativo
tabla_comparativa = pd.DataFrame({
    'ARIMA': resumen_no_diff,
    'ARIMA_Diff': resumen_diff
})

# Determinar el mejor escenario (el que tenga el valor menor, asumiendo que son errores)
tabla_comparativa['Mejor_Escenario'] = np.where(
    tabla_comparativa['ARIMA'] < tabla_comparativa['ARIMA_Diff'], 
    'ARIMA', 
    'ARIMA_Diff'
)

# Formatear la tabla para que se vea limpia
tabla_comparativa.index.name = 'Modelo'
tabla_comparativa = tabla_comparativa.sort_index()

# Mostrar resultados
print("Tabla Comparativa de Desempeño (Promedios):")
print(tabla_comparativa.to_string())

print(f"\nArchivo de unión guardado en: {path_salida}")

Tabla Comparativa de Desempeño (Promedios):
                         ARIMA  ARIMA_Diff Mejor_Escenario
Modelo                                                    
AREPD                10.031183    0.704149      ARIMA_Diff
AV-MCPS               3.324007    0.654257      ARIMA_Diff
Block Bootstrapping  11.251601    0.666133      ARIMA_Diff
DeepAR                4.329124    0.561822      ARIMA_Diff
EnCQR-LSTM            6.112344    0.880288      ARIMA_Diff
LSPM                  1.064804    0.648039      ARIMA_Diff
LSPMW                 3.079645    0.767172      ARIMA_Diff
MCPS                  3.218168    0.677471      ARIMA_Diff
Sieve Bootstrap       0.547481    0.546020      ARIMA_Diff

Archivo de unión guardado en: ./datos/Simulacion/Diferenciado/resultados_UNION_ARIMA.xlsx


### Analisis general

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.fft import fft
from pathlib import Path
import warnings

# Configuración inicial
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'

# ====================================================================================
# TEST DIEBOLD-MARIANO CON FIXED-M ASYMPTOTICS
# ====================================================================================
def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano con fixed-smoothing asymptotics (Coroneo & Iacone, 2020)
    
    Usa Fixed-m asymptotics (kernel Daniell) que es más robusto en muestras pequeñas
    y mantiene buen desempeño en muestras grandes.
    
    Parameters:
    -----------
    errors1, errors2 : array-like
        Errores de pronóstico (ECRPS) de los dos modelos
    h : int
        Horizonte de pronóstico (forecast horizon)
    
    Returns:
    --------
    hln_dm_stat : float
        Estadístico con fixed-m asymptotics
    p_value : float
        P-valor usando distribución t-Student con 2m grados de libertad
    dm_stat : float
        Estadístico DM original (para referencia)
    """
    # Calcular diferencial de pérdida
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    if T < 2:
        return np.nan, np.nan, np.nan
    
    # Desviaciones de la media
    u = d - d_bar
    
    # Fixed-m: bandwidth recomendado en el paper
    m = max(1, int(np.floor(T**(1/3))))
    
    # Calcular periodograma
    fft_u = fft(u)
    periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
    
    # WPE con kernel de Daniell: promedio de primeros m periodogramas
    # (excluyendo frecuencia 0)
    if m >= len(periodogram) - 1:
        m = len(periodogram) - 2
    
    sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
    
    if sigma_hat_sq <= 0:
        # Fallback: usar varianza simple
        sigma_hat_sq = np.var(d, ddof=1) / T
        if sigma_hat_sq <= 0:
            return 0, 1.0, 0
    
    # Estadístico DM
    dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
    
    # Fixed-m asymptotics: límite es t-Student con 2m grados de libertad
    df = 2 * m
    hln_dm_stat = dm_stat
    
    # P-valor usando t-Student
    p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat

# ====================================================================================
# 1. PREPARACIÓN DE DIRECTORIOS
# ====================================================================================
output_dir = Path("./Resultados_analisis/Simulacion_diff")
output_dir.mkdir(parents=True, exist_ok=True)
plots_dir = output_dir / "Graficos_Analisis"
plots_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
path_excel = "./datos/Simulacion/Diferenciado/resultados_140_ARIMA_AMBAS_MODALIDADES.xlsx"
try:
    df = pd.read_excel(path_excel)
    print(f"✓ Datos cargados: {df.shape}")
    print(f"✓ Columnas: {df.columns.tolist()}")
except Exception as e:
    print(f"Error crítico: {e}")
    exit()

# Identificar columnas
var_cols = ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 'Distribución', 
            'Varianza', 'Modalidad', 'Valor_Observado']
model_cols = [col for col in df.columns if col not in var_cols]

print(f"✓ Modelos identificados: {model_cols}")
print(f"✓ Valores únicos de d: {sorted(df['d'].unique())}")

# ====================================================================================
# FUNCIÓN PRINCIPAL: ANÁLISIS MISMO MÉTODO CON/SIN DIFF POR ESCENARIO
# ====================================================================================
def run_within_method_comparison_by_scenario():
    """
    Compara MISMO método en MISMO escenario: Con_Diff vs Sin_Diff
    NUNCA compara entre métodos diferentes
    
    Formato salida:
    Método | ECRPS sin diff | ECRPS con diff | Cambio % | % Escenarios Significativos
    """
    print("\n" + "="*80)
    print("ANÁLISIS: MISMO MÉTODO, CON DIFF VS SIN DIFF POR ESCENARIO")
    print("CORRECCIÓN DE BONFERRONI APLICADA")
    print("="*80)
    
    # Identificar escenarios únicos (combinaciones únicas de características)
    scenario_cols = ['d', 'ARMA_base', 'Distribución', 'Varianza']
    df['Escenario_ID'] = df[scenario_cols].astype(str).agg('|'.join, axis=1)
    
    unique_scenarios = df['Escenario_ID'].unique()
    n_scenarios = len(unique_scenarios)
    
    # Corrección de Bonferroni
    alpha_nominal = 0.05
    alpha_bonferroni = alpha_nominal / n_scenarios
    
    print(f"\n✓ Escenarios únicos detectados: {n_scenarios}")
    print(f"✓ α nominal: {alpha_nominal}")
    print(f"✓ α Bonferroni: {alpha_bonferroni:.6f}")
    print(f"✓ Criterio: p < {alpha_bonferroni:.6f} para significancia\n")
    
    # Almacenar resultados finales por método
    final_results = []
    
    # Procesar cada método independientemente
    for model in model_cols:
        print(f"\n{'='*60}")
        print(f"Procesando: {model}")
        print(f"{'='*60}")
        
        scenario_tests = []
        
        # Para cada escenario, comparar Con_Diff vs Sin_Diff del MISMO método
        for scenario_id in unique_scenarios:
            scenario_data = df[df['Escenario_ID'] == scenario_id]
            
            # Extraer errores del MISMO método en ambas modalidades
            sin_diff_errors = scenario_data[scenario_data['Modalidad'] == 'SIN_DIFF'][model].values
            con_diff_errors = scenario_data[scenario_data['Modalidad'] == 'CON_DIFF'][model].values
            
            # Validar que hay datos suficientes
            if len(sin_diff_errors) < 2 or len(con_diff_errors) < 2:
                continue
            
            # Calcular promedios del escenario
            mean_sin = sin_diff_errors.mean()
            mean_con = con_diff_errors.mean()
            
            # Test DM: comparando MISMO método Con vs Sin Diff
            dm_stat, p_value, _ = modified_diebold_mariano_test(sin_diff_errors, con_diff_errors, h=1)
            
            # ¿Es significativo con corrección Bonferroni?
            is_significant_bonferroni = p_value < alpha_bonferroni
            
            # Guardar resultado del escenario
            scenario_tests.append({
                'escenario': scenario_id,
                'mean_sin_diff': mean_sin,
                'mean_con_diff': mean_con,
                'dm_stat': dm_stat,
                'p_value': p_value,
                'significant_bonferroni': is_significant_bonferroni
            })
        
        # Convertir a DataFrame
        scenario_df = pd.DataFrame(scenario_tests)
        
        # Calcular métricas agregadas
        n_total_scenarios = len(scenario_df)
        n_significant = scenario_df['significant_bonferroni'].sum()
        pct_significant = (n_significant / n_total_scenarios * 100) if n_total_scenarios > 0 else 0
        
        # Promedios globales (todos los escenarios)
        global_mean_sin = scenario_df['mean_sin_diff'].mean()
        global_mean_con = scenario_df['mean_con_diff'].mean()
        global_change_pct = ((global_mean_sin - global_mean_con) / global_mean_sin * 100) if global_mean_sin != 0 else 0
        
        # Almacenar resultado del método
        final_results.append({
            'Método': model,
            'ECRPS_Sin_Diff': global_mean_sin,
            'ECRPS_Con_Diff': global_mean_con,
            'Cambio_%': global_change_pct,
            '%_Escenarios_Significativos': pct_significant,
            'N_Escenarios_Significativos': int(n_significant),
            'N_Escenarios_Total': n_total_scenarios
        })
        
        print(f"  ✓ Escenarios evaluados: {n_total_scenarios}")
        print(f"  ✓ Escenarios significativos (Bonferroni): {n_significant} ({pct_significant:.1f}%)")
        print(f"  ✓ ECRPS promedio Sin Diff: {global_mean_sin:.4f}")
        print(f"  ✓ ECRPS promedio Con Diff: {global_mean_con:.4f}")
        print(f"  ✓ Cambio porcentual: {global_change_pct:+.2f}%")
    
    # Crear DataFrame final con resumen por método
    summary_df = pd.DataFrame(final_results)
    
    # Ordenar por cambio porcentual (mejores primero)
    summary_df = summary_df.sort_values('Cambio_%', ascending=False)
    
    # Guardar Excel con formato profesional
    excel_path = output_dir / 'RESUMEN_ConDiff_vs_SinDiff_por_Metodo_Bonferroni.xlsx'
    
    with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
        # Hoja principal: Resumen
        summary_df.to_excel(writer, sheet_name='Resumen_Por_Metodo', index=False)
        
        # Obtener objetos de formato
        workbook = writer.book
        worksheet = writer.sheets['Resumen_Por_Metodo']
        
        # Formatos
        fmt_header = workbook.add_format({
            'bold': True,
            'bg_color': '#4472C4',
            'font_color': 'white',
            'align': 'center',
            'valign': 'vcenter',
            'border': 1,
            'text_wrap': True
        })
        
        fmt_green = workbook.add_format({
            'bg_color': '#C6EFCE',
            'font_color': '#006100',
            'num_format': '0.00'
        })
        
        fmt_red = workbook.add_format({
            'bg_color': '#FFC7CE',
            'font_color': '#9C0006',
            'num_format': '0.00'
        })
        
        fmt_number = workbook.add_format({'num_format': '0.0000'})
        fmt_percent = workbook.add_format({'num_format': '0.00"%"'})
        
        # Aplicar formato condicional a Cambio_%
        cambio_col_idx = summary_df.columns.get_loc('Cambio_%')
        worksheet.conditional_format(
            1, cambio_col_idx, len(summary_df), cambio_col_idx,
            {'type': 'cell', 'criteria': '>', 'value': 0, 'format': fmt_green}
        )
        worksheet.conditional_format(
            1, cambio_col_idx, len(summary_df), cambio_col_idx,
            {'type': 'cell', 'criteria': '<', 'value': 0, 'format': fmt_red}
        )
        
        # Formato condicional a % Escenarios Significativos (escala de color)
        sig_col_idx = summary_df.columns.get_loc('%_Escenarios_Significativos')
        worksheet.conditional_format(
            1, sig_col_idx, len(summary_df), sig_col_idx,
            {
                'type': '3_color_scale',
                'min_color': '#F8696B',
                'mid_color': '#FFEB84',
                'max_color': '#63BE7B'
            }
        )
        
        # Ajustar anchos de columna
        worksheet.set_column('A:A', 22)  # Método
        worksheet.set_column('B:D', 16)  # ECRPS y Cambio
        worksheet.set_column('E:E', 28)  # % Escenarios Significativos
        worksheet.set_column('F:G', 22)  # N Escenarios
        
        # Hoja 2: Información metodológica
        method_info = pd.DataFrame({
            'Parámetro': [
                'Tipo de comparación',
                'Alpha nominal',
                'Alpha Bonferroni',
                'Número de escenarios',
                'Test estadístico',
                'Corrección múltiple',
                'Interpretación Cambio %',
                'Interpretación % Sig'
            ],
            'Valor': [
                'Mismo método: Con Diff vs Sin Diff en cada escenario',
                alpha_nominal,
                f'{alpha_bonferroni:.6f}',
                n_scenarios,
                'Diebold-Mariano modificado (Fixed-m asymptotics)',
                'Bonferroni',
                'Positivo = Con Diff mejora | Negativo = Sin Diff mejor',
                'Porcentaje de escenarios donde diferencia es estadísticamente significativa'
            ]
        })
        method_info.to_excel(writer, sheet_name='Metodologia', index=False)
        
        worksheet_info = writer.sheets['Metodologia']
        worksheet_info.set_column('A:A', 25)
        worksheet_info.set_column('B:B', 60)
    
    print(f"\n{'='*80}")
    print(f"✓ RESUMEN GUARDADO EN: {excel_path}")
    print(f"{'='*80}\n")
    
    return summary_df

# ====================================================================================
# FUNCIÓN: TABLA DETALLADA POR MÉTODO Y ESCENARIO
# ====================================================================================
def generate_detailed_scenario_breakdown():
    """
    Genera tabla detallada mostrando cada escenario individual por método
    """
    print("\n" + "="*80)
    print("GENERANDO TABLA DETALLADA POR ESCENARIO")
    print("="*80)
    
    # Identificar escenarios
    scenario_cols = ['d', 'ARMA_base', 'Distribución', 'Varianza']
    df['Escenario_ID'] = df[scenario_cols].astype(str).agg('|'.join, axis=1)
    
    unique_scenarios = df['Escenario_ID'].unique()
    n_scenarios = len(unique_scenarios)
    alpha_bonferroni = 0.05 / n_scenarios
    
    excel_path = output_dir / 'DETALLE_Escenarios_ConDiff_vs_SinDiff_Bonferroni.xlsx'
    
    with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
        for model in model_cols:
            print(f"  Procesando detalle de: {model}")
            
            detailed_rows = []
            
            for scenario_id in unique_scenarios:
                scenario_data = df[df['Escenario_ID'] == scenario_id]
                
                # Obtener características del escenario
                d_val = scenario_data['d'].iloc[0]
                arma = scenario_data['ARMA_base'].iloc[0]
                dist = scenario_data['Distribución'].iloc[0]
                var_val = scenario_data['Varianza'].iloc[0]
                
                # Comparar MISMO método: Con vs Sin Diff
                sin_diff = scenario_data[scenario_data['Modalidad'] == 'SIN_DIFF'][model].values
                con_diff = scenario_data[scenario_data['Modalidad'] == 'CON_DIFF'][model].values
                
                if len(sin_diff) < 2 or len(con_diff) < 2:
                    continue
                
                mean_sin = sin_diff.mean()
                mean_con = con_diff.mean()
                cambio_pct = ((mean_sin - mean_con) / mean_sin * 100) if mean_sin != 0 else 0
                
                dm_stat, p_value, _ = modified_diebold_mariano_test(sin_diff, con_diff, h=1)
                is_sig = p_value < alpha_bonferroni
                
                detailed_rows.append({
                    'd': d_val,
                    'ARMA_base': arma,
                    'Distribución': dist,
                    'Varianza': var_val,
                    'ECRPS_Sin_Diff': mean_sin,
                    'ECRPS_Con_Diff': mean_con,
                    'Cambio_%': cambio_pct,
                    'DM_Stat': dm_stat,
                    'p_value': p_value,
                    'Sig_Bonferroni': 'Sí' if is_sig else 'No'
                })
            
            detail_df = pd.DataFrame(detailed_rows)
            detail_df = detail_df.sort_values(['d', 'Cambio_%'], ascending=[True, False])
            
            sheet_name = model[:31]
            detail_df.to_excel(writer, sheet_name=sheet_name, index=False)
            
            # Formato condicional
            workbook = writer.book
            worksheet = writer.sheets[sheet_name]
            
            fmt_green = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
            fmt_red = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
            
            cambio_col = detail_df.columns.get_loc('Cambio_%')
            worksheet.conditional_format(
                1, cambio_col, len(detail_df), cambio_col,
                {'type': 'cell', 'criteria': '>', 'value': 0, 'format': fmt_green}
            )
            worksheet.conditional_format(
                1, cambio_col, len(detail_df), cambio_col,
                {'type': 'cell', 'criteria': '<', 'value': 0, 'format': fmt_red}
            )
    
    print(f"\n✓ DETALLE GUARDADO EN: {excel_path}\n")

# ====================================================================================
# FUNCIÓN: GRÁFICO DE RESUMEN
# ====================================================================================
def plot_comparison_summary(summary_df):
    """
    Genera gráfico dual mostrando:
    1. Cambio porcentual por método
    2. % de escenarios significativos por método
    """
    print("\n=== Generando gráfico de resumen ===")
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
    
    # Panel 1: Cambio Porcentual
    colors_cambio = ['#2ecc71' if x > 0 else '#e74c3c' for x in summary_df['Cambio_%']]
    bars1 = ax1.barh(summary_df['Método'], summary_df['Cambio_%'],
                     color=colors_cambio, edgecolor='black', linewidth=1.2, alpha=0.85)
    
    ax1.bar_label(bars1, padding=5, fmt='%.2f%%', fontweight='bold', fontsize=9)
    ax1.axvline(0, color='black', linewidth=2, linestyle='--', alpha=0.7)
    ax1.set_xlabel('Cambio Porcentual (%)', fontweight='bold', fontsize=12)
    ax1.set_title('Mejora con Diferenciación\n(Con Diff vs Sin Diff)', 
                  fontweight='bold', fontsize=13)
    ax1.grid(axis='x', alpha=0.3, linestyle='--')
    ax1.set_axisbelow(True)
    
    # Panel 2: % Escenarios Significativos
    colors_sig = plt.cm.RdYlGn(summary_df['%_Escenarios_Significativos'] / 100)
    bars2 = ax2.barh(summary_df['Método'], summary_df['%_Escenarios_Significativos'],
                     color=colors_sig, edgecolor='black', linewidth=1.2, alpha=0.85)
    
    ax2.bar_label(bars2, padding=5, fmt='%.1f%%', fontweight='bold', fontsize=9)
    ax2.set_xlabel('% Escenarios Significativos (Bonferroni)', fontweight='bold', fontsize=12)
    ax2.set_title('Robustez Estadística\n(p < α Bonferroni)', 
                  fontweight='bold', fontsize=13)
    ax2.set_xlim(0, 100)
    ax2.grid(axis='x', alpha=0.3, linestyle='--')
    ax2.set_axisbelow(True)
    
    plt.tight_layout()
    plt.savefig(plots_dir / 'RESUMEN_ConDiff_vs_SinDiff_Comparacion.png', 
                dpi=300, bbox_inches='tight')
    plt.close()
    
    print("✓ Gráfico de resumen guardado\n")

# ====================================================================================
# FUNCIONES ORIGINALES MANTENIDAS (OPCIONAL)
# ====================================================================================
def plot_individual_comparisons():
    print("\n=== Generando gráficos de barras con valores por d ===")
    
    d_values = sorted(df['d'].unique())
    
    for d_val in d_values:
        stats_list = []
        subset_d = df[df['d'] == d_val]
        
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].mean()
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].mean()
            mejora = ((sin - con) / sin) * 100 if sin != 0 else 0
            stats_list.append({'Modelo': model, 'Sin': sin, 'Con': con, 'Mejora': mejora})
        
        df_stats = pd.DataFrame(stats_list).sort_values(by='Mejora', ascending=False)
        
        fig, ax = plt.subplots(figsize=(14, 6))
        x = np.arange(len(df_stats))
        width = 0.35
        
        b1 = ax.bar(x - width/2, df_stats['Sin'], width, label='Sin Dif.', 
                    color='#e74c3c', edgecolor='black', alpha=0.8)
        b2 = ax.bar(x + width/2, df_stats['Con'], width, label='Con Dif.', 
                    color='#2ecc71', edgecolor='black', alpha=0.8)
        
        ax.bar_label(b1, padding=3, fmt='%.3f', fontsize=7, rotation=45)
        ax.bar_label(b2, padding=3, fmt='%.3f', fontsize=7, rotation=45)
        
        ax.set_title(f'ECRPS Promedio por Modelo (d={d_val})', fontweight='bold', fontsize=12)
        ax.set_xticks(x)
        ax.set_xticklabels(df_stats['Modelo'], rotation=45, ha='right')
        ax.set_ylabel('ECRPS')
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.savefig(plots_dir / f'1.1_barras_ecrps_d{d_val}.png')
        plt.close()

def plot_heatmap_d_vs_modelo():
    print("\n=== Generando Heatmap d vs Modelo ===")
    
    results = []
    d_values = sorted(df['d'].unique())
    
    for d_val in d_values:
        subset_d = df[df['d'] == d_val]
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].mean()
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].mean()
            mejora = ((sin - con) / sin) * 100 if sin != 0 else 0
            results.append({'d': d_val, 'Modelo': model, 'Mejora_%': mejora})
    
    df_mejora = pd.DataFrame(results)
    pivot = df_mejora.pivot(index='Modelo', columns='d', values='Mejora_%')
    
    plt.figure(figsize=(14, 10))
    sns.heatmap(pivot, annot=True, fmt=".1f", cmap="RdYlGn", center=0, 
                cbar_kws={'label': 'Mejora (%)'}, linewidths=0.5)
    plt.title("Mejora Porcentual (%): Modelo vs Valor de d", fontweight='bold', fontsize=14)
    plt.xlabel("Valor de d (Grado de diferenciación)", fontweight='bold')
    plt.ylabel("Modelo", fontweight='bold')
    plt.tight_layout()
    plt.savefig(plots_dir / '2_heatmap_d_vs_modelo.png')
    plt.close()
    
    pivot.to_excel(output_dir / '2_tabla_d_vs_modelo.xlsx')

# ====================================================================================
# EJECUCIÓN PRINCIPAL
# ====================================================================================
if __name__ == "__main__":
    print("\n" + "="*80)
    print("ANÁLISIS: MISMO MÉTODO CON DIFF VS SIN DIFF")
    print("CORRECCIÓN DE BONFERRONI POR ESCENARIOS")
    print("="*80)
    
    # FUNCIÓN PRINCIPAL: Análisis mismo método Con vs Sin Diff
    summary_df = run_within_method_comparison_by_scenario()
    
    # Gráfico de resumen
    plot_comparison_summary(summary_df)
    
    # Tabla detallada por escenario
    generate_detailed_scenario_breakdown()
    
    # Análisis complementarios (opcional)
    plot_individual_comparisons()
    plot_heatmap_d_vs_modelo()
    
    print("\n" + "="*80)
    print("✓ ANÁLISIS COMPLETADO EXITOSAMENTE")
    print(f"✓ Resultados en: {output_dir}")
    print(f"✓ Gráficos en: {plots_dir}")
    print("="*80 + "\n")

✓ Datos cargados: (3360, 19)
✓ Columnas: ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 'Distribución', 'Varianza', 'Modalidad', 'Valor_Observado', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
✓ Modelos identificados: ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
✓ Valores únicos de d: [np.int64(1)]

ANÁLISIS: MISMO MÉTODO CON DIFF VS SIN DIFF
CORRECCIÓN DE BONFERRONI POR ESCENARIOS

ANÁLISIS: MISMO MÉTODO, CON DIFF VS SIN DIFF POR ESCENARIO
CORRECCIÓN DE BONFERRONI APLICADA

✓ Escenarios únicos detectados: 140
✓ α nominal: 0.05
✓ α Bonferroni: 0.000357
✓ Criterio: p < 0.000357 para significancia


Procesando: AREPD
  ✓ Escenarios evaluados: 140
  ✓ Escenarios significativos (Bonferroni): 59 (42.1%)
  ✓ ECRPS promedio Sin Diff: 10.0312
  ✓ ECRPS promedio Con Diff: 0.7046
  ✓ Cambio porcentual: +92.98%

Procesando: AV-MCPS
  ✓ Escenarios evaluados: 140
  ✓

## Aumento d ARIMA

### Pre-procesamiento

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.fft import fft
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# Configuración de gráficos
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'

# ====================================================================================
# CREAR DIRECTORIOS
# ====================================================================================
output_dir = Path("./Resultados_analisis/Sensibilidad_d")
output_dir.mkdir(parents=True, exist_ok=True)
plots_dir = output_dir / "Graficos"
plots_dir.mkdir(parents=True, exist_ok=True)

# ====================================================================================
# TEST DIEBOLD-MARIANO CON FIXED-M ASYMPTOTICS
# ====================================================================================
def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano con fixed-smoothing asymptotics
    """
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    if T < 2:
        return np.nan, np.nan, np.nan
    
    u = d - d_bar
    m = max(1, int(np.floor(T**(1/3))))
    
    fft_u = fft(u)
    periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
    
    if m >= len(periodogram) - 1:
        m = len(periodogram) - 2
    
    sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
    
    if sigma_hat_sq <= 0:
        sigma_hat_sq = np.var(d, ddof=1) / T
        if sigma_hat_sq <= 0:
            return 0, 1.0, 0
    
    dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
    df = 2 * m
    hln_dm_stat = dm_stat
    p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat

# ====================================================================================
# CARGAR DATOS
# ====================================================================================
print("\n" + "="*80)
print("ANÁLISIS DE SENSIBILIDAD AL INCREMENTO DE d")
print("="*80)

base_path = "./datos/Simulacion/Multi_D/resultados_ARIMA_d1_a_d10_DOBLE_MODALIDAD_COMPLETO.xlsx"
base = pd.read_excel(base_path)

# Modelos a analizar
modelos_analizar = ['Sieve Bootstrap', 'LSPM', 'LSPMW', 'AV-MCPS', 'MCPS']

print(f"\n✓ Datos cargados: {base.shape}")
print(f"✓ Modelos a analizar: {modelos_analizar}")

# Identificar columnas de características
caracteristicas = ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 
                   'Distribución', 'Varianza', 'Modalidad']

# ====================================================================================
# FUNCIÓN: ANÁLISIS POR ESCENARIOS CON DM TEST
# ====================================================================================
def analizar_modelo_por_escenarios(datos, modelo_nombre):
    """
    Analiza un modelo específico comparando Con Diff vs Sin Diff por escenario
    Retorna matriz de p-valores: escenarios x valores de d
    """
    print(f"\n{'='*60}")
    print(f"Analizando: {modelo_nombre}")
    print(f"{'='*60}")
    
    # Columnas necesarias
    cols_necesarias = caracteristicas + [modelo_nombre]
    datos_modelo = datos[cols_necesarias].copy()
    
    # Crear identificador de escenario (sin incluir d)
    escenario_cols = ['ARMA_base', 'Distribución', 'Varianza']
    datos_modelo['Escenario'] = datos_modelo[escenario_cols].astype(str).agg('-', axis=1)
    
    # Obtener valores únicos
    valores_d = sorted(datos_modelo['d'].unique())
    escenarios = sorted(datos_modelo['Escenario'].unique())
    
    n_escenarios = len(escenarios)
    n_valores_d = len(valores_d)
    
    print(f"  ✓ Valores de d: {valores_d}")
    print(f"  ✓ Número de escenarios: {n_escenarios}")
    
    # Matriz de p-valores
    matriz_pvalores = np.full((n_escenarios, n_valores_d), np.nan)
    
    # Matriz de diferencias medias (para contexto)
    matriz_diferencias = np.full((n_escenarios, n_valores_d), np.nan)
    
    # Iterar por cada combinación de escenario y d
    for i, escenario in enumerate(escenarios):
        for j, d_val in enumerate(valores_d):
            # Filtrar datos
            mask = (datos_modelo['Escenario'] == escenario) & (datos_modelo['d'] == d_val)
            subset = datos_modelo[mask]
            
            if len(subset) == 0:
                continue
            
            # Separar por modalidad
            sin_diff = subset[subset['Modalidad'] == 'SIN_DIFF'][modelo_nombre].values
            con_diff = subset[subset['Modalidad'] == 'CON_DIFF'][modelo_nombre].values
            
            if len(sin_diff) < 2 or len(con_diff) < 2:
                continue
            
            # Test DM
            _, p_value, _ = modified_diebold_mariano_test(sin_diff, con_diff, h=1)
            
            # Almacenar
            matriz_pvalores[i, j] = p_value
            matriz_diferencias[i, j] = sin_diff.mean() - con_diff.mean()
    
    # Crear DataFrames
    df_pvalores = pd.DataFrame(
        matriz_pvalores,
        index=escenarios,
        columns=[f'd={d}' for d in valores_d]
    )
    
    df_diferencias = pd.DataFrame(
        matriz_diferencias,
        index=escenarios,
        columns=[f'd={d}' for d in valores_d]
    )
    
    return df_pvalores, df_diferencias, valores_d, escenarios

# ====================================================================================
# FUNCIÓN: GENERAR HEATMAP DE P-VALORES
# ====================================================================================
def plot_pvalor_heatmap(df_pvalores, modelo_nombre, valores_d, alpha=0.05):
    """
    Genera heatmap de p-valores con corrección Bonferroni
    """
    n_comparisons = df_pvalores.notna().sum().sum()
    alpha_bonferroni = alpha / n_comparisons if n_comparisons > 0 else alpha
    
    fig, ax = plt.subplots(figsize=(14, max(10, len(df_pvalores) * 0.3)))
    
    # Crear máscara para valores faltantes
    mask = df_pvalores.isna()
    
    # Heatmap con escala logarítmica invertida
    # Valores pequeños (significativos) en verde, grandes en rojo
    sns.heatmap(
        -np.log10(df_pvalores + 1e-300),  # Transformación logarítmica
        mask=mask,
        cmap='RdYlGn',
        center=-np.log10(alpha_bonferroni),
        cbar_kws={'label': '-log₁₀(p-valor)'},
        linewidths=0.5,
        linecolor='gray',
        ax=ax,
        vmin=-np.log10(0.5),
        vmax=-np.log10(1e-10),
        annot=False  # Sin anotaciones para evitar saturación
    )
    
    # Línea de referencia para α Bonferroni
    cbar = ax.collections[0].colorbar
    cbar.ax.axhline(-np.log10(alpha_bonferroni), color='blue', linewidth=2, linestyle='--')
    cbar.ax.text(
        0.5, -np.log10(alpha_bonferroni), 
        f'  α_Bonf={alpha_bonferroni:.2e}',
        va='center', ha='left', color='blue', fontweight='bold', fontsize=8
    )
    
    # Títulos y etiquetas
    ax.set_title(
        f'Matriz de P-valores: {modelo_nombre}\n'
        f'Con Diff vs Sin Diff por Escenario y Valor de d\n'
        f'(Verde = Significativo | Rojo = No Significativo)',
        fontsize=13, fontweight='bold', pad=15
    )
    ax.set_xlabel('Valor de d', fontsize=11, fontweight='bold')
    ax.set_ylabel('Escenario (Config-Dist-Var)', fontsize=11, fontweight='bold')
    
    # Rotar etiquetas
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha='center')
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=7)
    
    plt.tight_layout()
    
    # Guardar
    filename = f'HEATMAP_pvalores_{modelo_nombre.replace(" ", "_")}.png'
    plt.savefig(plots_dir / filename, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  ✓ Heatmap guardado: {filename}")

# ====================================================================================
# FUNCIÓN: GRÁFICO RESUMEN DE SIGNIFICANCIA
# ====================================================================================
def plot_resumen_significancia(resultados_resumen, alpha=0.05):
    """
    Gráfico de barras mostrando % de escenarios significativos por modelo y d
    """
    fig, ax = plt.subplots(figsize=(16, 8))
    
    modelos = list(resultados_resumen.keys())
    valores_d = list(resultados_resumen[modelos[0]].keys())
    
    x = np.arange(len(valores_d))
    width = 0.15
    
    for i, modelo in enumerate(modelos):
        porcentajes = [resultados_resumen[modelo][d] for d in valores_d]
        offset = width * (i - len(modelos)/2 + 0.5)
        
        bars = ax.bar(x + offset, porcentajes, width, label=modelo, alpha=0.85, edgecolor='black')
        
        # Etiquetas en barras (solo si > 5%)
        for bar in bars:
            height = bar.get_height()
            if height > 5:
                ax.text(
                    bar.get_x() + bar.get_width()/2., height,
                    f'{height:.0f}%',
                    ha='center', va='bottom', fontsize=7, fontweight='bold'
                )
    
    ax.set_xlabel('Valor de d', fontsize=12, fontweight='bold')
    ax.set_ylabel('% Escenarios Significativos', fontsize=12, fontweight='bold')
    ax.set_title(
        'Porcentaje de Escenarios con Diferencias Significativas\n'
        f'(Con Diff vs Sin Diff, α Bonferroni por modelo)',
        fontsize=13, fontweight='bold'
    )
    ax.set_xticks(x)
    ax.set_xticklabels([f'd={d}' for d in valores_d])
    ax.legend(loc='upper right', fontsize=9)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_ylim(0, 100)
    
    plt.tight_layout()
    plt.savefig(plots_dir / 'RESUMEN_Significancia_por_d.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("\n✓ Gráfico de resumen guardado")

# ====================================================================================
# FUNCIÓN: GRÁFICO DE DIFERENCIAS PROMEDIO POR d (MEJORADO)
# ====================================================================================
def plot_diferencias_promedio_mejorado(base, modelos_analizar):
    """
    Gráfico de líneas mejorado mostrando diferencia promedio por d
    Evita superposición de etiquetas
    """
    fig, ax = plt.subplots(figsize=(14, 8))
    
    valores_d = sorted(base['d'].unique())
    
    for modelo in modelos_analizar:
        diferencias_por_d = []
        
        for d_val in valores_d:
            subset = base[base['d'] == d_val]
            sin_diff = subset[subset['Modalidad'] == 'SIN_DIFF'][modelo].mean()
            con_diff = subset[subset['Modalidad'] == 'CON_DIFF'][modelo].mean()
            dif = sin_diff - con_diff
            diferencias_por_d.append(dif)
        
        # Línea con marcadores
        ax.plot(valores_d, diferencias_por_d, marker='o', linewidth=2.5, 
                markersize=8, label=modelo, alpha=0.85)
    
    ax.axhline(0, color='black', linestyle='--', linewidth=1.5, alpha=0.7)
    ax.set_xlabel('Valor de d', fontsize=12, fontweight='bold')
    ax.set_ylabel('Diferencia ECRPS (Sin Diff - Con Diff)', fontsize=12, fontweight='bold')
    ax.set_title(
        'Evolución de la Mejora con Diferenciación según Valor de d\n'
        '(Valores positivos indican mejora con diferenciación)',
        fontsize=13, fontweight='bold'
    )
    ax.legend(loc='best', fontsize=10, framealpha=0.95)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xticks(valores_d)
    
    plt.tight_layout()
    plt.savefig(plots_dir / 'DIFERENCIAS_promedio_por_d_MEJORADO.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("✓ Gráfico de diferencias mejorado guardado")

# ====================================================================================
# FUNCIÓN: TABLA RESUMEN EN EXCEL
# ====================================================================================
def guardar_resumen_excel(todos_resultados, output_dir):
    """
    Guarda resumen consolidado en Excel
    """
    excel_path = output_dir / 'RESUMEN_Analisis_Sensibilidad_d.xlsx'
    
    with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
        for modelo, (df_pvalores, df_diferencias) in todos_resultados.items():
            # Hoja de p-valores
            sheet_name_p = f'{modelo[:25]}_pvalores'
            df_pvalores.to_excel(writer, sheet_name=sheet_name_p)
            
            # Hoja de diferencias
            sheet_name_d = f'{modelo[:25]}_diferencias'
            df_diferencias.to_excel(writer, sheet_name=sheet_name_d)
            
            # Formato condicional en p-valores
            workbook = writer.book
            worksheet_p = writer.sheets[sheet_name_p]
            
            # Formato para p < 0.05
            fmt_sig = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
            
            # Aplicar formato (aproximado, necesitaría cálculo exacto de Bonferroni)
            for row in range(1, len(df_pvalores) + 1):
                for col in range(1, len(df_pvalores.columns) + 1):
                    worksheet_p.conditional_format(
                        row, col, row, col,
                        {'type': 'cell', 'criteria': '<', 'value': 0.05, 'format': fmt_sig}
                    )
    
    print(f"\n✓ Excel guardado: {excel_path}")

# ====================================================================================
# EJECUCIÓN PRINCIPAL
# ====================================================================================
if __name__ == "__main__":
    
    # Almacenar resultados
    todos_resultados = {}
    resultados_resumen = {}
    
    # Analizar cada modelo
    for modelo in modelos_analizar:
        df_pvalores, df_diferencias, valores_d, escenarios = analizar_modelo_por_escenarios(
            base, modelo
        )
        
        # Guardar resultados
        todos_resultados[modelo] = (df_pvalores, df_diferencias)
        
        # Generar heatmap
        plot_pvalor_heatmap(df_pvalores, modelo, valores_d)
        
        # Calcular % significativos por d
        alpha = 0.05
        n_comparisons = df_pvalores.notna().sum().sum()
        alpha_bonferroni = alpha / n_comparisons if n_comparisons > 0 else alpha
        
        resultados_resumen[modelo] = {}
        for col in df_pvalores.columns:
            n_sig = (df_pvalores[col] < alpha_bonferroni).sum()
            n_total = df_pvalores[col].notna().sum()
            pct = (n_sig / n_total * 100) if n_total > 0 else 0
            d_val = int(col.split('=')[1])
            resultados_resumen[modelo][d_val] = pct
    
    # Gráficos complementarios
    plot_resumen_significancia(resultados_resumen)
    plot_diferencias_promedio_mejorado(base, modelos_analizar)
    
    # Guardar Excel
    guardar_resumen_excel(todos_resultados, output_dir)
    
    print("\n" + "="*80)
    print("✓ ANÁLISIS COMPLETADO")
    print(f"✓ Resultados en: {output_dir}")
    print(f"✓ Gráficos en: {plots_dir}")
    print("="*80 + "\n")


TEST DIEBOLD-MARIANO MODIFICADO (HLN): Comparación SIN_DIFF vs CON_DIFF por valor de d

H0: No hay diferencia significativa entre las modalidades
H1: Hay diferencia significativa entre las modalidades

Significancia: *** p<0.01, ** p<0.05, * p<0.10, No = no significativo


 d  N_obs  ECRPS_SIN_DIFF  ECRPS_CON_DIFF    Diferencia  DM_stat  HLN-DM_stat  p_valor Significativo
 1   1680    5.495070e-01    5.485800e-01  9.280000e-04   0.5334       0.5334   0.6067            No
 2   1680    2.884661e+01    2.883279e+01  1.382000e-02   0.8679       0.8679   0.4022            No
 3   1680    2.725984e+01    2.726032e+01 -4.790000e-04  -0.0186      -0.0186   0.9856            No
 4   1680    2.704997e+01    2.704544e+01  4.532000e-03   0.1216       0.1216   0.9066            No
 5   1680    3.451632e+01    2.821125e+01  6.305071e+00   1.8941       1.8941   0.0676             *
 6   1680    1.400979e+03    2.663349e+01  1.374346e+03   4.4940       4.4940   0.0000           ***
 7   1680    9.737

In [3]:
import pandas as pd
import numpy as np
from scipy import stats

# Cargar datos
base = pd.read_excel("./datos/Simulacion/Diferenciado/resultados_UNION_ARIMA.xlsx")

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano con fixed-smoothing asymptotics (Coroneo & Iacone, 2020)
    
    Implementa dos enfoques:
    1. Fixed-b asymptotics con kernel de Bartlett
    2. Fixed-m asymptotics con kernel de Daniell
    
    Parameters:
    -----------
    errors1, errors2 : array-like
        Errores de pronóstico (ECRPS) de los dos modelos
    h : int
        Horizonte de pronóstico (forecast horizon)
    
    Returns:
    --------
    hln_dm_stat : float
        Estadístico con fixed-smoothing asymptotics
    p_value : float
        P-valor usando distribución apropiada
    dm_stat : float
        Estadístico DM original (para referencia)
    """
    # Calcular diferencial de pérdida
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    if T < 2:
        return np.nan, np.nan, np.nan
    
    # Desviaciones de la media
    u = d - d_bar
    
    # Elegir método según tamaño de muestra
    if T >= 80:
        # Fixed-b con kernel de Bartlett para muestras medianas/grandes
        M = int(np.floor(T**(1/2)))  # Bandwidth recomendado
        b = M / T
        
        # Calcular WCE con kernel de Bartlett
        gamma_0 = np.mean(u**2)
        gamma_sum = gamma_0
        
        for j in range(1, min(M, T)):
            gamma_j = np.mean(u[j:] * u[:-j])
            weight = 1 - j/M  # Kernel de Bartlett
            gamma_sum += 2 * weight * gamma_j
        
        sigma_hat_sq = gamma_sum
        
        if sigma_hat_sq <= 0:
            return 0, 1.0, 0
        
        # Estadístico DM
        dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
        
        # Valor crítico fixed-b para Bartlett kernel (fórmula del paper)
        # Para test de dos colas al 5%
        alpha_0, alpha_1, alpha_2, alpha_3 = 1.9600, 2.9694, 0.4160, -0.5324
        critical_value = alpha_0 + alpha_1*b + alpha_2*b**2 + alpha_3*b**3
        
        # P-valor aproximado usando la distribución límite
        # Nota: esto es una aproximación, idealmente se simularía
        p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat) / (critical_value/1.96)))
        
        hln_dm_stat = dm_stat
        
    else:
        # Fixed-m con kernel de Daniell para muestras pequeñas
        m = int(np.floor(T**(1/3)))  # Bandwidth recomendado
        
        # Calcular periodograma
        from scipy.fft import fft
        
        # FFT de las desviaciones
        fft_u = fft(u)
        periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
        
        # WPE con kernel de Daniell (promedio de primeros m periodogramas)
        sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
        
        if sigma_hat_sq <= 0:
            return 0, 1.0, 0
        
        # Estadístico DM
        dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
        
        # Fixed-m asymptotics: límite es t-Student con 2m grados de libertad
        df = 2 * m
        hln_dm_stat = dm_stat
        
        # P-valor usando t-Student
        p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat


# Separar por diferenciación
sin_diff = base[base['Diferenciacion'] == 'No'].copy()
con_diff = base[base['Diferenciacion'] == 'Si'].copy()

# Métodos a comparar (todos excepto las columnas de configuración)
metodos = ['Block Bootstrapping', 'Sieve Bootstrap', 'LSPM', 'LSPMW', 
           'AREPD', 'MCPS', 'AV-MCPS', 'DeepAR', 'EnCQR-LSTM']

# Lista para almacenar resultados
resultados = []

print("\n" + "="*120)
print("TEST DIEBOLD-MARIANO MODIFICADO (HLN): Comparación Sin Diferenciación (No) vs Con Diferenciación (Si)")
print("="*120)
print("\nH0: No hay diferencia significativa entre aplicar o no diferenciación")
print("H1: Hay diferencia significativa entre aplicar o no diferenciación")
print("\nSignificancia: *** p<0.01, ** p<0.05, * p<0.10, No = no significativo")
print("="*120)

# Iterar sobre cada método
for metodo in metodos:
    # Obtener valores de ECRPS para cada modalidad
    ecrps_sin = sin_diff[metodo].values
    ecrps_con = con_diff[metodo].values
    
    # Verificar que ambas modalidades tengan datos
    if len(ecrps_sin) == 0 or len(ecrps_con) == 0:
        print(f"\nAdvertencia: {metodo} no tiene datos para ambas modalidades")
        continue
    
    # Verificar que tengan la misma longitud
    if len(ecrps_sin) != len(ecrps_con):
        print(f"\nAdvertencia: {metodo} tiene diferente número de observaciones")
        min_len = min(len(ecrps_sin), len(ecrps_con))
        ecrps_sin = ecrps_sin[:min_len]
        ecrps_con = ecrps_con[:min_len]
    
    # Calcular estadísticas descriptivas
    ecrps_sin_mean = np.mean(ecrps_sin)
    ecrps_con_mean = np.mean(ecrps_con)
    diferencia = ecrps_sin_mean - ecrps_con_mean
    
    # Realizar test Diebold-Mariano modificado
    hln_dm_stat, p_value, dm_stat = modified_diebold_mariano_test(ecrps_sin, ecrps_con, h=1)
    
    # Determinar significancia
    if p_value < 0.01:
        significativo = "***"
    elif p_value < 0.05:
        significativo = "**"
    elif p_value < 0.10:
        significativo = "*"
    else:
        significativo = "No"
    
    # Agregar a resultados
    resultados.append({
        'Método': metodo,
        'N_obs': len(ecrps_sin),
        'ECRPS_Sin_Diff': ecrps_sin_mean,
        'ECRPS_Con_Diff': ecrps_con_mean,
        'Diferencia': diferencia,
        'DM_stat': dm_stat,
        'HLN-DM_stat': hln_dm_stat,
        'p_valor': p_value,
        'Significativo': significativo
    })

# Crear DataFrame con resultados
resultados_df = pd.DataFrame(resultados)

# Formatear para mejor visualización
resultados_df['ECRPS_Sin_Diff'] = resultados_df['ECRPS_Sin_Diff'].round(6)
resultados_df['ECRPS_Con_Diff'] = resultados_df['ECRPS_Con_Diff'].round(6)
resultados_df['Diferencia'] = resultados_df['Diferencia'].round(6)
resultados_df['DM_stat'] = resultados_df['DM_stat'].round(4)
resultados_df['HLN-DM_stat'] = resultados_df['HLN-DM_stat'].round(4)
resultados_df['p_valor'] = resultados_df['p_valor'].round(4)

# Mostrar resultados
print("\n")
print(resultados_df.to_string(index=False))

# Resumen de resultados
print("\n" + "="*120)
print("RESUMEN")
print("="*120)
n_significativos = len(resultados_df[resultados_df['Significativo'] != 'No'])
n_total = len(resultados_df)
print(f"\nTotal de métodos comparados: {n_total}")
print(f"Diferencias significativas: {n_significativos} ({100*n_significativos/n_total:.1f}%)")
print(f"No significativas: {n_total - n_significativos} ({100*(n_total-n_significativos)/n_total:.1f}%)")

# Mostrar cuáles métodos tienen diferencias significativas
if n_significativos > 0:
    print("\nMétodos con diferencias significativas:")
    metodos_sig = resultados_df[resultados_df['Significativo'] != 'No'][['Método', 'Significativo', 'p_valor']]
    print(metodos_sig.to_string(index=False))


TEST DIEBOLD-MARIANO MODIFICADO (HLN): Comparación Sin Diferenciación (No) vs Con Diferenciación (Si)

H0: No hay diferencia significativa entre aplicar o no diferenciación
H1: Hay diferencia significativa entre aplicar o no diferenciación

Significancia: *** p<0.01, ** p<0.05, * p<0.10, No = no significativo


             Método  N_obs  ECRPS_Sin_Diff  ECRPS_Con_Diff  Diferencia  DM_stat  HLN-DM_stat  p_valor Significativo
Block Bootstrapping   1680       11.251601        0.666133   10.585468   5.9736       5.9736   0.0000           ***
    Sieve Bootstrap   1680        0.547481        0.546020    0.001461   1.1882       1.1882   0.2515            No
               LSPM   1680        1.064804        0.648039    0.416766  14.2307      14.2307   0.0000           ***
              LSPMW   1680        3.079645        0.767172    2.312473  10.3870      10.3870   0.0000           ***
              AREPD   1680       10.031183        0.704149    9.327035   5.7364       5.7364   0.0000     

### Analisis general

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
import warnings

# Configuración inicial
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'

# ====================================================================================
# 1. PREPARACIÓN DE DIRECTORIOS
# ====================================================================================
output_dir = Path("./Resultados_analisis/Multi_d")
output_dir.mkdir(parents=True, exist_ok=True)
plots_dir = output_dir / "Graficos_Analisis"
plots_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
path_excel = "./datos/Simulacion/Multi_D/resultados_ARIMA_d1_a_d10_DOBLE_MODALIDAD_COMPLETO.xlsx"
try:
    df = pd.read_excel(path_excel)
    print(f"✓ Datos cargados: {df.shape}")
    print(f"✓ Columnas: {df.columns.tolist()}")
except Exception as e:
    print(f"Error crítico: {e}")
    exit()

# Identificar columnas
var_cols = ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 'Distribución',
            'Varianza', 'Modalidad', 'Valor_Observado']
model_cols = [col for col in df.columns if col not in var_cols]
print(f"✓ Modelos identificados: {model_cols}")
print(f"✓ Valores únicos de d: {sorted(df['d'].unique())}")


# ====================================================================================
# TEST DE DIEBOLD-MARIANO CORREGIDO
# ====================================================================================
def diebold_mariano_test(loss1, loss2, h=1):
    """
    Test Diebold-Mariano correcto.

    Recibe dos arrays de PÉRDIDAS por observación (ej. ECRPS por paso/réplica)
    del mismo largo. Calcula el diferencial d_t = loss1_t - loss2_t y testea
    H0: E[d_t] = 0  vs  H1: E[d_t] != 0.

    Usa HAC (Newey-West) con bandwidth = h-1 para series de h pasos adelante,
    que es la corrección estándar para correlación serial inducida por el horizonte.

    DIFERENCIAS respecto al código original (que tenía 3 bugs):
    -------------------------------------------------------
    Bug 1 (original): Se calculaba sigma_hat_sq sobre las DESVIACIONES u = d - d_bar
                      en lugar de sobre la serie diferencial d_t completa.
                      → Cuando d_bar >> 0 (caso de interés), las desviaciones son
                        pequeñas y sigma se subestima, inflando el estadístico
                        artificialmente O deflacionándolo según el paso del FFT.

    Bug 2 (original): La varianza espectral usaba solo los primeros m periodogramas
                      del FFT de u, no de d. Esto es una estimación no estándar
                      que mezcla la media con la varianza de manera incorrecta.

    Bug 3 (original): El estadístico resultante a veces era enorme (rechaza cuando
                      no debería) y a veces cercano a cero (no rechaza cuando sí
                      debería), porque u ≈ 0 cuando loss1 >> loss2 implica que
                      todas las desviaciones de la media son similares entre sí,
                      haciendo que sigma → 0 y el test colapsa numéricamente.

    Solución: HAC estándar (Newey-West) directamente sobre d_t = loss1_t - loss2_t.
    """
    loss1 = np.asarray(loss1, dtype=float)
    loss2 = np.asarray(loss2, dtype=float)

    if len(loss1) != len(loss2):
        raise ValueError("Las series deben tener el mismo largo")

    T = len(loss1)
    if T < 4:
        return np.nan, np.nan

    # Diferencial de pérdidas (positivo = loss1 peor que loss2)
    d = loss1 - loss2
    d_bar = np.mean(d)

    # --- Varianza HAC Newey-West ---
    # La varianza de d_bar bajo correlación serial es:
    # V = (1/T) * [ gamma_0 + 2 * sum_{k=1}^{K} w_k * gamma_k ]
    # donde gamma_k es la autocovarianza muestral de d en lag k
    # y w_k = 1 - k/(K+1) son los pesos de Bartlett.
    # K = max(h-1, 1) es el bandwidth: para h=1 se usa solo gamma_0 (sin
    # corrección de autocorrelación), que es el estimador DM original.
    # Para h>1 se corrige por la autocorrelación inducida por el solapamiento.

    K = max(h - 1, 0)  # bandwidth (lags a incluir)
    d_centered = d - d_bar  # desviaciones para autocovarianzas

    # gamma_0: varianza muestral
    gamma_0 = np.dot(d_centered, d_centered) / T

    hac_var = gamma_0
    for k in range(1, K + 1):
        gamma_k = np.dot(d_centered[k:], d_centered[:-k]) / T
        w_k = 1.0 - k / (K + 1.0)  # peso de Bartlett
        hac_var += 2.0 * w_k * gamma_k

    # Protección numérica: si la varianza es no positiva, usar gamma_0
    if hac_var <= 0:
        hac_var = gamma_0
    if hac_var <= 0:
        # Caso degenerado: todas las diferencias son idénticas
        return 0.0, 1.0

    # Estadístico DM
    dm_stat = d_bar / np.sqrt(hac_var / T)

    # Corrección de muestra pequeña Harvey, Leybourne & Newbold (1997):
    # multiplica el estadístico por sqrt((T + 1 - 2h + h(h-1)/T) / T)
    # para mejorar la aproximación en muestras finitas.
    hln_factor = np.sqrt((T + 1 - 2 * h + h * (h - 1) / T) / T)
    dm_stat_hln = dm_stat * hln_factor

    # P-valor: t-Student con T-1 grados de libertad (HLN) o normal estándar
    p_value = 2.0 * (1.0 - stats.t.cdf(abs(dm_stat_hln), df=T - 1))

    return dm_stat_hln, p_value


# ====================================================================================
# ANÁLISIS 1: MEJORA PORCENTUAL POR DIFERENCIACIÓN (HEATMAP POR d)
# ====================================================================================
def plot_mejora_porcentual_por_d():
    """
    Heatmap de mejora porcentual (CON_DIFF vs SIN_DIFF) para cada modelo por d
    """
    print("\n=== ANÁLISIS 1: Mejora Porcentual por d ===")
    d_values = sorted(df['d'].unique())

    # Calcular mejoras para cada modelo y d
    mejoras = []
    for model in model_cols:
        mejoras_model = []
        for d_val in d_values:
            subset_d = df[df['d'] == d_val]
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].mean()
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].mean()
            mejora_pct = ((sin - con) / sin) * 100 if sin != 0 else 0
            mejoras_model.append(mejora_pct)
        mejoras.append(mejoras_model)

    df_mejora = pd.DataFrame(mejoras, index=model_cols, columns=[f'd={d}' for d in d_values])

    # Crear heatmap
    plt.figure(figsize=(16, 12))
    sns.heatmap(df_mejora, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
                cbar_kws={'label': 'Mejora (%)'}, linewidths=0.8,
                vmin=-20, vmax=20)
    plt.title('Mejora Porcentual: CON_DIFF vs SIN_DIFF por Orden de Diferenciación',
              fontweight='bold', fontsize=16, pad=20)
    plt.xlabel('Orden de Diferenciación (d)', fontweight='bold', fontsize=13)
    plt.ylabel('Modelo', fontweight='bold', fontsize=13)
    plt.xticks(rotation=0)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig(plots_dir / '1_MEJORA_PORCENTUAL_POR_D.png', dpi=300, bbox_inches='tight')
    plt.close()

    df_mejora.to_excel(output_dir / '1_tabla_mejora_por_d.xlsx')
    print(f"✓ Gráfico guardado: 1_MEJORA_PORCENTUAL_POR_D.png")
    print(f"✓ Tabla guardada: 1_tabla_mejora_por_d.xlsx")


# ====================================================================================
# ANÁLISIS 2: SENSIBILIDAD DE MODELOS POR d
# ====================================================================================
def plot_sensibilidad_modelos_por_d():
    """
    Heatmap de sensibilidad normalizada (CV) por modelo y d
    """
    print("\n=== ANÁLISIS 2: Sensibilidad de Modelos por d ===")
    d_values = sorted(df['d'].unique())

    sensibilidades = []
    for model in model_cols:
        sens_model = []
        for d_val in d_values:
            subset_d = df[df['d'] == d_val]
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].mean()
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].mean()
            valores = [sin, con]
            std = np.std(valores)
            mean = np.mean(valores)
            cv = (std / mean * 100) if mean != 0 else 0
            sens_model.append(cv)
        sensibilidades.append(sens_model)

    df_sens = pd.DataFrame(sensibilidades, index=model_cols,
                           columns=[f'd={d}' for d in d_values])

    plt.figure(figsize=(16, 12))
    sns.heatmap(df_sens, annot=True, fmt='.2f', cmap='YlOrRd',
                cbar_kws={'label': 'Coeficiente de Variación (%)'},
                linewidths=0.8, vmin=0)
    plt.title('Sensibilidad de Modelos por Orden de Diferenciación\n(Coeficiente de Variación entre Modalidades)',
              fontweight='bold', fontsize=16, pad=20)
    plt.xlabel('Orden de Diferenciación (d)', fontweight='bold', fontsize=13)
    plt.ylabel('Modelo', fontweight='bold', fontsize=13)
    plt.xticks(rotation=0)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig(plots_dir / '2_SENSIBILIDAD_MODELOS_POR_D.png', dpi=300, bbox_inches='tight')
    plt.close()

    df_sens.to_excel(output_dir / '2_tabla_sensibilidad_por_d.xlsx')
    print(f"✓ Gráfico guardado: 2_SENSIBILIDAD_MODELOS_POR_D.png")
    print(f"✓ Tabla guardada: 2_tabla_sensibilidad_por_d.xlsx")


# ====================================================================================
# ANÁLISIS 3: TEST DIEBOLD-MARIANO CON BONFERRONI POR MODELO
# ====================================================================================
def plot_dm_bonferroni_por_modelo():
    """
    Para cada modelo, compara SIN_DIFF vs CON_DIFF usando el test DM corregido.

    La clave del test DM es que necesita PARES de pérdidas por observación:
      - loss1_t = ECRPS del modelo SIN_DIFF en el paso t (réplica t)
      - loss2_t = ECRPS del modelo CON_DIFF en el paso t (réplica t)
    El test evalúa si E[loss1_t - loss2_t] = 0.

    Para que los pares sean válidos, se emparejan por 'Paso' dentro de cada
    (Escenario, d): cada Paso corresponde a una réplica de simulación, y la
    diferencia de pérdidas entre modalidades en ese Paso es la observación
    del diferencial d_t que alimenta el test.
    """
    print("\n=== ANÁLISIS 3: Test Diebold-Mariano con Bonferroni por Modelo ===")

    modelos_analizar = ['AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM',
                        'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

    modelos_disponibles = [m for m in modelos_analizar if m in model_cols]
    if len(modelos_disponibles) < len(modelos_analizar):
        faltantes = set(modelos_analizar) - set(modelos_disponibles)
        print(f"⚠ Modelos no encontrados: {faltantes}")
        print(f"  Modelos en el dataset: {model_cols}")

    d_values = sorted(df['d'].unique())

    # --- Crear columnas auxiliares de escenario ---
    def extract_pq(arma_str):
        try:
            parts = arma_str.replace('ARMA_I(', '').replace(')', '').split(',')
            if len(parts) == 3:
                p, d_val, q = parts
                return f"ARMA({p},{q})"
            return arma_str
        except Exception:
            return arma_str

    def grupo_proceso_arima(arma_str):
        try:
            parts = arma_str.replace('ARMA_I(', '').replace(')', '').split(',')
            if len(parts) == 3:
                p, d_val, q = parts
                return f"ARIMA({p},d,{q})"
            return arma_str
        except Exception:
            return arma_str

    df['ARMA_pq'] = df['ARMA_base'].apply(extract_pq)
    df['Proceso_Agrupado'] = df['ARMA_base'].apply(grupo_proceso_arima)

    # Escenario excluye d (varía en columnas del heatmap)
    df['Escenario'] = (df['Proceso'].astype(str) + '_' +
                       df['ARMA_pq'].astype(str) + '_' +
                       df['Distribución'].astype(str) + '_' +
                       df['Varianza'].astype(str))

    escenarios_unicos = sorted(df['Escenario'].unique())
    print(f"✓ Escenarios únicos: {len(escenarios_unicos)}")

    for model in modelos_disponibles:
        print(f"\n  Procesando modelo: {model}")

        resultados_detalle = []
        pvalor_matrix = []
        escenarios_lista = []

        for escenario in escenarios_unicos:
            pvalores_d = [np.nan] * len(d_values)
            tiene_datos = False

            for idx_d, d_val in enumerate(d_values):
                subset = df[(df['Escenario'] == escenario) & (df['d'] == d_val)]
                if len(subset) == 0:
                    continue

                proceso = subset['Proceso'].iloc[0]
                distribucion = subset['Distribución'].iloc[0]
                varianza = subset['Varianza'].iloc[0]
                proceso_agrupado = subset['Proceso_Agrupado'].iloc[0]

                # ----------------------------------------------------------------
                # CORRECCIÓN CLAVE: emparejar por 'Paso' para obtener pares
                # (loss_SIN_t, loss_CON_t) con el mismo índice temporal/réplica.
                # Antes se usaban arrays sin alinear, lo que rompía el test DM.
                # ----------------------------------------------------------------
                sin_df = subset[subset['Modalidad'] == 'SIN_DIFF'][['Paso', model]].rename(
                    columns={model: 'loss_sin'})
                con_df = subset[subset['Modalidad'] == 'CON_DIFF'][['Paso', model]].rename(
                    columns={model: 'loss_con'})

                # Inner join: solo pasos presentes en ambas modalidades
                paired = pd.merge(sin_df, con_df, on='Paso', how='inner')
                paired = paired.dropna(subset=['loss_sin', 'loss_con'])

                n_pairs = len(paired)

                # Umbral mínimo de pares para que el test sea informativo
                if n_pairs < 5:
                    continue

                loss_sin = paired['loss_sin'].values
                loss_con = paired['loss_con'].values

                dm_stat, p_value = diebold_mariano_test(loss_sin, loss_con, h=1)

                if np.isnan(p_value):
                    continue

                pvalores_d[idx_d] = p_value
                tiene_datos = True

                resultados_detalle.append({
                    'Proceso': proceso,
                    'Distribución': distribucion,
                    'Varianza': varianza,
                    'd': d_val,
                    'p_valor': p_value,
                    'n_pares': n_pairs,
                    'Proceso_Agrupado': proceso_agrupado
                })

            if tiene_datos:
                pvalor_matrix.append(pvalores_d)
                escenarios_lista.append(escenario)

        if len(pvalor_matrix) == 0:
            print(f"  ⚠ Sin datos válidos para {model}")
            continue

        df_pvalor = pd.DataFrame(pvalor_matrix,
                                 index=escenarios_lista,
                                 columns=[f'd={d}' for d in d_values])

        # Corrección Bonferroni sobre el número de tests válidos
        n_tests_validos = int(df_pvalor.notna().sum().sum())
        alpha_bonferroni = 0.05 / n_tests_validos if n_tests_validos > 0 else 0.05
        print(f"  α Bonferroni = 0.05 / {n_tests_validos} = {alpha_bonferroni:.6f}")

        df_significancia = df_pvalor < alpha_bonferroni

        # ==================== EXCEL CON 3 HOJAS ====================
        safe_name = model.replace(' ', '_').replace('/', '_').replace('-', '_')
        excel_path = output_dir / f'3_DM_COMPLETO_{safe_name}.xlsx'

        df_detalle = pd.DataFrame(resultados_detalle)
        df_agrupado = df_detalle[['Proceso_Agrupado', 'Distribución', 'Varianza', 'd', 'p_valor']].rename(
            columns={'Proceso_Agrupado': 'Proceso_ARIMA'})

        procesos_agrupados = sorted(df_agrupado['Proceso_ARIMA'].unique())
        matriz_pct_sig = []
        for proceso_ag in procesos_agrupados:
            fila_pct = []
            for d_val in d_values:
                subset_proc_d = df_agrupado[
                    (df_agrupado['Proceso_ARIMA'] == proceso_ag) &
                    (df_agrupado['d'] == d_val)
                ]
                if len(subset_proc_d) > 0:
                    n_total = len(subset_proc_d)
                    n_sig = (subset_proc_d['p_valor'] < alpha_bonferroni).sum()
                    fila_pct.append((n_sig / n_total) * 100)
                else:
                    fila_pct.append(np.nan)
            matriz_pct_sig.append(fila_pct)

        df_pct_sig = pd.DataFrame(matriz_pct_sig,
                                  index=procesos_agrupados,
                                  columns=[f'd={d}' for d in d_values])

        with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
            df_detalle[['Proceso', 'Distribución', 'Varianza', 'd', 'p_valor', 'n_pares']].to_excel(
                writer, sheet_name='1_Detalle', index=False)
            df_agrupado.to_excel(writer, sheet_name='2_Agrupado', index=False)
            df_pct_sig.to_excel(writer, sheet_name='3_Porcentaje_Significancia')

            workbook = writer.book
            for sheet_name, df_temp in [('1_Detalle', df_detalle), ('2_Agrupado', df_agrupado)]:
                worksheet = writer.sheets[sheet_name]
                fmt_green = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
                fmt_red = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
                col_idx = df_temp.columns.get_loc('p_valor')
                worksheet.conditional_format(1, col_idx, len(df_temp), col_idx,
                    {'type': 'cell', 'criteria': '<', 'value': alpha_bonferroni, 'format': fmt_green})
                worksheet.conditional_format(1, col_idx, len(df_temp), col_idx,
                    {'type': 'cell', 'criteria': '>=', 'value': alpha_bonferroni, 'format': fmt_red})

        print(f"  ✓ Excel guardado: {excel_path.name}")

        # ==================== HEATMAP % SIGNIFICANCIA ====================
        fig, ax = plt.subplots(figsize=(14, max(8, len(procesos_agrupados) * 0.4)))
        sns.heatmap(df_pct_sig, annot=True, fmt='.1f', cmap='RdYlGn',
                    cbar_kws={'label': '% Significativo (Bonferroni)'},
                    linewidths=0.8, vmin=0, vmax=100, ax=ax)
        ax.set_title(f'Porcentaje de p-valores Significativos: {model}\n'
                     f'Por Proceso ARIMA y d (α Bonferroni = {alpha_bonferroni:.6f})',
                     fontweight='bold', fontsize=14, pad=20)
        ax.set_xlabel('Orden de Diferenciación (d)', fontweight='bold', fontsize=12)
        ax.set_ylabel('Proceso ARIMA(p,d,q)', fontweight='bold', fontsize=12)
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
        ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
        plt.tight_layout()
        plt.savefig(plots_dir / f'3_HEATMAP_PCT_SIGNIFICANCIA_{safe_name}.png',
                    dpi=300, bbox_inches='tight')
        plt.close()
        print(f"  ✓ Heatmap % significancia guardado")

        # ==================== HEATMAP P-VALORES ====================
        fig, ax = plt.subplots(figsize=(16, max(12, len(escenarios_lista) * 0.3)))
        sns.heatmap(df_pvalor, annot=True, fmt='.4f', cmap='RdYlGn',
                    cbar_kws={'label': 'p-valor'}, linewidths=0.5,
                    vmin=0, vmax=0.1, center=alpha_bonferroni, ax=ax)

        for i in range(len(escenarios_lista)):
            for j in range(len(d_values)):
                if df_significancia.iloc[i, j]:
                    ax.add_patch(plt.Rectangle((j, i), 1, 1, fill=False,
                                               edgecolor='black', lw=3))

        ax.set_title(f'Test Diebold-Mariano: {model}\n'
                     f'SIN_DIFF vs CON_DIFF por Escenario y d\n'
                     f'(α Bonferroni = {alpha_bonferroni:.6f}, bordes negros = significativo)',
                     fontweight='bold', fontsize=14, pad=20)
        ax.set_xlabel('Orden de Diferenciación (d)', fontweight='bold', fontsize=12)
        ax.set_ylabel('Escenario (Proceso_ARMA(p,q)_Dist_Var)', fontweight='bold', fontsize=12)
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=7)
        ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
        plt.tight_layout()
        plt.savefig(plots_dir / f'3_DM_PVALORES_{safe_name}.png',
                    dpi=300, bbox_inches='tight')
        plt.close()
        print(f"  ✓ Heatmap p-valores guardado")

        n_significativos = int(df_significancia.sum().sum())
        pct_sig = (n_significativos / n_tests_validos) * 100 if n_tests_validos > 0 else 0
        print(f"  ✓ Comparaciones significativas: {n_significativos}/{n_tests_validos} ({pct_sig:.1f}%)")

    print(f"\n✓ Análisis 3 completado")


# ====================================================================================
# EJECUCIÓN PRINCIPAL
# ====================================================================================
if __name__ == "__main__":
    print("\n" + "=" * 80)
    print("ANÁLISIS MULTI-D — VERSIÓN CORREGIDA")
    print("=" * 80)

    plot_mejora_porcentual_por_d()
    plot_sensibilidad_modelos_por_d()
    plot_dm_bonferroni_por_modelo()

    print("\n" + "=" * 80)
    print(f"✓ ANÁLISIS COMPLETADO")
    print(f"✓ Resultados: {output_dir}")
    print(f"✓ Gráficos:   {plots_dir}")
    print("=" * 80)

✓ Datos cargados: (26880, 19)
✓ Columnas: ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 'Distribución', 'Varianza', 'Modalidad', 'Valor_Observado', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
✓ Modelos identificados: ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
✓ Valores únicos de d: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(10)]

ANÁLISIS MULTI-D — VERSIÓN CORREGIDA

=== ANÁLISIS 1: Mejora Porcentual por d ===
✓ Gráfico guardado: 1_MEJORA_PORCENTUAL_POR_D.png
✓ Tabla guardada: 1_tabla_mejora_por_d.xlsx

=== ANÁLISIS 2: Sensibilidad de Modelos por d ===
✓ Gráfico guardado: 2_SENSIBILIDAD_MODELOS_POR_D.png
✓ Tabla guardada: 2_tabla_sensibilidad_por_d.xlsx

=== ANÁLISIS 3: Test Diebold-Mariano con Bonferroni por Modelo ===
✓ Escenarios únicos: 1120

  Procesando modelo: AV-MCPS
  α Bonferroni = 0

## Analisis Tamaño

### Pre-procesamiento

In [6]:
import pandas as pd
import numpy as np

# 1. Leer los tres archivos
arma_df = pd.read_excel("./datos/Simulacion/Tamaño/resultados_TAMANOS_CRECIENTES_ARMA.xlsx")
arima_df = pd.read_excel("./datos/Simulacion/Tamaño/resultados_TAMANOS_CRECIENTES_ARIMA.xlsx")
setar_df = pd.read_excel("./datos/Simulacion/Tamaño/resultados_TAMANOS_CRECIENTES_SETAR.xlsx")

# 2. ESTANDARIZAR NOMBRES DE COLUMNAS
# Renombramos 'Block Bootstrap' a 'Block Bootstrapping' en ARIMA y SETAR para que coincidan con ARMA
arima_df = arima_df.rename(columns={'Block Bootstrap': 'Block Bootstrapping'})
setar_df = setar_df.rename(columns={'Block Bootstrap': 'Block Bootstrapping'})

# 3. Asignar escenarios
arma_df['ESCENARIO'] = "Lineal Estacionario"
arima_df['ESCENARIO'] = "Lineal No estacionario"
setar_df['ESCENARIO'] = "No lineal Estacionario"

# 4. Filtrar los que no tienen "Promedio" en la columna "Paso"
arma_df = arma_df[arma_df['Paso'] != 'Promedio']
arima_df = arima_df[arima_df['Paso'] != 'Promedio']
setar_df = setar_df[setar_df['Paso'] != 'Promedio']

# 5. Lista de modelos (ahora el nombre es uniforme)
modelos = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# 6. Crear tabla comparativa
comparacion = []

for modelo in modelos:
    fila = {'Modelo': modelo}
    
    # Calcular promedio para cada escenario
    arma_promedio = arma_df[modelo].mean() if modelo in arma_df.columns else np.nan
    arima_promedio = arima_df[modelo].mean() if modelo in arima_df.columns else np.nan
    setar_promedio = setar_df[modelo].mean() if modelo in setar_df.columns else np.nan
    
    fila['ARMA'] = arma_promedio
    fila['ARIMA'] = arima_promedio
    fila['SETAR'] = setar_promedio
    
    # Determinar mejor escenario (menor promedio)
    promedios = {
        'ARMA': arma_promedio,
        'ARIMA': arima_promedio,
        'SETAR': setar_promedio
    }
    
    promedios_validos = {k: v for k, v in promedios.items() if not pd.isna(v)}
    
    if promedios_validos:
        mejor_escenario = min(promedios_validos, key=promedios_validos.get)
        fila['Mejor_Escenario'] = mejor_escenario
    else:
        fila['Mejor_Escenario'] = 'N/A'
    
    comparacion.append(fila)

# Crear DataFrame con la tabla comparativa
tabla_comparativa = pd.DataFrame(comparacion)
columnas_numericas = ['ARMA', 'ARIMA', 'SETAR']
tabla_comparativa[columnas_numericas] = tabla_comparativa[columnas_numericas].round(4)

# Mostrar tabla comparativa
print("\n" + "="*80)
print("TABLA COMPARATIVA DE MODELOS POR ESCENARIO")
print("(Promedio de amplitud de intervalos de predicción)")
print("="*80)
print(tabla_comparativa.to_string(index=False))
print("="*80 + "\n")

# 7. Procesamiento final y exportación
if 'Descripción' in setar_df.columns:
    setar_df = setar_df.drop('Descripción', axis=1)

# Concatenar los tres dataframes (ahora las columnas se alinearán perfectamente)
base_consolidada = pd.concat([arma_df, arima_df, setar_df], ignore_index=True)

# Guardar en un archivo Excel
base_consolidada.to_excel("./datos/Simulacion/Tamaño/Base_Tamaño_3_escenarios.xlsx", index=False)

print("\nArchivo 'Base_Tamaño_3_escenarios.xlsx' creado exitosamente!")
print(f"Columna 'Block Bootstrapping' verificada en el archivo final.")
print(f"Total de filas: {len(base_consolidada)}")


TABLA COMPARATIVA DE MODELOS POR ESCENARIO
(Promedio de amplitud de intervalos de predicción)
             Modelo   ARMA   ARIMA  SETAR Mejor_Escenario
              AREPD 0.9264 12.3986 0.8148           SETAR
            AV-MCPS 0.6780  3.0524 0.6585           SETAR
Block Bootstrapping 0.9036 14.2503 0.7915           SETAR
             DeepAR 0.5745  2.8056 0.5736           SETAR
         EnCQR-LSTM 0.7947  6.1857 0.7544           SETAR
               LSPM 0.7721  1.0854 0.6490           SETAR
              LSPMW 0.7839  1.0846 0.6610           SETAR
               MCPS 0.6613  2.8912 0.6488           SETAR
    Sieve Bootstrap 0.5444  0.5473 0.5544            ARMA


Archivo 'Base_Tamaño_3_escenarios.xlsx' creado exitosamente!
Columna 'Block Bootstrapping' verificada en el archivo final.
Total de filas: 25200


### Analisis general

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
from pathlib import Path
from itertools import combinations

# ====================================================================================
# 1. CONFIGURACIÓN Y PREPARACIÓN DE DATOS
# ====================================================================================
warnings.filterwarnings('ignore')

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 11
plt.rcParams['font.family'] = 'serif'

output_dir = Path("./Resultados_analisis/Tamaño")
output_dir.mkdir(parents=True, exist_ok=True)

try:
    df = pd.read_excel("./datos/Simulacion/Tamaño/Base_Tamaño_3_escenarios.xlsx")
except FileNotFoundError:
    print("Error: No se encontró el archivo Excel. Verifica la ruta.")
    exit()

# Renombrar escenarios de tipo
df['ESCENARIO'] = df['ESCENARIO'].replace({
    "Lineal Estacionario":      "Lineal Estacionario (ARMA)",
    "Lineal No estacionario":   "Lineal No Estacionario (ARIMA)",
    "No lineal Estacionario":   "No lineal Estacionario (SETAR)"
})

var_cols = ['Paso', 'Proceso', 'Tipo_Proceso', 'Distribución', 'Varianza',
            'N_Train', 'N_Calib', 'N_Total', 'Valor_Observado', 'ESCENARIO', 'Size']

model_cols = [col for col in df.columns
              if col not in var_cols and pd.api.types.is_numeric_dtype(df[col])]

n_total_values = sorted(df['N_Total'].unique())
tipos_escenario = sorted(df['ESCENARIO'].unique())   # los 3 tipos (ARMA, ARIMA, SETAR)

# ====================================================================================
# ESCENARIO = combinación única de (Proceso, Distribución, Varianza)
# Esta es la unidad analítica del test DM:
# comparamos modelo A con N=n1 vs modelo A con N=n2 DENTRO del mismo escenario.
# ====================================================================================
df['escenario_id'] = (df['Proceso'].astype(str) + ' | ' +
                      df['Distribución'].astype(str) + ' | ' +
                      df['Varianza'].astype(str))

escenarios_unicos = sorted(df['escenario_id'].unique())

print(f"\n{'='*80}")
print(f"✓ Datos cargados:        {df.shape}")
print(f"✓ Modelos:               {len(model_cols)}")
print(f"✓ Tamaños N_Total:       {n_total_values}")
print(f"✓ Tipos de escenario:    {tipos_escenario}")
print(f"✓ Escenarios únicos      "
      f"(Proceso × Dist × Var): {len(escenarios_unicos)}")

# Mostrar cuántos escenarios hay por tipo
for t in tipos_escenario:
    ids_tipo = df[df['ESCENARIO'] == t]['escenario_id'].unique()
    print(f"   {t}: {len(ids_tipo)} escenarios")
print(f"{'='*80}\n")

# ====================================================================================
# PALETA DE COLORES PARA GRÁFICAS
# ====================================================================================
def generate_maximally_distinct_colors(n):
    """Genera n colores maximamente distintivos"""
    if n <= 10:
        return plt.cm.tab10(np.linspace(0, 1, 10))[:n]
    elif n <= 20:
        return plt.cm.tab20(np.linspace(0, 1, 20))[:n]
    else:
        colors = []
        tab20 = plt.cm.tab20(np.linspace(0, 1, 20))
        colors.extend(tab20)
        if n > 20:
            set3 = plt.cm.Set3(np.linspace(0, 1, 12))
            colors.extend(set3)
        if n > 32:
            paired = plt.cm.Paired(np.linspace(0, 1, 12))
            colors.extend(paired)
        if n > 44:
            remaining = n - len(colors)
            for i in range(remaining):
                hue = (i * 0.618033988749895) % 1.0
                rgb = plt.cm.hsv(hue)
                colors.append(rgb)
        return np.array(colors[:n])

MODEL_COLORS = dict(zip(model_cols, generate_maximally_distinct_colors(len(model_cols))))

# ====================================================================================
# 2. TEST DIEBOLD-MARIANO CORREGIDO
#    HAC Newey-West + corrección Harvey-Leybourne-Newbold (1997)
# ====================================================================================

def diebold_mariano_test(loss1, loss2, h=1):
    """
    Test DM bilateral para H0: E[loss1_t - loss2_t] = 0.

    loss1_t, loss2_t deben ser pérdidas del MISMO paso/réplica t
    (emparejar por 'Paso' antes de llamar esta función).

    Parameters
    ----------
    loss1, loss2 : array-like, mismo largo
    h            : horizonte de pronóstico (default=1)

    Returns
    -------
    dm_stat : float   estadístico con corrección HLN
    p_value : float   p-valor bilateral t_{T-1}
    """
    loss1 = np.asarray(loss1, dtype=float)
    loss2 = np.asarray(loss2, dtype=float)

    T = len(loss1)
    if T < 4:
        return np.nan, np.nan

    d     = loss1 - loss2
    d_bar = np.mean(d)

    # Varianza HAC Newey-West con bandwidth K = h-1
    K         = max(h - 1, 0)
    d_c       = d - d_bar       # desviaciones centradas
    gamma_0   = np.dot(d_c, d_c) / T
    hac_var   = gamma_0

    for k in range(1, K + 1):
        gamma_k = np.dot(d_c[k:], d_c[:-k]) / T
        w_k     = 1.0 - k / (K + 1.0)
        hac_var += 2.0 * w_k * gamma_k

    if hac_var <= 0:
        hac_var = gamma_0
    if hac_var <= 0:
        return 0.0, 1.0

    dm_stat    = d_bar / np.sqrt(hac_var / T)

    # Corrección HLN (Harvey, Leybourne & Newbold 1997)
    hln_factor = np.sqrt((T + 1 - 2 * h + h * (h - 1) / T) / T)
    dm_stat   *= hln_factor

    p_value    = 2.0 * (1.0 - stats.t.cdf(abs(dm_stat), df=T - 1))
    return dm_stat, p_value


# ====================================================================================
# 3. FUNCIÓN CENTRAL: tabla de p-valores por modelo (POR TIPO DE ESCENARIO)
# ====================================================================================

def compute_pvalue_table(model, tipo_escenario=None):
    """
    Genera la tabla detallada de p-valores para un modelo.
    
    Si tipo_escenario es None: usa todos los escenarios
    Si tipo_escenario es especificado: filtra solo ese tipo
    
    Filas    = escenario único (Proceso | Dist | Var)
    Columnas = par de tamaños  (N=n1 vs N=n2)

    Emparejamiento: inner-join por 'Paso' entre N=n1 y N=n2
    dentro del mismo escenario → misma réplica de simulación.
    """
    # Filtrar datos si se especifica tipo de escenario
    if tipo_escenario:
        df_filtrado = df[df['ESCENARIO'] == tipo_escenario]
        escenarios_usar = sorted(df_filtrado['escenario_id'].unique())
    else:
        df_filtrado = df
        escenarios_usar = escenarios_unicos
    
    size_pairs = list(combinations(n_total_values, 2))
    col_names  = [f"N={n1} vs N={n2}" for n1, n2 in size_pairs]

    rows_pval, rows_dm = [], []

    for esc_id in escenarios_usar:
        subset_esc = df_filtrado[df_filtrado['escenario_id'] == esc_id]

        row_pval, row_dm = [], []

        for (n1, n2) in size_pairs:
            s1 = (subset_esc[subset_esc['N_Total'] == n1][['Paso', model]]
                  .rename(columns={model: 'l1'}))
            s2 = (subset_esc[subset_esc['N_Total'] == n2][['Paso', model]]
                  .rename(columns={model: 'l2'}))

            paired = pd.merge(s1, s2, on='Paso', how='inner').dropna()
            n_pairs = len(paired)

            if n_pairs < 5:
                row_pval.append(np.nan)
                row_dm.append(np.nan)
                continue

            dm_stat, p_val = diebold_mariano_test(
                paired['l1'].values, paired['l2'].values, h=1)

            row_pval.append(p_val)
            row_dm.append(dm_stat)

        rows_pval.append(row_pval)
        rows_dm.append(row_dm)

    df_pval = pd.DataFrame(rows_pval, index=escenarios_usar, columns=col_names)
    df_dm   = pd.DataFrame(rows_dm,   index=escenarios_usar, columns=col_names)

    return df_pval, df_dm


# ====================================================================================
# 4. CALCULAR TODAS LAS TABLAS Y BONFERRONI GLOBAL (POR TIPO)
# ====================================================================================

def compute_all_tables(tipo_escenario=None):
    """
    Recorre todos los modelos, calcula las tablas de p-valores y determina
    el α de Bonferroni global.
    
    Si tipo_escenario es None: análisis general
    Si tipo_escenario especificado: solo ese tipo
    """
    tipo_str = tipo_escenario if tipo_escenario else "GENERAL"
    print(f"\n[Calculando tablas de p-valores - {tipo_str}]")

    all_pval_tables = {}
    all_dm_tables   = {}
    n_tests_total   = 0

    for model in model_cols:
        df_pval, df_dm = compute_pvalue_table(model, tipo_escenario)
        all_pval_tables[model] = df_pval
        all_dm_tables[model]   = df_dm
        n_valid = int(df_pval.notna().sum().sum())
        n_tests_total += n_valid

    alpha_bonf = 0.05 / n_tests_total if n_tests_total > 0 else 0.05
    print(f"✓ Tests válidos totales: {n_tests_total}")
    print(f"✓ α Bonferroni = 0.05 / {n_tests_total} = {alpha_bonf:.8f}")

    # ECRPS medio por (N_Total, modelo)
    df_usar = df if tipo_escenario is None else df[df['ESCENARIO'] == tipo_escenario]
    ecrps_data = {model: [df_usar[df_usar['N_Total'] == nt][model].mean()
                           for nt in n_total_values]
                  for model in model_cols}
    df_ecrps = pd.DataFrame(ecrps_data, index=n_total_values)
    df_ecrps.index.name = 'N_Total'

    return all_pval_tables, all_dm_tables, alpha_bonf, df_ecrps


# ====================================================================================
# 5. TABLA RESUMEN: % escenarios significativos por (N_Total, modelo)
# ====================================================================================

def compute_summary_pct(all_pval_tables, alpha_bonf):
    """
    Tabla resumen % escenarios con diferencia significativa.
    Filas = Tamaños N_Total, Columnas = Modelos
    """
    pct_data = {}
    for model in model_cols:
        df_pval = all_pval_tables[model]
        col_pct = []
        for nt in n_total_values:
            cols_nt = [c for c in df_pval.columns if f"N={nt}" in c]
            vals = df_pval[cols_nt].values.flatten()
            vals = vals[~np.isnan(vals)]
            if len(vals) == 0:
                col_pct.append(np.nan)
            else:
                col_pct.append(100.0 * np.sum(vals < alpha_bonf) / len(vals))
        pct_data[model] = col_pct

    df_pct = pd.DataFrame(pct_data, index=n_total_values)
    df_pct.index.name = 'N_Total'
    return df_pct


# ====================================================================================
# 6. VISUALIZACIÓN: HEATMAP COMBINADO (ECRPS + % SIGNIFICANCIA)
# ====================================================================================

def plot_combined_heatmap(df_ecrps, df_pct, alpha_bonf, tipo_escenario=None):
    """
    Heatmap resumen:
      Fondo = % comparaciones significativas (verde alto, rojo bajo)
      Anotación = "ECRPS\n(% sig)"
    """
    suffix = tipo_escenario.replace(" ", "_").replace("(", "").replace(")", "") if tipo_escenario else "General"
    
    n_rows = len(n_total_values)
    n_cols = len(model_cols)

    fig, ax = plt.subplots(figsize=(max(14, n_cols * 1.4),
                                    max(5,  n_rows * 0.9)))

    annot = np.empty((n_rows, n_cols), dtype=object)
    for i, nt in enumerate(n_total_values):
        for j, model in enumerate(model_cols):
            ecrps_val = df_ecrps.iloc[i, j]
            pct_val   = df_pct.iloc[i, j]
            if np.isnan(pct_val):
                annot[i, j] = f"{ecrps_val:.3f}\n(—)"
            else:
                annot[i, j] = f"{ecrps_val:.3f}\n({pct_val:.0f}%)"

    sns.heatmap(
        df_pct,
        annot=annot,
        fmt='',
        cmap='RdYlGn',
        vmin=0, vmax=100, center=50,
        linewidths=0.6, linecolor='white',
        cbar_kws={'label': '% Escenarios con diferencia significativa (Bonferroni)',
                  'shrink': 0.7},
        ax=ax,
        annot_kws={'fontsize': 8, 'va': 'center'}
    )

    title = f'ECRPS Medio y % Escenarios con Diferencia Significativa por Tamaño Muestral'
    if tipo_escenario:
        title += f'\n{tipo_escenario}'
    title += f'\nCelda: ECRPS medio  |  (%) escenarios sign. Bonferroni  α = {alpha_bonf:.6f}'
    
    ax.set_title(title, fontweight='bold', fontsize=12, pad=15)
    ax.set_xlabel('Modelo', fontweight='bold', fontsize=11)
    ax.set_ylabel('Tamaño Muestral (N_Total)', fontweight='bold', fontsize=11)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha='right', fontsize=9)
    ax.set_yticklabels([f'N={nt}' for nt in n_total_values], rotation=0, fontsize=10)

    plt.tight_layout()
    plt.savefig(output_dir / f'heatmap_ecrps_significancia_{suffix}.png',
                dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Heatmap combinado guardado: {suffix}")


# ====================================================================================
# 7. HEATMAP: Z-SCORES POR MODELO
# ====================================================================================

def plot_heatmap_zscores(data, tipo_escenario=None):
    """
    Heatmap de Modelos (filas) vs N_Total (columnas) con Z-scores
    Z-score calculado POR MODELO (estandarización por fila)
    """
    suffix = tipo_escenario.replace(" ", "_").replace("(", "").replace(")", "") if tipo_escenario else "General"
    
    if tipo_escenario:
        data = data[data['ESCENARIO'] == tipo_escenario]
    
    # Crear matriz de ECRPS promedio
    ecrps_data = []
    for model in model_cols:
        row = []
        for nt in n_total_values:
            ecrps_mean = data[data['N_Total'] == nt][model].mean()
            row.append(ecrps_mean)
        ecrps_data.append(row)
    
    ecrps_df = pd.DataFrame(ecrps_data, 
                             index=model_cols, 
                             columns=[f'N={nt}' for nt in n_total_values])
    
    # Calcular Z-scores POR MODELO (por fila)
    zscore_df = ecrps_df.apply(lambda row: (row - row.mean()) / row.std() if row.std() > 0 else 0, axis=1)
    
    # Guardar datos
    ecrps_df.to_excel(output_dir / f'heatmap_ecrps_{suffix}.xlsx')
    zscore_df.to_excel(output_dir / f'heatmap_zscores_{suffix}.xlsx')
    
    # Graficar
    plt.figure(figsize=(10, max(8, len(model_cols) * 0.4)))
    sns.heatmap(zscore_df, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, vmin=-2, vmax=2,
                cbar_kws={'label': 'Z-score'},
                linewidths=0.5, linecolor='gray')
    
    title = 'Z-scores de ECRPS: Modelos vs N_Total'
    if tipo_escenario:
        title += f'\n{tipo_escenario}'
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Tamaño Muestral Total', fontsize=12)
    plt.ylabel('Modelo', fontsize=12)
    plt.tight_layout()
    plt.savefig(output_dir / f'heatmap_zscore_{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Heatmap Z-scores generado: {suffix}")


# ====================================================================================
# 8. GRÁFICA DE MEJORA RELATIVA
# ====================================================================================

def plot_relative_improvement(data, tipo_escenario=None):
    """Mejora relativa (%) respecto al tamaño base (primer N_Total)"""
    suffix = tipo_escenario.replace(" ", "_").replace("(", "").replace(")", "") if tipo_escenario else "General"
    
    if tipo_escenario:
        data = data[data['ESCENARIO'] == tipo_escenario]
    
    baseline = n_total_values[0]
    improvements = {}
    
    for model in model_cols:
        base_perf = data[data['N_Total'] == baseline][model].mean()
        model_impr = []
        for nt in n_total_values:
            current = data[data['N_Total'] == nt][model].mean()
            improvement = ((base_perf - current) / base_perf) * 100 if base_perf != 0 else 0
            model_impr.append(improvement)
        improvements[model] = model_impr
    
    # Guardar datos
    improvements_df = pd.DataFrame(improvements, index=n_total_values)
    improvements_df.index.name = 'N_Total'
    improvements_df.to_excel(output_dir / f'mejora_relativa_{suffix}.xlsx')
    
    # Graficar
    plt.figure(figsize=(12, 7))
    for model, values in improvements.items():
        plt.plot(n_total_values, values, marker='o', label=model, 
                color=MODEL_COLORS[model], linewidth=2.5, markersize=8)
    
    plt.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    
    title = f'Mejora Relativa vs N={n_total_values[0]}'
    if tipo_escenario:
        title += f' - {tipo_escenario}'
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('N_Total', fontsize=12)
    plt.ylabel('Mejora Relativa (%)', fontsize=12)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    plt.grid(True, alpha=0.3, linestyle=':', linewidth=0.8)
    plt.tight_layout()
    plt.savefig(output_dir / f'mejora_relativa_{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Mejora relativa generada: {suffix}")


# ====================================================================================
# 9. EXPORTAR EXCEL CONSOLIDADO
# ====================================================================================

def export_consolidated_excel(results_dict):
    """
    Excel consolidado con todas las tablas de p-valores y resúmenes
    
    results_dict = {
        'General': (all_pval_tables, all_dm_tables, alpha_bonf, df_ecrps, df_pct),
        'ARMA': (...),
        'ARIMA': (...),
        'SETAR': (...)
    }
    """
    excel_path = output_dir / 'DM_tamaño_muestral_consolidado.xlsx'

    with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
        wb = writer.book

        # Formatos
        fmt_hdr  = wb.add_format({'bold': True, 'bg_color': '#2E4057',
                                   'font_color': 'white', 'align': 'center',
                                   'valign': 'vcenter', 'border': 1})
        fmt_sig  = wb.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100',
                                   'align': 'center', 'border': 1})
        fmt_no   = wb.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006',
                                   'align': 'center', 'border': 1})

        sheet_counter = 1
        
        for tipo, (all_pval_tables, all_dm_tables, alpha_bonf, df_ecrps, df_pct) in results_dict.items():
            tipo_short = tipo.replace("Lineal Estacionario (ARMA)", "ARMA") \
                             .replace("Lineal No Estacionario (ARIMA)", "ARIMA") \
                             .replace("No lineal Estacionario (SETAR)", "SETAR") \
                             .replace("General", "General")
            
            # Hoja 1: ECRPS medios
            sheet_name = f'{sheet_counter}_ECRPS_{tipo_short}'
            df_ecrps.to_excel(writer, sheet_name=sheet_name)
            ws = writer.sheets[sheet_name]
            ws.set_column('A:A', 12)
            ws.set_column('B:Z', 18)
            for c, col in enumerate(df_ecrps.columns, 1):
                ws.write(0, c, col, fmt_hdr)
            sheet_counter += 1
            
            # Hoja 2: % Significancia
            sheet_name = f'{sheet_counter}_PctSig_{tipo_short}'
            df_pct.to_excel(writer, sheet_name=sheet_name)
            ws2 = writer.sheets[sheet_name]
            ws2.set_column('A:A', 12)
            ws2.set_column('B:Z', 18)
            for c, col in enumerate(df_pct.columns, 1):
                ws2.write(0, c, col, fmt_hdr)
            for c in range(1, len(model_cols) + 1):
                ws2.conditional_format(1, c, len(n_total_values), c,
                    {'type': 'cell', 'criteria': '>=', 'value': 50, 'format': fmt_sig})
                ws2.conditional_format(1, c, len(n_total_values), c,
                    {'type': 'cell', 'criteria': '<',  'value': 50, 'format': fmt_no})
            sheet_counter += 1

        # Hoja final: Metodología
        meta = pd.DataFrame({
            'Parámetro': [
                'Unidad de escenario',
                'Emparejamiento dentro del test DM',
                'Test estadístico',
                'Corrección de muestra pequeña',
                'Corrección de multiplicidad',
                'α nominal',
                'Métrica de pérdida',
                'Modelos evaluados',
                'Tamaños N_Total',
                'Tipos de escenario',
                'Escenarios únicos totales'
            ],
            'Valor': [
                'Combinación única (Proceso, Distribución, Varianza)',
                'Inner-join por columna Paso (réplica de simulación)',
                'Diebold-Mariano bilateral',
                'Harvey, Leybourne & Newbold (1997)',
                'Bonferroni sobre todos los tests válidos (por tipo)',
                0.05,
                'ECRPS',
                ', '.join(model_cols),
                ', '.join(map(str, n_total_values)),
                ', '.join(tipos_escenario),
                len(escenarios_unicos)
            ]
        })
        meta.to_excel(writer, sheet_name='ZZ_Metodologia', index=False)
        ws_m = writer.sheets['ZZ_Metodologia']
        ws_m.set_column('A:A', 45)
        ws_m.set_column('B:B', 80)

    print(f"\n✓ Excel consolidado guardado: {excel_path}")


# ====================================================================================
# 10. EJECUCIÓN PRINCIPAL
# ====================================================================================

if __name__ == "__main__":
    print("\n" + "=" * 80)
    print("ANÁLISIS DM CORREGIDO — EFECTOS DEL TAMAÑO MUESTRAL")
    print("Escenario = combinación única (Proceso × Distribución × Varianza)")
    print("=" * 80)

    # Diccionario para guardar todos los resultados
    all_results = {}
    
    # ============================================================
    # ANÁLISIS GENERAL
    # ============================================================
    print("\n" + "="*80)
    print("ANÁLISIS GENERAL (TODOS LOS ESCENARIOS)")
    print("="*80)
    
    all_pval_tables, all_dm_tables, alpha_bonf, df_ecrps = compute_all_tables(tipo_escenario=None)
    df_pct = compute_summary_pct(all_pval_tables, alpha_bonf)
    
    all_results['General'] = (all_pval_tables, all_dm_tables, alpha_bonf, df_ecrps, df_pct)
    
    print("\n[Generando visualizaciones - GENERAL]")
    plot_combined_heatmap(df_ecrps, df_pct, alpha_bonf, tipo_escenario=None)
    plot_heatmap_zscores(df, tipo_escenario=None)
    plot_relative_improvement(df, tipo_escenario=None)
    
    # ============================================================
    # ANÁLISIS POR TIPO DE ESCENARIO
    # ============================================================
    for tipo in tipos_escenario:
        print("\n" + "="*80)
        print(f"ANÁLISIS: {tipo}")
        print("="*80)
        
        all_pval_tables, all_dm_tables, alpha_bonf, df_ecrps = compute_all_tables(tipo_escenario=tipo)
        df_pct = compute_summary_pct(all_pval_tables, alpha_bonf)
        
        all_results[tipo] = (all_pval_tables, all_dm_tables, alpha_bonf, df_ecrps, df_pct)
        
        print(f"\n[Generando visualizaciones - {tipo}]")
        plot_combined_heatmap(df_ecrps, df_pct, alpha_bonf, tipo_escenario=tipo)
        plot_heatmap_zscores(df, tipo_escenario=tipo)
        plot_relative_improvement(df, tipo_escenario=tipo)
    
    # ============================================================
    # EXPORTAR EXCEL CONSOLIDADO
    # ============================================================
    print("\n" + "="*80)
    print("EXPORTANDO EXCEL CONSOLIDADO")
    print("="*80)
    export_consolidated_excel(all_results)
    
    # ============================================================
    # RESUMEN FINAL
    # ============================================================
    print("\n" + "=" * 80)
    print("✓ ANÁLISIS COMPLETADO")
    print(f"✓ Resultados en: {output_dir}")
    print("\nArchivos generados:")
    print("  1. HEATMAPS COMBINADOS (ECRPS + % Significancia):")
    print("     - 4 archivos .png (General + ARMA + ARIMA + SETAR)")
    print("  2. HEATMAPS Z-SCORES:")
    print("     - 4 archivos .png + 8 .xlsx (General + ARMA + ARIMA + SETAR)")
    print("  3. CURVAS DE MEJORA RELATIVA:")
    print("     - 4 archivos .png + 4 .xlsx (General + ARMA + ARIMA + SETAR)")
    print("  4. EXCEL CONSOLIDADO:")
    print("     - 1 archivo con 9 hojas (2 por análisis + metodología)")
    print("\nMETODOLOGÍA:")
    print("  - Test DM: HAC Newey-West + corrección HLN (1997)")
    print("  - Emparejamiento: por Paso dentro de cada escenario único")
    print("  - Corrección: Bonferroni por tipo de análisis")
    print("=" * 80 + "\n")


✓ Datos cargados:        (25200, 20)
✓ Modelos:               9
✓ Tamaños N_Total:       [np.int64(120), np.int64(240), np.int64(360), np.int64(600), np.int64(1200)]
✓ Tipos de escenario:    ['Lineal Estacionario (ARMA)', 'Lineal No Estacionario (ARIMA)', 'No lineal Estacionario (SETAR)']
✓ Escenarios únicos      (Proceso × Dist × Var): 420
   Lineal Estacionario (ARMA): 140 escenarios
   Lineal No Estacionario (ARIMA): 140 escenarios
   No lineal Estacionario (SETAR): 140 escenarios


ANÁLISIS DM CORREGIDO — EFECTOS DEL TAMAÑO MUESTRAL
Escenario = combinación única (Proceso × Distribución × Varianza)

ANÁLISIS GENERAL (TODOS LOS ESCENARIOS)

[Calculando tablas de p-valores - GENERAL]
✓ Tests válidos totales: 37800
✓ α Bonferroni = 0.05 / 37800 = 0.00000132

[Generando visualizaciones - GENERAL]
✓ Heatmap combinado guardado: General
✓ Heatmap Z-scores generado: General
✓ Mejora relativa generada: General

ANÁLISIS: Lineal Estacionario (ARMA)

[Calculando tablas de p-valores - Lineal E

## Analisis proporciones

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
from pathlib import Path
from itertools import combinations

# ====================================================================================
# 1. CONFIGURACIÓN Y PREPARACIÓN DE DATOS
# ====================================================================================
warnings.filterwarnings('ignore')

# Configuración de gráficas
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 11
plt.rcParams['font.family'] = 'serif'

# Rutas
output_dir = Path("./Resultados_analisis/Proporciones")
output_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
try:
    df = pd.read_excel("./datos/Simulacion/proporciones/resultados_PROPORCIONES_240_TODOS.xlsx")
except FileNotFoundError:
    print("Error: No se encontró el archivo Excel. Verifica la ruta.")
    exit()

# Renombrar escenarios
df['Tipo_Proceso'] = df['Tipo_Proceso'].replace({
    "ARMA": "Lineal Estacionario (ARMA)",
    "ARIMA": "Lineal No Estacionario (ARIMA)",
    "SETAR": "No Lineal Estacionario (SETAR)"
})

# Limpiar Prop_Calib
def clean_prop_calib(value):
    if pd.isna(value):
        return np.nan
    value_str = str(value).strip()
    if '%' in value_str:
        value_str = value_str.split('%')[0]
    try:
        value_num = float(value_str)
        if value_num < 1:
            return int(value_num * 100)
        else:
            return int(value_num)
    except:
        return np.nan

df['Prop_Calib_Pct'] = df['Prop_Calib'].apply(clean_prop_calib)
df['Prop_Calib'] = df['Prop_Calib_Pct'] / 100

# Limpiar Paso
def clean_paso(value):
    if pd.isna(value):
        return np.nan
    value_str = str(value).strip()
    import re
    numbers = re.findall(r'\d+', value_str)
    if numbers:
        return int(numbers[0])
    else:
        return np.nan

df['Paso'] = df['Paso'].apply(clean_paso)

# Identificación de variables y modelos
var_cols = ['Paso', 'Proceso', 'Distribución', 'Varianza', 
            'N_Train', 'N_Calib', 'Prop_Calib', 'Prop_Calib_Pct', 'Tipo_Proceso']
model_cols = [col for col in df.columns if col not in var_cols and pd.api.types.is_numeric_dtype(df[col])]

# Limpiar datos numéricos
for col in ['N_Train', 'N_Calib', 'Varianza', 'Paso']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

for model in model_cols:
    df[model] = pd.to_numeric(df[model], errors='coerce')

# Eliminar NaN
critical_cols = ['Paso', 'Prop_Calib_Pct', 'N_Train', 'N_Calib'] + model_cols
df = df.dropna(subset=critical_cols)

prop_values = sorted(df['Prop_Calib_Pct'].unique())
escenarios = sorted(df['Tipo_Proceso'].unique())

# Crear escenario_id único (Proceso | Distribución | Varianza)
df['escenario_id'] = (df['Proceso'].astype(str) + ' | ' +
                      df['Distribución'].astype(str) + ' | ' +
                      df['Varianza'].astype(str))

escenarios_unicos = sorted(df['escenario_id'].unique())

# ====================================================================================
# PALETA DE COLORES ÚNICA PARA MODELOS
# ====================================================================================
def generate_maximally_distinct_colors(n):
    if n <= 10:
        base_colors = plt.cm.tab10(np.linspace(0, 1, 10))
        return base_colors[:n]
    elif n <= 20:
        base_colors = plt.cm.tab20(np.linspace(0, 1, 20))
        return base_colors[:n]
    else:
        colors = []
        tab20 = plt.cm.tab20(np.linspace(0, 1, 20))
        colors.extend(tab20)
        
        if n > 20:
            set3 = plt.cm.Set3(np.linspace(0, 1, 12))
            colors.extend(set3)
        
        if n > 32:
            paired = plt.cm.Paired(np.linspace(0, 1, 12))
            colors.extend(paired)
        
        if n > 44:
            remaining = n - len(colors)
            hsv_colors = []
            for i in range(remaining):
                hue = (i * 0.618033988749895) % 1.0
                saturation = 0.6 + (i % 3) * 0.15
                value = 0.7 + (i % 2) * 0.2
                rgb = plt.cm.hsv(hue)
                hsv_colors.append(rgb)
            colors.extend(hsv_colors)
        
        return np.array(colors[:n])

MODEL_COLORS = dict(zip(model_cols, generate_maximally_distinct_colors(len(model_cols))))

print(f"\n{'='*80}")
print(f"SIMULACIÓN 4: PROPORCIONES DE CALIBRACIÓN (N=240 FIJO)")
print(f"CON TEST DIEBOLD-MARIANO MODIFICADO Y CORRECCIÓN DE BONFERRONI")
print(f"{'='*80}")
print(f"\nModelos evaluados: {len(model_cols)}")
print(f"Proporciones: {prop_values}")
print(f"Escenarios: {list(escenarios)}")
print(f"Escenarios únicos (Proceso × Dist × Var): {len(escenarios_unicos)}\n")

# ====================================================================================
# 2. TEST DIEBOLD-MARIANO MODIFICADO (NUEVA IMPLEMENTACIÓN)
#    Fixed-smoothing asymptotics con estimación espectral
# ====================================================================================

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano con fixed-smoothing asymptotics.
    
    Parameters
    ----------
    errors1, errors2 : array-like
        Pérdidas/errores de los dos métodos a comparar
    h : int
        Horizonte de pronóstico (default=1)
    
    Returns
    -------
    dm_stat : float
        Estadístico DM
    p_value : float
        P-valor bilateral
    mean_diff : float
        Diferencia media de pérdidas
    """
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    if T < 2: 
        return np.nan, np.nan, np.nan
    
    # Desviaciones centradas
    u = d - d_bar
    
    # Bandwidth para fixed-smoothing asymptotics
    m = max(1, int(np.floor(T**(1/3))))
    
    # Estimación espectral usando periodograma
    from scipy.fft import fft
    fft_u = fft(u)
    periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
    
    # Asegurar que m no exceda el tamaño del periodograma
    if m >= len(periodogram) - 1: 
        m = len(periodogram) - 2
    
    # Varianza de largo plazo (long-run variance)
    sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
    
    # Fallback si la estimación es no positiva
    if sigma_hat_sq <= 0: 
        sigma_hat_sq = np.var(d, ddof=1) / T
    
    # Estadístico DM
    dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
    
    # Grados de libertad para distribución t
    df_t = 2 * m
    
    # P-valor bilateral
    p_value = 2 * (1 - stats.t.cdf(abs(dm_stat), df_t))
    
    return dm_stat, p_value, d_bar


# ====================================================================================
# 3. FUNCIÓN CENTRAL: tabla de p-valores por modelo (POR TIPO DE ESCENARIO)
# ====================================================================================

def compute_pvalue_table_props(model, tipo_escenario=None):
    """
    Genera la tabla detallada de p-valores para un modelo comparando proporciones.
    
    Si tipo_escenario es None: usa todos los escenarios
    Si tipo_escenario es especificado: filtra solo ese tipo
    
    Filas    = escenario único (Proceso | Dist | Var)
    Columnas = par de proporciones (Prop=p1 vs Prop=p2)

    Emparejamiento: inner-join por 'Paso' entre Prop=p1 y Prop=p2
    dentro del mismo escenario → misma réplica de simulación.
    """
    # Filtrar datos si se especifica tipo de escenario
    if tipo_escenario:
        df_filtrado = df[df['Tipo_Proceso'] == tipo_escenario]
        escenarios_usar = sorted(df_filtrado['escenario_id'].unique())
    else:
        df_filtrado = df
        escenarios_usar = escenarios_unicos
    
    prop_pairs = list(combinations(prop_values, 2))
    col_names = [f"Prop={p1}% vs Prop={p2}%" for p1, p2 in prop_pairs]

    rows_pval, rows_dm, rows_diff = [], [], []

    for esc_id in escenarios_usar:
        subset_esc = df_filtrado[df_filtrado['escenario_id'] == esc_id]

        row_pval, row_dm, row_diff = [], [], []

        for (p1, p2) in prop_pairs:
            s1 = (subset_esc[subset_esc['Prop_Calib_Pct'] == p1][['Paso', model]]
                  .rename(columns={model: 'l1'}))
            s2 = (subset_esc[subset_esc['Prop_Calib_Pct'] == p2][['Paso', model]]
                  .rename(columns={model: 'l2'}))

            paired = pd.merge(s1, s2, on='Paso', how='inner').dropna()
            n_pairs = len(paired)

            if n_pairs < 5:
                row_pval.append(np.nan)
                row_dm.append(np.nan)
                row_diff.append(np.nan)
                continue

            dm_stat, p_val, mean_diff = modified_diebold_mariano_test(
                paired['l1'].values, paired['l2'].values, h=1)

            row_pval.append(p_val)
            row_dm.append(dm_stat)
            row_diff.append(mean_diff)

        rows_pval.append(row_pval)
        rows_dm.append(row_dm)
        rows_diff.append(row_diff)

    df_pval = pd.DataFrame(rows_pval, index=escenarios_usar, columns=col_names)
    df_dm   = pd.DataFrame(rows_dm,   index=escenarios_usar, columns=col_names)
    df_diff = pd.DataFrame(rows_diff, index=escenarios_usar, columns=col_names)

    return df_pval, df_dm, df_diff


# ====================================================================================
# 4. CALCULAR TODAS LAS TABLAS Y BONFERRONI GLOBAL (POR TIPO)
# ====================================================================================

def compute_all_tables(tipo_escenario=None):
    """
    Recorre todos los modelos, calcula las tablas de p-valores y determina
    el α de Bonferroni global.
    
    Si tipo_escenario es None: análisis general
    Si tipo_escenario especificado: solo ese tipo
    """
    tipo_str = tipo_escenario if tipo_escenario else "GENERAL"
    print(f"\n[Calculando tablas de p-valores - {tipo_str}]")

    all_pval_tables = {}
    all_dm_tables   = {}
    all_diff_tables = {}
    n_tests_total   = 0

    for model in model_cols:
        df_pval, df_dm, df_diff = compute_pvalue_table_props(model, tipo_escenario)
        all_pval_tables[model] = df_pval
        all_dm_tables[model]   = df_dm
        all_diff_tables[model] = df_diff
        n_valid = int(df_pval.notna().sum().sum())
        n_tests_total += n_valid

    alpha_bonf = 0.05 / n_tests_total if n_tests_total > 0 else 0.05
    print(f"✓ Tests válidos totales: {n_tests_total}")
    print(f"✓ α Bonferroni = 0.05 / {n_tests_total} = {alpha_bonf:.8f}")

    # ECRPS medio por (Prop_Calib_Pct, modelo)
    df_usar = df if tipo_escenario is None else df[df['Tipo_Proceso'] == tipo_escenario]
    ecrps_data = {model: [df_usar[df_usar['Prop_Calib_Pct'] == prop][model].mean()
                           for prop in prop_values]
                  for model in model_cols}
    df_ecrps = pd.DataFrame(ecrps_data, index=prop_values)
    df_ecrps.index.name = 'Prop_Calib_%'

    return all_pval_tables, all_dm_tables, all_diff_tables, alpha_bonf, df_ecrps


# ====================================================================================
# 5. TABLA RESUMEN: % escenarios significativos por (Proporción, modelo)
# ====================================================================================

def compute_summary_pct(all_pval_tables, alpha_bonf):
    """
    Tabla resumen % escenarios con diferencia significativa.
    Filas = Proporciones, Columnas = Modelos
    """
    pct_data = {}
    for model in model_cols:
        df_pval = all_pval_tables[model]
        col_pct = []
        for prop in prop_values:
            cols_prop = [c for c in df_pval.columns if f"Prop={prop}%" in c]
            vals = df_pval[cols_prop].values.flatten()
            vals = vals[~np.isnan(vals)]
            if len(vals) == 0:
                col_pct.append(np.nan)
            else:
                col_pct.append(100.0 * np.sum(vals < alpha_bonf) / len(vals))
        pct_data[model] = col_pct

    df_pct = pd.DataFrame(pct_data, index=prop_values)
    df_pct.index.name = 'Prop_Calib_%'
    return df_pct


# ====================================================================================
# 6. VISUALIZACIÓN: HEATMAP COMBINADO (ECRPS + % SIGNIFICANCIA)
# ====================================================================================

def plot_combined_heatmap(df_ecrps, df_pct, alpha_bonf, tipo_escenario=None):
    """
    Heatmap resumen:
      Fondo = % comparaciones significativas (verde alto, rojo bajo)
      Anotación = "ECRPS\n(% sig)"
    """
    suffix = tipo_escenario.replace(" ", "_").replace("(", "").replace(")", "") if tipo_escenario else "General"
    
    n_rows = len(prop_values)
    n_cols = len(model_cols)

    fig, ax = plt.subplots(figsize=(max(14, n_cols * 1.4),
                                    max(5,  n_rows * 0.9)))

    annot = np.empty((n_rows, n_cols), dtype=object)
    for i, prop in enumerate(prop_values):
        for j, model in enumerate(model_cols):
            ecrps_val = df_ecrps.iloc[i, j]
            pct_val   = df_pct.iloc[i, j]
            if np.isnan(pct_val):
                annot[i, j] = f"{ecrps_val:.3f}\n(—)"
            else:
                annot[i, j] = f"{ecrps_val:.3f}\n({pct_val:.0f}%)"

    sns.heatmap(
        df_pct,
        annot=annot,
        fmt='',
        cmap='RdYlGn',
        vmin=0, vmax=100, center=50,
        linewidths=0.6, linecolor='white',
        cbar_kws={'label': '% Escenarios con diferencia significativa (Bonferroni)',
                  'shrink': 0.7},
        ax=ax,
        annot_kws={'fontsize': 8, 'va': 'center'}
    )

    title = f'ECRPS Medio y % Escenarios con Diferencia Significativa por Proporción de Calibración'
    if tipo_escenario:
        title += f'\n{tipo_escenario}'
    title += f'\nCelda: ECRPS medio  |  (%) escenarios sign. Bonferroni  α = {alpha_bonf:.6f}'
    
    ax.set_title(title, fontweight='bold', fontsize=12, pad=15)
    ax.set_xlabel('Modelo', fontweight='bold', fontsize=11)
    ax.set_ylabel('Proporción de Calibración (%)', fontweight='bold', fontsize=11)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha='right', fontsize=9)
    ax.set_yticklabels([f'{prop}%' for prop in prop_values], rotation=0, fontsize=10)

    plt.tight_layout()
    plt.savefig(output_dir / f'heatmap_ecrps_significancia_{suffix}.png',
                dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Heatmap combinado guardado: {suffix}")


# ====================================================================================
# 7. HEATMAP: Z-SCORES POR MODELO
# ====================================================================================

def plot_heatmap_zscores(data, tipo_escenario=None):
    """
    Heatmap de Modelos (filas) vs Proporciones (columnas) con Z-scores
    Z-score calculado POR MODELO (estandarización por fila)
    """
    suffix = tipo_escenario.replace(" ", "_").replace("(", "").replace(")", "") if tipo_escenario else "General"
    
    if tipo_escenario:
        data = data[data['Tipo_Proceso'] == tipo_escenario]
    
    # Crear matriz de ECRPS promedio
    ecrps_data = []
    for model in model_cols:
        row = []
        for prop in prop_values:
            ecrps_mean = data[data['Prop_Calib_Pct'] == prop][model].mean()
            row.append(ecrps_mean)
        ecrps_data.append(row)
    
    ecrps_df = pd.DataFrame(ecrps_data, 
                             index=model_cols, 
                             columns=[f'{p}%' for p in prop_values])
    
    # Calcular Z-scores POR MODELO (por fila)
    zscore_df = ecrps_df.apply(lambda row: (row - row.mean()) / row.std() if row.std() > 0 else 0, axis=1)
    
    # Guardar datos
    ecrps_df.to_excel(output_dir / f'heatmap_ecrps_{suffix}.xlsx')
    zscore_df.to_excel(output_dir / f'heatmap_zscores_{suffix}.xlsx')
    
    # Graficar
    plt.figure(figsize=(10, max(8, len(model_cols) * 0.4)))
    sns.heatmap(zscore_df, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, vmin=-2, vmax=2,
                cbar_kws={'label': 'Z-score'},
                linewidths=0.5, linecolor='gray')
    
    title = 'Z-scores de ECRPS: Modelos vs Proporción de Calibración'
    if tipo_escenario:
        title += f'\n{tipo_escenario}'
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Proporción de Calibración', fontsize=12)
    plt.ylabel('Modelo', fontsize=12)
    plt.tight_layout()
    plt.savefig(output_dir / f'heatmap_zscore_{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Heatmap Z-scores generado: {suffix}")


# ====================================================================================
# 8. GRÁFICA DE MEJORA RELATIVA
# ====================================================================================

def plot_relative_improvement(data, tipo_escenario=None):
    """Mejora relativa (%) respecto a la proporción base (primera proporción)"""
    suffix = tipo_escenario.replace(" ", "_").replace("(", "").replace(")", "") if tipo_escenario else "General"
    
    if tipo_escenario:
        data = data[data['Tipo_Proceso'] == tipo_escenario]
    
    baseline = prop_values[0]
    improvements = {}
    
    for model in model_cols:
        base_perf = data[data['Prop_Calib_Pct'] == baseline][model].mean()
        model_impr = []
        for prop in prop_values:
            current = data[data['Prop_Calib_Pct'] == prop][model].mean()
            improvement = ((base_perf - current) / base_perf) * 100 if base_perf != 0 else 0
            model_impr.append(improvement)
        improvements[model] = model_impr
    
    # Guardar datos
    improvements_df = pd.DataFrame(improvements, index=prop_values)
    improvements_df.index.name = 'Prop_Calib_%'
    improvements_df.to_excel(output_dir / f'mejora_relativa_{suffix}.xlsx')
    
    # Graficar
    plt.figure(figsize=(12, 7))
    for model, values in improvements.items():
        plt.plot(prop_values, values, marker='o', label=model, 
                color=MODEL_COLORS[model], linewidth=2.5, markersize=8)
    
    plt.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    
    title = f'Mejora Relativa vs Proporción Base ({baseline}%)'
    if tipo_escenario:
        title += f' - {tipo_escenario}'
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Proporción de Calibración (%)', fontsize=12)
    plt.ylabel('Mejora Relativa (%)', fontsize=12)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    plt.grid(True, alpha=0.3, linestyle=':', linewidth=0.8)
    plt.tight_layout()
    plt.savefig(output_dir / f'mejora_relativa_{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Mejora relativa generada: {suffix}")


# ====================================================================================
# 9. GRÁFICA DE EVOLUCIÓN POR PROPORCIÓN
# ====================================================================================

def plot_evolution_by_proportion(data, scenario_name=None):
    """Evolución del ECRPS de cada modelo a través de proporciones"""
    if scenario_name:
        data = data[data['Tipo_Proceso'] == scenario_name]
        suffix = scenario_name.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Evolución del ECRPS por Proporción\n{scenario_name}'
    else:
        suffix = "General"
        title = 'Evolución del ECRPS por Proporción\nTodos los Escenarios'
    
    plt.figure(figsize=(12, 7))
    for model in model_cols:
        means = [data[data['Prop_Calib_Pct'] == p][model].mean() for p in prop_values]
        plt.plot(prop_values, means, marker='o', label=model, 
                color=MODEL_COLORS[model], linewidth=2.5, markersize=8)
    
    plt.xlabel('Proporción de Calibración (%)', fontsize=12)
    plt.ylabel('ECRPS Promedio', fontsize=12)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    plt.grid(True, alpha=0.3, linestyle=':', linewidth=0.8)
    plt.tight_layout()
    plt.savefig(output_dir / f'evolucion_proporciones_{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Evolución por proporción generada: {suffix}")


# ====================================================================================
# 10. PROPORCIÓN ÓPTIMA POR MODELO
# ====================================================================================

def analyze_optimal_proportion(data, scenario_name=None):
    """Identifica la proporción óptima para cada modelo"""
    if scenario_name:
        data = data[data['Tipo_Proceso'] == scenario_name]
        suffix = scenario_name.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Proporción Óptima por Modelo\n{scenario_name}'
    else:
        suffix = "General"
        title = 'Proporción Óptima por Modelo\nTodos los Escenarios'
    
    optimal_results = []
    
    for model in model_cols:
        prop_means = {}
        for prop in prop_values:
            prop_means[prop] = data[data['Prop_Calib_Pct'] == prop][model].mean()
        
        optimal_prop = min(prop_means, key=prop_means.get)
        worst_prop = max(prop_means, key=prop_means.get)
        
        optimal_results.append({
            'Modelo': model,
            'Proporcion_Optima_%': optimal_prop,
            'ECRPS_Optimo': prop_means[optimal_prop],
            'Proporcion_Peor_%': worst_prop,
            'ECRPS_Peor': prop_means[worst_prop],
            'Diferencia_%': ((prop_means[worst_prop] - prop_means[optimal_prop]) / prop_means[optimal_prop]) * 100
        })
    
    optimal_df = pd.DataFrame(optimal_results).sort_values('ECRPS_Optimo')
    
    # Gráfica
    fig, ax = plt.subplots(figsize=(12, 8))
    
    y_pos = np.arange(len(optimal_df))
    colors = [MODEL_COLORS[m] for m in optimal_df['Modelo']]
    
    bars = ax.barh(y_pos, optimal_df['ECRPS_Optimo'], 
                   color=colors, alpha=0.85, edgecolor='black', linewidth=1.5)
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(optimal_df['Modelo'])
    ax.set_xlabel('ECRPS en Proporción Óptima', fontsize=11, fontweight='bold')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Anotar proporción óptima
    for i, (bar, (_, row)) in enumerate(zip(bars, optimal_df.iterrows())):
        width = bar.get_width()
        ax.text(width + width*0.01, bar.get_y() + bar.get_height()/2.,
               f"{row['Proporcion_Optima_%']}%",
               ha='left', va='center', fontsize=9, fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.5))
    
    plt.tight_layout()
    plt.savefig(output_dir / f'proporcion_optima_{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Guardar tabla
    optimal_df.to_excel(output_dir / f'proporcion_optima_{suffix}.xlsx', index=False)
    
    print(f"✓ Análisis proporción óptima generado: {suffix}")
    
    return optimal_df


# ====================================================================================
# 11. ANÁLISIS DE SENSIBILIDAD A LA PROPORCIÓN
# ====================================================================================

def analyze_sensitivity_to_proportion(data, scenario_name=None):
    """Identifica qué modelos son más sensibles a cambios en la proporción"""
    if scenario_name:
        data = data[data['Tipo_Proceso'] == scenario_name]
        suffix = scenario_name.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Sensibilidad a Proporción de Calibración\n{scenario_name}'
    else:
        suffix = "General"
        title = 'Sensibilidad a Proporción de Calibración\nTodos los Escenarios'
    
    sensitivity_results = []
    
    for model in model_cols:
        means_by_prop = []
        for prop in prop_values:
            means_by_prop.append(data[data['Prop_Calib_Pct'] == prop][model].mean())
        
        means_array = np.array(means_by_prop)
        
        # Métricas de sensibilidad
        rango_absoluto = means_array.max() - means_array.min()
        rango_relativo = (rango_absoluto / means_array[0]) * 100
        volatilidad = np.std(means_array)
        cv = volatilidad / np.mean(means_array)
        
        # Correlación con proporción
        from scipy.stats import spearmanr
        corr, p_val = spearmanr(prop_values, means_array)
        
        sensitivity_results.append({
            'Modelo': model,
            'Rango_Absoluto': rango_absoluto,
            'Rango_Relativo_%': rango_relativo,
            'Volatilidad': volatilidad,
            'CV': cv,
            'Correlacion_Spearman': corr,
            'P_value': p_val,
            'ECRPS_Medio': means_array.mean()
        })
    
    sensitivity_df = pd.DataFrame(sensitivity_results).sort_values('Rango_Relativo_%', ascending=False)
    
    # Gráfica
    fig, ax = plt.subplots(figsize=(12, 8))
    
    colors = plt.cm.RdYlGn_r(sensitivity_df['Rango_Relativo_%'] / sensitivity_df['Rango_Relativo_%'].max())
    bars = ax.barh(sensitivity_df['Modelo'], sensitivity_df['Rango_Relativo_%'],
                  color=colors, alpha=0.85, edgecolor='black', linewidth=1.5)
    
    ax.set_xlabel('Variación Relativa Máxima (%)', fontsize=11, fontweight='bold')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    for bar, (_, row) in zip(bars, sensitivity_df.iterrows()):
        width = bar.get_width()
        ax.text(width + 0.5, bar.get_y() + bar.get_height()/2.,
               f"{row['Rango_Relativo_%']:.2f}%",
               ha='left', va='center', fontsize=8, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(output_dir / f'sensibilidad_proporcion_{suffix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    sensitivity_df.to_excel(output_dir / f'sensibilidad_proporcion_{suffix}.xlsx', index=False)
    
    print(f"✓ Análisis de sensibilidad generado: {suffix}")
    
    return sensitivity_df


# ====================================================================================
# 12. EXPORTAR EXCEL CONSOLIDADO
# ====================================================================================

def export_consolidated_excel(results_dict):
    """
    Excel consolidado con todas las tablas de p-valores y resúmenes
    
    results_dict = {
        'General': (all_pval_tables, all_dm_tables, all_diff_tables, alpha_bonf, df_ecrps, df_pct),
        'ARMA': (...),
        'ARIMA': (...),
        'SETAR': (...)
    }
    """
    excel_path = output_dir / 'DM_proporciones_consolidado.xlsx'

    with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
        wb = writer.book

        # Formatos
        fmt_hdr  = wb.add_format({'bold': True, 'bg_color': '#2E4057',
                                   'font_color': 'white', 'align': 'center',
                                   'valign': 'vcenter', 'border': 1})
        fmt_sig  = wb.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100',
                                   'align': 'center', 'border': 1})
        fmt_no   = wb.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006',
                                   'align': 'center', 'border': 1})

        sheet_counter = 1
        
        for tipo, (all_pval_tables, all_dm_tables, all_diff_tables, alpha_bonf, df_ecrps, df_pct) in results_dict.items():
            tipo_short = tipo.replace("Lineal Estacionario (ARMA)", "ARMA") \
                             .replace("Lineal No Estacionario (ARIMA)", "ARIMA") \
                             .replace("No Lineal Estacionario (SETAR)", "SETAR") \
                             .replace("General", "General")
            
            # Hoja 1: ECRPS medios
            sheet_name = f'{sheet_counter}_ECRPS_{tipo_short}'
            df_ecrps.to_excel(writer, sheet_name=sheet_name)
            ws = writer.sheets[sheet_name]
            ws.set_column('A:A', 12)
            ws.set_column('B:Z', 18)
            for c, col in enumerate(df_ecrps.columns, 1):
                ws.write(0, c, col, fmt_hdr)
            sheet_counter += 1
            
            # Hoja 2: % Significancia
            sheet_name = f'{sheet_counter}_PctSig_{tipo_short}'
            df_pct.to_excel(writer, sheet_name=sheet_name)
            ws2 = writer.sheets[sheet_name]
            ws2.set_column('A:A', 12)
            ws2.set_column('B:Z', 18)
            for c, col in enumerate(df_pct.columns, 1):
                ws2.write(0, c, col, fmt_hdr)
            for c in range(1, len(model_cols) + 1):
                ws2.conditional_format(1, c, len(prop_values), c,
                    {'type': 'cell', 'criteria': '>=', 'value': 50, 'format': fmt_sig})
                ws2.conditional_format(1, c, len(prop_values), c,
                    {'type': 'cell', 'criteria': '<',  'value': 50, 'format': fmt_no})
            sheet_counter += 1

        # Hoja final: Metodología
        meta = pd.DataFrame({
            'Parámetro': [
                'Unidad de escenario',
                'Emparejamiento dentro del test DM',
                'Test estadístico',
                'Método de estimación de varianza',
                'Corrección de multiplicidad',
                'α nominal',
                'Métrica de pérdida',
                'Modelos evaluados',
                'Proporciones evaluadas',
                'Tipos de escenario',
                'Escenarios únicos totales'
            ],
            'Valor': [
                'Combinación única (Proceso, Distribución, Varianza)',
                'Inner-join por columna Paso (réplica de simulación)',
                'Diebold-Mariano bilateral con fixed-smoothing asymptotics',
                'Estimación espectral con periodograma y bandwidth T^(1/3)',
                'Bonferroni sobre todos los tests válidos (por tipo)',
                0.05,
                'ECRPS',
                ', '.join(model_cols),
                ', '.join([f'{p}%' for p in prop_values]),
                ', '.join(escenarios),
                len(escenarios_unicos)
            ]
        })
        meta.to_excel(writer, sheet_name='ZZ_Metodologia', index=False)
        ws_m = writer.sheets['ZZ_Metodologia']
        ws_m.set_column('A:A', 45)
        ws_m.set_column('B:B', 80)

    print(f"\n✓ Excel consolidado guardado: {excel_path}")


# ====================================================================================
# 13. EJECUCIÓN PRINCIPAL
# ====================================================================================

if __name__ == "__main__":
    print("\n" + "=" * 80)
    print("ANÁLISIS DM CON FIXED-SMOOTHING ASYMPTOTICS — EFECTOS DE LA PROPORCIÓN")
    print("Escenario = combinación única (Proceso × Distribución × Varianza)")
    print("=" * 80)

    # Diccionario para guardar todos los resultados
    all_results = {}
    
    # ============================================================
    # ANÁLISIS GENERAL
    # ============================================================
    print("\n" + "="*80)
    print("ANÁLISIS GENERAL (TODOS LOS ESCENARIOS)")
    print("="*80)
    
    all_pval_tables, all_dm_tables, all_diff_tables, alpha_bonf, df_ecrps = compute_all_tables(tipo_escenario=None)
    df_pct = compute_summary_pct(all_pval_tables, alpha_bonf)
    
    all_results['General'] = (all_pval_tables, all_dm_tables, all_diff_tables, alpha_bonf, df_ecrps, df_pct)
    
    print("\n[Generando visualizaciones - GENERAL]")
    plot_combined_heatmap(df_ecrps, df_pct, alpha_bonf, tipo_escenario=None)
    plot_heatmap_zscores(df, tipo_escenario=None)
    plot_relative_improvement(df, tipo_escenario=None)
    plot_evolution_by_proportion(df, scenario_name=None)
    
    # ============================================================
    # ANÁLISIS POR TIPO DE ESCENARIO
    # ============================================================
    for tipo in escenarios:
        print("\n" + "="*80)
        print(f"ANÁLISIS: {tipo}")
        print("="*80)
        
        all_pval_tables, all_dm_tables, all_diff_tables, alpha_bonf, df_ecrps = compute_all_tables(tipo_escenario=tipo)
        df_pct = compute_summary_pct(all_pval_tables, alpha_bonf)
        
        all_results[tipo] = (all_pval_tables, all_dm_tables, all_diff_tables, alpha_bonf, df_ecrps, df_pct)
        
        print(f"\n[Generando visualizaciones - {tipo}]")
        plot_combined_heatmap(df_ecrps, df_pct, alpha_bonf, tipo_escenario=tipo)
        plot_heatmap_zscores(df, tipo_escenario=tipo)
        plot_relative_improvement(df, tipo_escenario=tipo)
        plot_evolution_by_proportion(df, scenario_name=tipo)
    
    # ============================================================
    # ANÁLISIS ADICIONALES (PROPORCIÓN ÓPTIMA Y SENSIBILIDAD)
    # ============================================================
    print("\n" + "="*80)
    print("ANÁLISIS DE PROPORCIÓN ÓPTIMA Y SENSIBILIDAD")
    print("="*80)
    
    optimal_general = analyze_optimal_proportion(df)
    for scen in escenarios:
        analyze_optimal_proportion(df, scen)
    
    sensitivity_general = analyze_sensitivity_to_proportion(df)
    for scen in escenarios:
        analyze_sensitivity_to_proportion(df, scen)
    
    # ============================================================
    # EXPORTAR EXCEL CONSOLIDADO
    # ============================================================
    print("\n" + "="*80)
    print("EXPORTANDO EXCEL CONSOLIDADO")
    print("="*80)
    export_consolidated_excel(all_results)
    
    # ============================================================
    # RESUMEN FINAL
    # ============================================================
    print("\n" + "=" * 80)
    print("✓ ANÁLISIS COMPLETADO")
    print(f"✓ Resultados en: {output_dir}")
    print("\nArchivos generados:")
    print("  1. HEATMAPS COMBINADOS (ECRPS + % Significancia):")
    print("     - 4 archivos .png (General + ARMA + ARIMA + SETAR)")
    print("  2. HEATMAPS Z-SCORES:")
    print("     - 4 archivos .png + 8 .xlsx (General + ARMA + ARIMA + SETAR)")
    print("  3. CURVAS DE MEJORA RELATIVA:")
    print("     - 4 archivos .png + 4 .xlsx (General + ARMA + ARIMA + SETAR)")
    print("  4. CURVAS DE EVOLUCIÓN:")
    print("     - 4 archivos .png (General + ARMA + ARIMA + SETAR)")
    print("  5. ANÁLISIS PROPORCIÓN ÓPTIMA:")
    print("     - 4 archivos .png + 4 .xlsx (General + ARMA + ARIMA + SETAR)")
    print("  6. ANÁLISIS DE SENSIBILIDAD:")
    print("     - 4 archivos .png + 4 .xlsx (General + ARMA + ARIMA + SETAR)")
    print("  7. EXCEL CONSOLIDADO:")
    print("     - 1 archivo con 9 hojas (2 por análisis + metodología)")
    print("\nMETODOLOGÍA:")
    print("  - Test DM: Fixed-smoothing asymptotics con estimación espectral")
    print("  - Bandwidth: T^(1/3) para estimación de varianza de largo plazo")
    print("  - Emparejamiento: por Paso dentro de cada escenario único")
    print("  - Corrección: Bonferroni por tipo de análisis")
    print("=" * 80 + "\n")


SIMULACIÓN 4: PROPORCIONES DE CALIBRACIÓN (N=240 FIJO)
CON TEST DIEBOLD-MARIANO MODIFICADO Y CORRECCIÓN DE BONFERRONI

Modelos evaluados: 9
Proporciones: [np.int64(10), np.int64(20), np.int64(30), np.int64(40), np.int64(50)]
Escenarios: ['Lineal Estacionario (ARMA)', 'Lineal No Estacionario (ARIMA)', 'No Lineal Estacionario (SETAR)']
Escenarios únicos (Proceso × Dist × Var): 420


ANÁLISIS DM CON FIXED-SMOOTHING ASYMPTOTICS — EFECTOS DE LA PROPORCIÓN
Escenario = combinación única (Proceso × Distribución × Varianza)

ANÁLISIS GENERAL (TODOS LOS ESCENARIOS)

[Calculando tablas de p-valores - GENERAL]
✓ Tests válidos totales: 37800
✓ α Bonferroni = 0.05 / 37800 = 0.00000132

[Generando visualizaciones - GENERAL]
✓ Heatmap combinado guardado: General
✓ Heatmap Z-scores generado: General
✓ Mejora relativa generada: General
✓ Evolución por proporción generada: General

ANÁLISIS: Lineal Estacionario (ARMA)

[Calculando tablas de p-valores - Lineal Estacionario (ARMA)]
✓ Tests válidos totales

# Analisis Simulacion h

### Preprocesamiento

In [5]:
import pandas as pd

# 1. Leer los archivos
arma_df = pd.read_excel("./datos/Simulacion_h/resultados_140_trayectorias_ARMA_FINAL.xlsx")
arima_df = pd.read_excel("./datos/Simulacion_h/resultados_140_trayectorias_ARIMA_FINAL.xlsx")
setar_df = pd.read_excel("./datos/Simulacion_h/resultados_140_trayectorias_SETAR_FINAL.xlsx")
LSPMW_df = pd.read_excel("./datos/Simulacion_h/resultados_LSPMW_todos_procesos.xlsx")


# 2. Asignar la columna ESCENARIO a cada dataframe
arma_df['ESCENARIO'] = "Lineal Estacionario"
arima_df['ESCENARIO'] = "Lineal No estacionario"
setar_df['ESCENARIO'] = "No lineal Estacionario"

# 3. Juntarlos uno bajo el otro (Concatenar)
df_total = pd.concat([arma_df, arima_df, setar_df], ignore_index=True)

# Seleccionar solo las columnas requeridas
columnas_deseadas = [
    "Paso", "Config", "Dist", "Var",  
    "Sieve Bootstrap", "LSPM",  "MCPS", 
    "DeepAR", "ESCENARIO"
]
df_total = df_total[columnas_deseadas]

# Definimos cuáles son las columnas que representan a los modelos predictivos
modelos = [
    "Sieve Bootstrap", "LSPM",  "MCPS", 
    "DeepAR"
]

# 4. Guardar el dataframe consolidado
df_total.to_excel("./datos/Simulacion_h/dataframe_consolidado.xlsx", index=False)

# 5. Generar y mostrar las tablas (Media y Mediana)
metricas = {'MEDIA': 'mean', 'MEDIANA': 'median'}

for nombre_metrica, funcion in metricas.items():
    # Calculamos el valor general según la métrica (mean o median)
    if funcion == 'mean':
        resumen_general = df_total[modelos].mean()
        resumen_escenarios = df_total.groupby('ESCENARIO')[modelos].mean().T
    else:
        resumen_general = df_total[modelos].median()
        resumen_escenarios = df_total.groupby('ESCENARIO')[modelos].median().T

    # Construimos la tabla final para esta métrica
    tabla_resumen = pd.DataFrame(index=modelos)
    tabla_resumen['General'] = resumen_general
    tabla_resumen['ARMA'] = resumen_escenarios['Lineal Estacionario']
    tabla_resumen['ARIMA'] = resumen_escenarios['Lineal No estacionario']
    tabla_resumen['SETAR'] = resumen_escenarios['No lineal Estacionario']

    # Determinar el Mejor_Escenario (valor mínimo entre los tres escenarios)
    escenarios_cols = ['ARMA', 'ARIMA', 'SETAR']
    tabla_resumen['Mejor_Escenario'] = tabla_resumen[escenarios_cols].idxmin(axis=1)

    # Imprimir resultado
    print(f"\n--- Tabla Comparativa de Modelos (Basada en {nombre_metrica}) ---")
    print(tabla_resumen.reset_index().rename(columns={'index': 'Modelo'}).to_string(index=False))


--- Tabla Comparativa de Modelos (Basada en MEDIA) ---
         Modelo  General     ARMA    ARIMA    SETAR Mejor_Escenario
Sieve Bootstrap 1.261013 0.793567 2.405825 0.583647           SETAR
           LSPM 1.423114 0.863196 2.811590 0.594555           SETAR
           MCPS 2.122821 0.885279 4.871372 0.611811           SETAR
         DeepAR 3.126949 0.833898 7.970984 0.575966           SETAR

--- Tabla Comparativa de Modelos (Basada en MEDIANA) ---
         Modelo  General     ARMA    ARIMA    SETAR Mejor_Escenario
Sieve Bootstrap 0.722924 0.622694 1.688908 0.497262           SETAR
           LSPM 0.786162 0.661725 1.946193 0.502726           SETAR
           MCPS 0.850184 0.660088 2.845228 0.528392           SETAR
         DeepAR 0.821647 0.635593 3.456565 0.473015           SETAR


### Analisis

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import normaltest
import warnings
from pathlib import Path
import itertools

warnings.filterwarnings('ignore')

# Configuración general mejorada
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titleweight'] = 'bold'
sns.set_style("whitegrid")
sns.set_palette("husl")

# Crear carpeta de resultados
output_dir = Path("./Resultados_analisis/Simulacion_h")
output_dir.mkdir(parents=True, exist_ok=True)

# Crear carpeta de interacciones
interactions_dir = output_dir / "Interacciones"
interactions_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
df = pd.read_excel("./datos/Simulacion_h/dataframe_consolidado.xlsx")

# 1) CAMBIO DE NOMBRES DE ESCENARIOS
df['ESCENARIO'] = df['ESCENARIO'].replace({
    "Lineal Estacionario": "Lineal Estacionario (ARMA)",
    "Lineal No estacionario": "Lineal No Estacionario (ARIMA)",
    "No lineal Estacionario": "No lineal Estacionario (SETAR)"
})

# Identificar columnas de modelos
var_cols = ['Paso', 'Config', 'Dist', 'Var', 'ESCENARIO']
original_model_cols = [col for col in df.columns if col not in var_cols]

# 2) ORGANIZAR MODELOS POR RENDIMIENTO EN "Lineal No Estacionario (ARIMA)" (Menor a mayor)
target_scenario = "Lineal No Estacionario (ARIMA)"
model_order_scores = df[df['ESCENARIO'] == target_scenario][original_model_cols].mean().sort_values()
model_cols = list(model_order_scores.index)

print("Nuevo orden de modelos (basado en Lineal No Estacionario (ARIMA)):")
for i, m in enumerate(model_cols, 1):
    print(f"{i}. {m}: {model_order_scores[m]:.4f}")

# Mapeo de escenarios
escenarios_map = {
    'Lineal Estacionario (ARMA)': 'Lineal Estacionario (ARMA)',
    'Lineal No Estacionario (ARIMA)': 'Lineal No Estacionario (ARIMA)',
    'No lineal Estacionario (SETAR)': 'No lineal Estacionario (SETAR)'
}

# Definir colores para cada modelo
palette = sns.color_palette("husl", len(model_cols))
model_colors = {model: palette[i] for i, model in enumerate(model_cols)}

# ====================================================================================
# SECCIÓN 1: RENDIMIENTO POR ESCENARIOS
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 1: RENDIMIENTO POR ESCENARIOS")
print("="*80)

def plot_performance_by_scenario():
    fig, ax = plt.subplots(figsize=(14, 8))
    
    scenarios = ["Lineal Estacionario (ARMA)", "Lineal No Estacionario (ARIMA)", "No lineal Estacionario (SETAR)"]
    x = np.arange(len(model_cols))
    width = 0.25 
    
    scenario_colors = {
        'Lineal Estacionario (ARMA)': '#5D3FD3',    
        'Lineal No Estacionario (ARIMA)': '#808080', 
        'No lineal Estacionario (SETAR)': '#00A36C'  
    }
    
    for idx, scenario in enumerate(scenarios):
        means = [df[df['ESCENARIO'] == scenario][model].mean() for model in model_cols]
        position = x + (idx - 1) * width
        
        bars = ax.bar(position, means, width, label=scenario, 
                     color=scenario_colors[scenario], alpha=0.85, edgecolor='black', linewidth=0.5)
        
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2f}', ha='center', va='bottom', fontsize=7, rotation=0)
    
    ax.set_xlabel('Modelo (Ordenados por desempeño en ARIMA)', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title('Rendimiento de Modelos por Escenario (ECRPS)', fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(model_cols, rotation=45, ha='right', fontsize=9)
    ax.legend(loc='upper left', ncol=1, fontsize=10, framealpha=0.9)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '1.1_rendimiento_por_escenario.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 1.1 guardada")

plot_performance_by_scenario()

def plot_relative_performance():
    base_scenario = 'Lineal Estacionario (ARMA)'
    scenarios_compare = ['Lineal No Estacionario (ARIMA)', 'No lineal Estacionario (SETAR)']
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    y = np.arange(len(model_cols))
    height = 0.35  
    
    for idx, scenario in enumerate(scenarios_compare):
        changes = []
        for model in model_cols:
            base_value = df[df['ESCENARIO'] == base_scenario][model].mean()
            scenario_value = df[df['ESCENARIO'] == scenario][model].mean()
            pct_change = ((scenario_value - base_value) / base_value) * 100
            changes.append(pct_change)
        
        position = y + idx * height
        bars = ax.barh(position, changes, height, label=scenario, alpha=0.85, edgecolor='black', linewidth=0.5)
        
        for bar, val in zip(bars, changes):
            width = bar.get_width()
            ax.text(width + (1 if width > 0 else -1), bar.get_y() + bar.get_height()/2.,
                   f'{val:+.1f}%', ha='left' if val > 0 else 'right', 
                   va='center', fontsize=7)
    
    ax.set_yticks(y + height / 2)
    ax.set_yticklabels(model_cols, fontsize=10)
    ax.set_xlabel('Cambio Relativo (%)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    ax.set_title(f'Cambio Relativo en ECRPS vs. {base_scenario}', fontsize=14, fontweight='bold', pad=20)
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1.5)
    ax.legend(loc='best', fontsize=10, framealpha=0.9)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '1.2_cambio_relativo_escenario_base.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 1.2 guardada")

plot_relative_performance()

# ====================================================================================
# SECCIÓN 2: ANÁLISIS POR CONFIG
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 2: ANÁLISIS POR CONFIG")
print("="*80)

def plot_zscore_heatmap_config(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Z-scores de ECRPS por Configuración\n{scenario}'
    else:
        data_filtered = df.copy()
        title = 'Z-scores de ECRPS por Configuración\n(General)'
    
    pivot_data = data_filtered.groupby('Config')[model_cols].mean()
    z_scores = pivot_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(z_scores.T, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, cbar_kws={'label': 'Z-score'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=13, fontweight='bold', pad=20)
    ax.set_xlabel('Configuración', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    filename = f'2.1{suffix}_zscore_config.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 2.1{suffix} guardada")

plot_zscore_heatmap_config()
plot_zscore_heatmap_config('Lineal Estacionario (ARMA)', '.a')
plot_zscore_heatmap_config('Lineal No Estacionario (ARIMA)', '.b')
plot_zscore_heatmap_config('No lineal Estacionario (SETAR)', '.c')

def plot_variability_config(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Configuración\n{scenario}'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Configuración\n(General)'
    
    pivot_std = data_filtered.groupby('Config')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    fmt = '.2f' if suffix == '' else '.4f'
    sns.heatmap(pivot_std.T, annot=True, fmt=fmt, cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=13, fontweight='bold', pad=20)
    ax.set_xlabel('Configuración', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    filename = f'2.2{suffix}_variabilidad_config.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 2.2{suffix} guardada")

plot_variability_config()
plot_variability_config('Lineal Estacionario (ARMA)', '.a')
plot_variability_config('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_config('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 3: ANÁLISIS POR DIST
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 3: ANÁLISIS POR DIST")
print("="*80)

def plot_zscore_heatmap_dist(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Z-scores de ECRPS por Distribución\n{scenario}'
    else:
        data_filtered = df.copy()
        title = 'Z-scores de ECRPS por Distribución\n(General)'
    
    pivot_data = data_filtered.groupby('Dist')[model_cols].mean()
    z_scores = pivot_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(z_scores.T, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, cbar_kws={'label': 'Z-score'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=13, fontweight='bold', pad=20)
    ax.set_xlabel('Distribución', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    filename = f'3.1{suffix}_zscore_dist.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 3.1{suffix} guardada")

plot_zscore_heatmap_dist()
plot_zscore_heatmap_dist('Lineal Estacionario (ARMA)', '.a')
plot_zscore_heatmap_dist('Lineal No Estacionario (ARIMA)', '.b')
plot_zscore_heatmap_dist('No lineal Estacionario (SETAR)', '.c')

def plot_variability_dist(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Distribución\n{scenario}'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Distribución\n(General)'
    
    pivot_std = data_filtered.groupby('Dist')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=13, fontweight='bold', pad=20)
    ax.set_xlabel('Distribución', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    filename = f'3.2{suffix}_variabilidad_dist.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 3.2{suffix} guardada")

plot_variability_dist()
plot_variability_dist('Lineal Estacionario (ARMA)', '.a')
plot_variability_dist('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_dist('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 4: ANÁLISIS POR VAR
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 4: ANÁLISIS POR VAR")
print("="*80)

def plot_evolution_var(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Evolución de ECRPS por Varianza\n{scenario}'
    else:
        data_filtered = df.copy()
        title = 'Evolución de ECRPS por Varianza\n(General)'
    
    fig, ax = plt.subplots(figsize=(12, 7))
    var_values = sorted(data_filtered['Var'].unique())
    
    for model in model_cols:
        means = []
        for var in var_values:
            mean_val = data_filtered[data_filtered['Var'] == var][model].mean()
            means.append(mean_val)
        
        ax.plot(var_values, means, marker='o', label=model, color=model_colors[model],
                linewidth=2.5, markersize=7, alpha=0.85)
    
    ax.set_xlabel('Varianza', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=13, fontweight='bold', pad=20)
    ax.legend(loc='best', fontsize=9, ncol=2, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    filename = f'4.1{suffix}_evolucion_var.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 4.1{suffix} guardada")

plot_evolution_var()
plot_evolution_var('Lineal Estacionario (ARMA)', '.a')
plot_evolution_var('Lineal No Estacionario (ARIMA)', '.b')
plot_evolution_var('No lineal Estacionario (SETAR)', '.c')

def plot_variability_var(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Varianza\n{scenario}'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Varianza\n(General)'
    
    pivot_std = data_filtered.groupby('Var')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=13, fontweight='bold', pad=20)
    ax.set_xlabel('Varianza', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    filename = f'4.2{suffix}_variabilidad_var.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 4.2{suffix} guardada")

plot_variability_var()
plot_variability_var('Lineal Estacionario (ARMA)', '.a')
plot_variability_var('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_var('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)")
print("="*80)

def plot_evolution_paso(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Evolución de ECRPS por Horizonte de Pronóstico\n{scenario}'
    else:
        data_filtered = df.copy()
        title = 'Evolución de ECRPS por Horizonte de Pronóstico\n(General)'
    
    fig, ax = plt.subplots(figsize=(12, 7))
    pasos = sorted(data_filtered['Paso'].unique())
    
    for model in model_cols:
        means = []
        for paso in pasos:
            mean_val = data_filtered[data_filtered['Paso'] == paso][model].mean()
            means.append(mean_val)
        
        ax.plot(pasos, means, marker='o', label=model, color=model_colors[model],
                linewidth=2.5, markersize=7, alpha=0.85)
    
    ax.set_xlabel('Horizonte de Pronóstico', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=13, fontweight='bold', pad=20)
    ax.legend(loc='best', fontsize=9, ncol=2, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    filename = f'5.1{suffix}_evolucion_paso.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 5.1{suffix} guardada")

plot_evolution_paso()
plot_evolution_paso('Lineal Estacionario (ARMA)', '.a')
plot_evolution_paso('Lineal No Estacionario (ARIMA)', '.b')
plot_evolution_paso('No lineal Estacionario (SETAR)', '.c')

def plot_variability_paso(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Horizonte\n{scenario}'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Horizonte\n(General)'
    
    pivot_std = data_filtered.groupby('Paso')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=13, fontweight='bold', pad=20)
    ax.set_xlabel('Horizonte de Pronóstico', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    filename = f'5.2{suffix}_variabilidad_paso.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 5.2{suffix} guardada")

plot_variability_paso()
plot_variability_paso('Lineal Estacionario (ARMA)', '.a')
plot_variability_paso('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_paso('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 5.5: ANÁLISIS DE INTERACCIONES (FORMATO 2x2 MEJORADO)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 5.5: ANÁLISIS DE INTERACCIONES (FORMATO 2x2)")
print("="*80)

# INTERACCIÓN CONFIG × VAR - Formato 2x2
def plot_interaction_config_var_2x2(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title_base = 'General' if not scenario else scenario
    
    configs = sorted(data_filtered['Config'].unique())
    vars_val = sorted(data_filtered['Var'].unique())
    
    # Seleccionar 4 modelos representativos (mejor, peor, y dos intermedios)
    model_means = data_filtered[model_cols].mean().sort_values()
    n_models = len(model_cols)
    selected_models = [
        model_cols[0],  # Mejor
        model_cols[n_models // 3],  # Intermedio bajo
        model_cols[2 * n_models // 3],  # Intermedio alto
        model_cols[-1]  # Peor
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    colors_config = sns.color_palette("Set2", len(configs))
    
    for idx, model in enumerate(selected_models):
        ax = axes[idx]
        
        for c_idx, config in enumerate(configs):
            means = []
            for var in vars_val:
                mean_val = data_filtered[
                    (data_filtered['Config'] == config) & 
                    (data_filtered['Var'] == var)
                ][model].mean()
                means.append(mean_val)
            
            ax.plot(vars_val, means, marker='o', label=f'Config {config}', 
                   color=colors_config[c_idx], linewidth=2.5, markersize=8, alpha=0.85)
        
        # Personalización de cada subplot
        ax.set_title(f'{model}', fontsize=11, fontweight='bold', pad=10)
        ax.set_xlabel('Varianza', fontsize=10, fontweight='bold')
        ax.set_ylabel('ECRPS Promedio', fontsize=10, fontweight='bold')
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.legend(fontsize=8, loc='best', framealpha=0.9)
    
    plt.suptitle(f'Interacción Config × Var - Modelos Representativos\n{title_base}', 
                fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.3{suffix}_interaccion_config_var_2x2.png', 
                bbox_inches='tight', dpi=300)
    plt.close()
    print(f"✓ Gráfica 5.3{suffix} guardada (Config × Var 2x2)")

# INTERACCIÓN CONFIG × DIST - Formato 2x2
def plot_interaction_config_dist_2x2(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title_base = 'General' if not scenario else scenario
    
    configs = sorted(data_filtered['Config'].unique())
    dists = sorted(data_filtered['Dist'].unique())
    
    # Seleccionar 4 modelos representativos
    n_models = len(model_cols)
    selected_models = [
        model_cols[0],
        model_cols[n_models // 3],
        model_cols[2 * n_models // 3],
        model_cols[-1]
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    colors_config = sns.color_palette("Set1", len(configs))
    
    for idx, model in enumerate(selected_models):
        ax = axes[idx]
        
        x_pos = np.arange(len(dists))
        width = 0.25
        
        for c_idx, config in enumerate(configs):
            means = []
            for dist in dists:
                mean_val = data_filtered[
                    (data_filtered['Config'] == config) & 
                    (data_filtered['Dist'] == dist)
                ][model].mean()
                means.append(mean_val)
            
            offset = (c_idx - 1) * width
            bars = ax.bar(x_pos + offset, means, width, label=f'Config {config}',
                         color=colors_config[c_idx], alpha=0.85, edgecolor='black', linewidth=0.5)
            
            # Añadir valores en las barras
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.2f}', ha='center', va='bottom', fontsize=7)
        
        ax.set_title(f'{model}', fontsize=11, fontweight='bold', pad=10)
        ax.set_xlabel('Distribución', fontsize=10, fontweight='bold')
        ax.set_ylabel('ECRPS Promedio', fontsize=10, fontweight='bold')
        ax.set_xticks(x_pos)
        ax.set_xticklabels(dists, fontsize=9)
        ax.grid(axis='y', alpha=0.3, linestyle='--')
        ax.legend(fontsize=8, loc='best', framealpha=0.9)
    
    plt.suptitle(f'Interacción Config × Dist - Modelos Representativos\n{title_base}', 
                fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.4{suffix}_interaccion_config_dist_2x2.png', 
                bbox_inches='tight', dpi=300)
    plt.close()
    print(f"✓ Gráfica 5.4{suffix} guardada (Config × Dist 2x2)")

# INTERACCIÓN DIST × PASO - Formato 2x2
def plot_interaction_dist_paso_2x2(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title_base = 'General' if not scenario else scenario
    
    dists = sorted(data_filtered['Dist'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    
    # Seleccionar 4 modelos representativos
    n_models = len(model_cols)
    selected_models = [
        model_cols[0],
        model_cols[n_models // 3],
        model_cols[2 * n_models // 3],
        model_cols[-1]
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    colors_dist = sns.color_palette("viridis", len(dists))
    
    for idx, model in enumerate(selected_models):
        ax = axes[idx]
        
        for d_idx, dist in enumerate(dists):
            means = []
            for paso in pasos:
                mean_val = data_filtered[
                    (data_filtered['Dist'] == dist) & 
                    (data_filtered['Paso'] == paso)
                ][model].mean()
                means.append(mean_val)
            
            ax.plot(pasos, means, marker='o', label=f'Dist {dist}',
                   color=colors_dist[d_idx], linewidth=2.5, markersize=8, alpha=0.85)
        
        ax.set_title(f'{model}', fontsize=11, fontweight='bold', pad=10)
        ax.set_xlabel('Horizonte (Paso)', fontsize=10, fontweight='bold')
        ax.set_ylabel('ECRPS Promedio', fontsize=10, fontweight='bold')
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.legend(fontsize=8, loc='best', framealpha=0.9)
    
    plt.suptitle(f'Interacción Dist × Paso - Modelos Representativos\n{title_base}', 
                fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.5{suffix}_interaccion_dist_paso_2x2.png', 
                bbox_inches='tight', dpi=300)
    plt.close()
    print(f"✓ Gráfica 5.5{suffix} guardada (Dist × Paso 2x2)")

# INTERACCIÓN DIST × VAR - Formato 2x2
def plot_interaction_dist_var_2x2(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title_base = 'General' if not scenario else scenario
    
    dists = sorted(data_filtered['Dist'].unique())
    vars_val = sorted(data_filtered['Var'].unique())
    
    # Seleccionar 4 modelos representativos
    n_models = len(model_cols)
    selected_models = [
        model_cols[0],
        model_cols[n_models // 3],
        model_cols[2 * n_models // 3],
        model_cols[-1]
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    colors_dist = sns.color_palette("magma", len(dists))
    
    for idx, model in enumerate(selected_models):
        ax = axes[idx]
        
        for d_idx, dist in enumerate(dists):
            means = []
            for var in vars_val:
                mean_val = data_filtered[
                    (data_filtered['Dist'] == dist) & 
                    (data_filtered['Var'] == var)
                ][model].mean()
                means.append(mean_val)
            
            ax.plot(vars_val, means, marker='s', label=f'Dist {dist}',
                   color=colors_dist[d_idx], linewidth=2.5, markersize=8, alpha=0.85)
        
        ax.set_title(f'{model}', fontsize=11, fontweight='bold', pad=10)
        ax.set_xlabel('Varianza', fontsize=10, fontweight='bold')
        ax.set_ylabel('ECRPS Promedio', fontsize=10, fontweight='bold')
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.legend(fontsize=8, loc='best', framealpha=0.9)
    
    plt.suptitle(f'Interacción Dist × Var - Modelos Representativos\n{title_base}', 
                fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.6{suffix}_interaccion_dist_var_2x2.png', 
                bbox_inches='tight', dpi=300)
    plt.close()
    print(f"✓ Gráfica 5.6{suffix} guardada (Dist × Var 2x2)")

# INTERACCIÓN CONFIG × PASO - Formato 2x2
def plot_interaction_config_paso_2x2(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title_base = 'General' if not scenario else scenario
    
    configs = sorted(data_filtered['Config'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    
    # Seleccionar 4 modelos representativos
    n_models = len(model_cols)
    selected_models = [
        model_cols[0],
        model_cols[n_models // 3],
        model_cols[2 * n_models // 3],
        model_cols[-1]
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    colors_config = sns.color_palette("tab10", len(configs))
    
    for idx, model in enumerate(selected_models):
        ax = axes[idx]
        
        for c_idx, config in enumerate(configs):
            means = []
            for paso in pasos:
                mean_val = data_filtered[
                    (data_filtered['Config'] == config) & 
                    (data_filtered['Paso'] == paso)
                ][model].mean()
                means.append(mean_val)
            
            ax.plot(pasos, means, marker='^', label=f'Config {config}',
                   color=colors_config[c_idx], linewidth=2.5, markersize=8, alpha=0.85)
        
        ax.set_title(f'{model}', fontsize=11, fontweight='bold', pad=10)
        ax.set_xlabel('Horizonte (Paso)', fontsize=10, fontweight='bold')
        ax.set_ylabel('ECRPS Promedio', fontsize=10, fontweight='bold')
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.legend(fontsize=8, loc='best', framealpha=0.9)
    
    plt.suptitle(f'Interacción Config × Paso - Modelos Representativos\n{title_base}', 
                fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.7{suffix}_interaccion_config_paso_2x2.png', 
                bbox_inches='tight', dpi=300)
    plt.close()
    print(f"✓ Gráfica 5.7{suffix} guardada (Config × Paso 2x2)")

# INTERACCIÓN VAR × HORIZONTE - Formato 2x2
def plot_interaction_var_horizonte_2x2(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title_base = 'General' if not scenario else scenario
    
    vars_val = sorted(data_filtered['Var'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    
    # Seleccionar 4 modelos representativos
    n_models = len(model_cols)
    selected_models = [
        model_cols[0],
        model_cols[n_models // 3],
        model_cols[2 * n_models // 3],
        model_cols[-1]
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    colors_var = sns.color_palette("rocket", len(vars_val))
    
    for idx, model in enumerate(selected_models):
        ax = axes[idx]
        
        for v_idx, var in enumerate(vars_val):
            means = []
            for paso in pasos:
                mean_val = data_filtered[
                    (data_filtered['Var'] == var) & 
                    (data_filtered['Paso'] == paso)
                ][model].mean()
                means.append(mean_val)
            
            ax.plot(pasos, means, marker='d', label=f'Var {var}',
                   color=colors_var[v_idx], linewidth=2.5, markersize=8, alpha=0.85)
        
        ax.set_title(f'{model}', fontsize=11, fontweight='bold', pad=10)
        ax.set_xlabel('Horizonte (Paso)', fontsize=10, fontweight='bold')
        ax.set_ylabel('ECRPS Promedio', fontsize=10, fontweight='bold')
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.legend(fontsize=8, loc='best', framealpha=0.9)
    
    plt.suptitle(f'Interacción Var × Horizonte - Modelos Representativos\n{title_base}', 
                fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.8{suffix}_interaccion_var_horizonte_2x2.png', 
                bbox_inches='tight', dpi=300)
    plt.close()
    print(f"✓ Gráfica 5.8{suffix} guardada (Var × Horizonte 2x2)")

# Ejecutar todas las interacciones para todos los escenarios
for sc_name, sc_suf in [(None, ''), 
                        ('Lineal Estacionario (ARMA)', '.a'), 
                        ('Lineal No Estacionario (ARIMA)', '.b'), 
                        ('No lineal Estacionario (SETAR)', '.c')]:
    plot_interaction_config_var_2x2(sc_name, sc_suf)
    plot_interaction_config_dist_2x2(sc_name, sc_suf)
    plot_interaction_dist_paso_2x2(sc_name, sc_suf)
    plot_interaction_dist_var_2x2(sc_name, sc_suf)
    plot_interaction_config_paso_2x2(sc_name, sc_suf)
    plot_interaction_var_horizonte_2x2(sc_name, sc_suf)

# ====================================================================================
# SECCIÓN 6: ROBUSTEZ Y TEST DIEBOLD-MARIANO (CON PROCESO PASO A PASO)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 6: ROBUSTEZ Y TEST DIEBOLD-MARIANO")
print("="*80)

def plot_robustness():
    fig, ax = plt.subplots(figsize=(12, 8))
    
    cv_data = []
    for model in model_cols:
        cv = df[model].std() / df[model].mean()
        cv_data.append((model, cv))
    
    cv_df = pd.DataFrame(cv_data, columns=['Modelo', 'CV'])
    cv_df = cv_df.sort_values('CV')
    
    colors_cv = ['#2ecc71' if cv < cv_df['CV'].median() else '#e74c3c' 
                 for cv in cv_df['CV']]
    
    bars = ax.barh(cv_df['Modelo'], cv_df['CV'], color=colors_cv, alpha=0.8, edgecolor='black')
    
    for bar, cv in zip(bars, cv_df['CV']):
        width = bar.get_width()
        ax.text(width + 0.001, bar.get_y() + bar.get_height()/2.,
               f'{cv:.4f}', ha='left', va='center', fontsize=9)
    
    ax.set_xlabel('Coeficiente de Variación', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    ax.set_title('Robustez: Coeficiente de Variación\n(Menor valor = Mayor estabilidad)', 
                  fontsize=13, fontweight='bold', pad=20)
    ax.axvline(x=cv_df['CV'].median(), color='black', linestyle='--', 
              linewidth=1.5, alpha=0.5, label='Mediana')
    ax.legend(loc='best', fontsize=10)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '6.1_robustez_coeficiente_variacion.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 6.1 guardada")

plot_robustness()

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano con fixed-smoothing asymptotics
    """
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    if T < 2:
        return np.nan, np.nan
    
    u = d - d_bar
    m = max(1, int(np.floor(T**(1/3))))
    
    from scipy.fft import fft
    fft_u = fft(u)
    periodogram = np.abs(fft_u)**2 / (2 * np.pi * T)
    
    if m >= len(periodogram) - 1:
        m = len(periodogram) - 2
    
    sigma_hat_sq = 2 * np.pi * np.mean(periodogram[1:m+1])
    
    if sigma_hat_sq <= 0:
        sigma_hat_sq = np.var(d, ddof=1) / T
        if sigma_hat_sq <= 0:
            return 0, 1.0
    
    dm_stat = np.sqrt(T) * d_bar / np.sqrt(sigma_hat_sq)
    df = 2 * m
    p_value = 2 * (1 - stats.t.cdf(abs(dm_stat), df))
    
    return dm_stat, p_value

def compute_dm_tests_step_by_step(scenario=None, suffix=''):
    """
    PROCESO PASO A PASO VISIBLE EN EXCEL:
    PASO 1: Tabla de escenarios únicos (420 filas)
    PASO 2: Para cada escenario, calcular p-valores para cada par de modelos (6 columnas)
    PASO 3: Clasificar si es significativo o no
    PASO 4: Reducir a tablas resumen
    """
    
    print("\n" + "-"*80)
    print("CÁLCULO TEST DIEBOLD-MARIANO - PROCESO PASO A PASO")
    print("-"*80)
    
    # Determinar tipo de análisis
    analysis_type = {
        '': 'GENERAL',
        '.a': 'ARMA',
        '.b': 'ARIMA',
        '.c': 'SETAR'
    }
    tipo = analysis_type.get(suffix, 'GENERAL')
    
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
    else:
        data_filtered = df.copy()
    
    print(f"Tipo de análisis: {tipo}")
    
    # PASO 1: Identificar escenarios únicos
    scenario_cols = ['Paso', 'Config', 'Dist', 'Var']
    unique_scenarios = data_filtered[scenario_cols].drop_duplicates().reset_index(drop=True)
    n_scenarios = len(unique_scenarios)
    
    print(f"PASO 1: Escenarios únicos identificados: {n_scenarios}")
    
    # Generar pares de modelos
    from itertools import combinations
    model_pairs = list(combinations(model_cols, 2))
    n_comparisons = len(model_pairs)
    
    print(f"Número de comparaciones por escenario: {n_comparisons}")
    
    # Calcular alpha con Bonferroni
    alpha = 0.05
    alpha_bonferroni = alpha / n_comparisons
    
    print(f"α nominal: {alpha}")
    print(f"α Bonferroni: {alpha_bonferroni:.6f}")
    
    # PASO 2: Crear tabla 420 × 6 con p-valores
    print("\nPASO 2: Calculando p-valores para cada escenario y par de modelos...")
    
    # Crear columnas para cada comparación
    comparison_names = [f"{m1}_vs_{m2}" for m1, m2 in model_pairs]
    pvalue_columns = [f"pval_{comp}" for comp in comparison_names]
    dmstat_columns = [f"dmstat_{comp}" for comp in comparison_names]
    mean_diff_columns = [f"meandiff_{comp}" for comp in comparison_names]
    
    # DataFrame para almacenar resultados por escenario
    scenario_results = unique_scenarios.copy()
    scenario_results['Escenario_ID'] = range(1, n_scenarios + 1)
    
    # Añadir columnas de p-valores, estadísticos DM y diferencias de medias
    for comp_name in comparison_names:
        scenario_results[f'pval_{comp_name}'] = np.nan
        scenario_results[f'dmstat_{comp_name}'] = np.nan
        scenario_results[f'meandiff_{comp_name}'] = np.nan
        scenario_results[f'mean_A_{comp_name}'] = np.nan
        scenario_results[f'mean_B_{comp_name}'] = np.nan
    
    # Calcular para cada escenario
    for idx, scenario_row in unique_scenarios.iterrows():
        # Filtrar datos para este escenario
        scenario_data = data_filtered[
            (data_filtered['Paso'] == scenario_row['Paso']) &
            (data_filtered['Config'] == scenario_row['Config']) &
            (data_filtered['Dist'] == scenario_row['Dist']) &
            (data_filtered['Var'] == scenario_row['Var'])
        ]
        
        paso_h = int(scenario_row['Paso'])
        
        # Para cada par de modelos
        for (model_a, model_b), comp_name in zip(model_pairs, comparison_names):
            errors_a = scenario_data[model_a].values
            errors_b = scenario_data[model_b].values
            
            if len(errors_a) > 0 and len(errors_b) > 0:
                # Calcular test DM
                dm_stat, p_value = modified_diebold_mariano_test(errors_a, errors_b, h=paso_h)
                
                # Calcular medias
                mean_a = np.mean(errors_a)
                mean_b = np.mean(errors_b)
                mean_diff = mean_a - mean_b
                
                # Guardar en tabla
                scenario_results.loc[idx, f'pval_{comp_name}'] = p_value
                scenario_results.loc[idx, f'dmstat_{comp_name}'] = dm_stat
                scenario_results.loc[idx, f'meandiff_{comp_name}'] = mean_diff
                scenario_results.loc[idx, f'mean_A_{comp_name}'] = mean_a
                scenario_results.loc[idx, f'mean_B_{comp_name}'] = mean_b
    
    print(f"✓ Tabla de {n_scenarios} escenarios × {n_comparisons} comparaciones completada")
    
    # PASO 3: Clasificar significancia
    print("\nPASO 3: Clasificando significancia estadística...")
    
    significance_table = scenario_results[['Escenario_ID', 'Paso', 'Config', 'Dist', 'Var']].copy()
    
    for comp_name in comparison_names:
        pval_col = f'pval_{comp_name}'
        sig_col = f'sig_{comp_name}'
        
        significance_table[sig_col] = scenario_results[pval_col] < alpha_bonferroni
        significance_table[sig_col] = significance_table[sig_col].map({True: 'SÍ', False: 'NO'})
    
    print(f"✓ Tabla de significancia creada")
    
    # PASO 4: Reducir a resumen
    print("\nPASO 4: Creando tablas resumen...")
    
    # Tabla resumen por comparación
    summary_by_comparison = []
    
    for (model_a, model_b), comp_name in zip(model_pairs, comparison_names):
        pval_col = f'pval_{comp_name}'
        dmstat_col = f'dmstat_{comp_name}'
        mean_a_col = f'mean_A_{comp_name}'
        mean_b_col = f'mean_B_{comp_name}'
        
        n_tests = scenario_results[pval_col].notna().sum()
        n_significant = (scenario_results[pval_col] < alpha_bonferroni).sum()
        pct_significant = (n_significant / n_tests * 100) if n_tests > 0 else 0
        
        mean_ecrps_a = scenario_results[mean_a_col].mean()
        mean_ecrps_b = scenario_results[mean_b_col].mean()
        
        # A es mejor cuando tiene menor ECRPS
        n_a_better = (scenario_results[mean_a_col] < scenario_results[mean_b_col]).sum()
        pct_a_better = (n_a_better / n_tests * 100) if n_tests > 0 else 0
        
        summary_by_comparison.append({
            'Modelo_A': model_a,
            'Modelo_B': model_b,
            'Comparación': f'{model_a} vs {model_b}',
            'N_Escenarios_Evaluados': n_tests,
            'N_Significativos': n_significant,
            'Pct_Significativos': pct_significant,
            'ECRPS_Medio_A': mean_ecrps_a,
            'ECRPS_Medio_B': mean_ecrps_b,
            'Diferencia_Media': mean_ecrps_a - mean_ecrps_b,
            'N_A_Mejor_que_B': n_a_better,
            'Pct_A_Mejor_que_B': pct_a_better
        })
    
    summary_df = pd.DataFrame(summary_by_comparison)
    
    print(f"✓ Tabla resumen por comparación creada ({len(summary_df)} filas)")
    
    # Matriz modelo vs modelo
    n_models = len(model_cols)
    sig_matrix = np.zeros((n_models, n_models))
    ecrps_matrix = np.zeros((n_models, n_models))
    
    for idx_a, model_a in enumerate(model_cols):
        for idx_b, model_b in enumerate(model_cols):
            if idx_a == idx_b:
                # Diagonal: ECRPS medio del modelo
                model_data = data_filtered[model_a]
                ecrps_matrix[idx_a, idx_b] = model_data.mean()
            else:
                # Buscar comparación
                comp_found = False
                for row in summary_by_comparison:
                    if row['Modelo_A'] == model_a and row['Modelo_B'] == model_b:
                        sig_matrix[idx_a, idx_b] = row['Pct_Significativos']
                        ecrps_matrix[idx_a, idx_b] = row['ECRPS_Medio_A']
                        comp_found = True
                        break
                    elif row['Modelo_A'] == model_b and row['Modelo_B'] == model_a:
                        sig_matrix[idx_a, idx_b] = row['Pct_Significativos']
                        ecrps_matrix[idx_a, idx_b] = row['ECRPS_Medio_B']
                        comp_found = True
                        break
    
    sig_matrix_df = pd.DataFrame(sig_matrix, index=model_cols, columns=model_cols)
    ecrps_matrix_df = pd.DataFrame(ecrps_matrix, index=model_cols, columns=model_cols)
    
    print(f"✓ Matrices modelo vs modelo creadas")
    
    # GUARDAR EN EXCEL CON MÚLTIPLES HOJAS
    excel_filename = f'6.2{suffix}_dm_proceso_completo.xlsx'
    
    with pd.ExcelWriter(output_dir / excel_filename, engine='openpyxl') as writer:
        # Hoja 0: Metodología
        method_info = pd.DataFrame({
            'Parámetro': ['Tipo de análisis', 'Alpha nominal', 'Alpha Bonferroni', 
                          'Número de modelos', 'Comparaciones por escenario', 
                          'Número de escenarios únicos', 'Total de tests',
                          'Test estadístico', 'Distribución'],
            'Valor': [tipo, alpha, alpha_bonferroni, len(model_cols), n_comparisons,
                      n_scenarios, n_scenarios * n_comparisons, 
                      'Diebold-Mariano (HLN)', 't-Student']
        })
        method_info.to_excel(writer, sheet_name='0_Metodologia', index=False)
        
        # Hoja 1: PASO 1 - Escenarios únicos (420 filas)
        unique_scenarios_display = unique_scenarios.copy()
        unique_scenarios_display.insert(0, 'Escenario_ID', range(1, n_scenarios + 1))
        unique_scenarios_display.to_excel(writer, sheet_name='1_Escenarios_Unicos', index=False)
        
        # Hoja 2: PASO 2 - Tabla 420×6 con P-valores
        pvalue_table = scenario_results[['Escenario_ID', 'Paso', 'Config', 'Dist', 'Var'] + 
                                        [f'pval_{comp}' for comp in comparison_names]].copy()
        pvalue_table.to_excel(writer, sheet_name='2_PValores_Por_Escenario', index=False)
        
        # Hoja 2b: Estadísticos DM
        dmstat_table = scenario_results[['Escenario_ID', 'Paso', 'Config', 'Dist', 'Var'] + 
                                        [f'dmstat_{comp}' for comp in comparison_names]].copy()
        dmstat_table.to_excel(writer, sheet_name='2b_DMStats_Por_Escenario', index=False)
        
        # Hoja 2c: Medias por modelo
        means_table = scenario_results[['Escenario_ID', 'Paso', 'Config', 'Dist', 'Var']].copy()
        for comp_name in comparison_names:
            means_table[f'mean_A_{comp_name}'] = scenario_results[f'mean_A_{comp_name}']
            means_table[f'mean_B_{comp_name}'] = scenario_results[f'mean_B_{comp_name}']
        means_table.to_excel(writer, sheet_name='2c_Medias_Por_Escenario', index=False)
        
        # Hoja 3: PASO 3 - Clasificación de significancia
        significance_table.to_excel(writer, sheet_name='3_Clasificacion_Significancia', index=False)
        
        # Hoja 4: PASO 4 - Resumen por comparación
        summary_df.to_excel(writer, sheet_name='4_Resumen_Por_Comparacion', index=False)
        
        # Hoja 5: Matriz de % Significancia
        sig_matrix_df.to_excel(writer, sheet_name='5_Matriz_Pct_Significancia')
        
        # Hoja 6: Matriz de ECRPS Medio
        ecrps_matrix_df.to_excel(writer, sheet_name='6_Matriz_ECRPS_Medio')
    
    print(f"\n✓ Excel guardado: {excel_filename}")
    print(f"  - Hoja 1: {n_scenarios} escenarios únicos")
    print(f"  - Hoja 2: {n_scenarios} × {n_comparisons} p-valores")
    print(f"  - Hoja 3: {n_scenarios} × {n_comparisons} clasificaciones")
    print(f"  - Hoja 4: {n_comparisons} resúmenes por comparación")
    print(f"  - Hojas 5-6: Matrices {n_models}×{n_models}")
    
    # CREAR VISUALIZACIÓN
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 10))
    
    # Heatmap 1: % Significancia
    sns.heatmap(sig_matrix_df, annot=True, fmt='.1f', cmap='RdYlGn', 
                center=50, vmin=0, vmax=100, ax=ax1,
                cbar_kws={'label': '% Tests Significativos'},
                linewidths=0.5, linecolor='gray', annot_kws={'fontsize': 9})
    
    ax1.set_title(f'% de Tests Significativos\n(α Bonferroni = {alpha_bonferroni:.6f}) [{tipo}]', 
                  fontsize=12, fontweight='bold', pad=15)
    ax1.set_xlabel('Modelo B', fontsize=11, fontweight='bold')
    ax1.set_ylabel('Modelo A', fontsize=11, fontweight='bold')
    
    # Heatmap 2: ECRPS Medio
    sns.heatmap(ecrps_matrix_df, annot=True, fmt='.4f', cmap='YlOrRd', 
                ax=ax2, cbar_kws={'label': 'ECRPS Promedio'},
                linewidths=0.5, linecolor='gray', annot_kws={'fontsize': 9})
    
    ax2.set_title(f'ECRPS Promedio por Modelo\n[{tipo}]', 
                  fontsize=12, fontweight='bold', pad=15)
    ax2.set_xlabel('Modelo B', fontsize=11, fontweight='bold')
    ax2.set_ylabel('Modelo A', fontsize=11, fontweight='bold')
    
    plt.suptitle(f'Test Diebold-Mariano: Proceso Completo\n{n_scenarios} escenarios × {n_comparisons} comparaciones = {n_scenarios * n_comparisons} tests', 
                 fontsize=14, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.savefig(output_dir / f'6.2{suffix}_dm_matriz_comparativa.png', bbox_inches='tight', dpi=300)
    plt.close()
    
    print(f"✓ Gráfica guardada: 6.2{suffix}_dm_matriz_comparativa.png")
    
    return scenario_results, summary_df, sig_matrix_df, ecrps_matrix_df

# Ejecutar análisis DM para cada tipo de escenario
for scenario_name, suffix in [(None, ''), 
                               ('Lineal Estacionario (ARMA)', '.a'),
                               ('Lineal No Estacionario (ARIMA)', '.b'),
                               ('No lineal Estacionario (SETAR)', '.c')]:
    compute_dm_tests_step_by_step(scenario_name, suffix)

print("\n" + "="*80)
print("PROCESO COMPLETADO CON ÉXITO")
print("="*80)
print("\n📊 ESTRUCTURA DE OUTPUTS:")
print("\nEXCEL (6.2[suffix]_dm_proceso_completo.xlsx):")
print("  Hoja 0: Metodología")
print("  Hoja 1: 420 escenarios únicos (Paso, Config, Dist, Var)")
print("  Hoja 2: 420 × 6 p-valores (cada columna = 1 comparación)")
print("  Hoja 2b: 420 × 6 estadísticos DM")
print("  Hoja 2c: 420 × 12 medias (A y B por comparación)")
print("  Hoja 3: 420 × 6 significancia (SÍ/NO)")
print("  Hoja 4: Resumen reducido (6 comparaciones)")
print("  Hoja 5: Matriz NxN % significancia")
print("  Hoja 6: Matriz NxN ECRPS medio")
print("\nGRÁFICA (6.2[suffix]_dm_matriz_comparativa.png):")
print("  Panel izquierdo: Heatmap % significancia")
print("  Panel derecho: Heatmap ECRPS medio")
print("="*80)

Nuevo orden de modelos (basado en Lineal No Estacionario (ARIMA)):
1. Sieve Bootstrap: 2.4058
2. LSPM: 2.8116
3. MCPS: 4.8714
4. DeepAR: 7.9710

SECCIÓN 1: RENDIMIENTO POR ESCENARIOS
✓ Gráfica 1.1 guardada
✓ Gráfica 1.2 guardada

SECCIÓN 2: ANÁLISIS POR CONFIG
✓ Gráfica 2.1 guardada
✓ Gráfica 2.1.a guardada
✓ Gráfica 2.1.b guardada
✓ Gráfica 2.1.c guardada
✓ Gráfica 2.2 guardada
✓ Gráfica 2.2.a guardada
✓ Gráfica 2.2.b guardada
✓ Gráfica 2.2.c guardada

SECCIÓN 3: ANÁLISIS POR DIST
✓ Gráfica 3.1 guardada
✓ Gráfica 3.1.a guardada
✓ Gráfica 3.1.b guardada
✓ Gráfica 3.1.c guardada
✓ Gráfica 3.2 guardada
✓ Gráfica 3.2.a guardada
✓ Gráfica 3.2.b guardada
✓ Gráfica 3.2.c guardada

SECCIÓN 4: ANÁLISIS POR VAR
✓ Gráfica 4.1 guardada
✓ Gráfica 4.1.a guardada
✓ Gráfica 4.1.b guardada
✓ Gráfica 4.1.c guardada
✓ Gráfica 4.2 guardada
✓ Gráfica 4.2.a guardada
✓ Gráfica 4.2.b guardada
✓ Gráfica 4.2.c guardada

SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)
✓ Gráfica 5.1 guardada
✓ Gráfica 5.1.a guardada
✓ 

### DM

In [12]:
"""
TEST DIEBOLD-MARIANO — LÓGICA CORRECTA
=======================================
Para cada grupo (Config × Dist × Var) dentro de un ESCENARIO:
  - Ordenar por Paso (1..12)
  - Serie A = ECRPS del Modelo A en esos 12 pasos  → vector de longitud 12
  - Serie B = ECRPS del Modelo B en esos 12 pasos  → vector de longitud 12
  - d_t = A_t - B_t  (t = 1..12)
  - Aplicar DM → 1 p-valor por par de modelos por grupo

Resultado: tabla 105 × 6  (por escenario) ó 315 × 6 (general)
           → resumen en heatmap con % Significancia y % Fila < Columna
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from itertools import combinations
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

# ------------------------------------------------------------------ #
# CONFIGURACIÓN
# ------------------------------------------------------------------ #
plt.rcParams.update({
    'figure.dpi': 300, 'savefig.dpi': 300,
    'font.size': 10, 'font.family': 'sans-serif',
    'axes.labelweight': 'bold', 'axes.titleweight': 'bold'
})
sns.set_style("whitegrid")

output_dir = Path("./Resultados_analisis/Simulacion_h")
output_dir.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------------ #
# CARGAR DATOS
# ------------------------------------------------------------------ #
df = pd.read_excel("./datos/Simulacion_h/dataframe_consolidado.xlsx")

df['ESCENARIO'] = df['ESCENARIO'].replace({
    "Lineal Estacionario":    "Lineal Estacionario (ARMA)",
    "Lineal No estacionario": "Lineal No Estacionario (ARIMA)",
    "No lineal Estacionario": "No lineal Estacionario (SETAR)"
})

var_cols   = ['Paso', 'Config', 'Dist', 'Var', 'ESCENARIO']
model_cols = [c for c in df.columns if c not in var_cols]

# Ordenar modelos por desempeño en ARIMA (menor ECRPS primero)
arima_means = df[df['ESCENARIO'] == "Lineal No Estacionario (ARIMA)"][model_cols].mean()
model_cols  = list(arima_means.sort_values().index)

ESCENARIOS = {
    '':   None,                               # General (todos)
    '.a': 'Lineal Estacionario (ARMA)',
    '.b': 'Lineal No Estacionario (ARIMA)',
    '.c': 'No lineal Estacionario (SETAR)',
}

LABEL = {
    '': 'GENERAL', '.a': 'ARMA', '.b': 'ARIMA', '.c': 'SETAR'
}

# ------------------------------------------------------------------ #
# FUNCIÓN DM — Newey-West + HLN
# ------------------------------------------------------------------ #
def dm_hln(ea: np.ndarray, eb: np.ndarray, h: int = 1):
    """
    Diebold-Mariano con Newey-West (kernel Bartlett) y corrección HLN.

    ea, eb : arrays de ECRPS alineados por Paso (misma longitud T).
    h      : horizonte de pronóstico del test. Para pronósticos multi-paso
             donde cada elemento de la serie es un horizonte distinto (Paso 1..12),
             se usa h=1 (interpretación: comparamos forecasters, no h-step-ahead).
             NO debe pasarse len(ea) porque con h=T el factor HLN se anula.

    Retorna (dm_stat, p_value) o (NaN, NaN) si T < 3.
    """
    d  = np.asarray(ea, float) - np.asarray(eb, float)
    T  = len(d)
    if T < 3:
        return np.nan, np.nan

    d_bar = d.mean()

    # --- Varianza de largo plazo: Newey-West con kernel Bartlett ---
    m       = max(1, int(np.floor(T ** (1.0 / 3.0))))
    gamma   = np.array([
        np.mean((d[:T-k] - d_bar) * (d[k:] - d_bar))
        for k in range(m + 1)
    ])
    weights = 1.0 - np.arange(1, m + 1) / (m + 1)
    var_d   = gamma[0] + 2.0 * np.dot(weights, gamma[1:])

    # --- Fallback si var_d es nula (serie d perfectamente constante) ---
    # Esto ocurre cuando los errores de ambos modelos siguen exactamente
    # el mismo patrón: la diferencia es sistemática y estable.
    # En ese caso usamos varianza empírica muestral como estimador mínimo.
    if var_d <= 1e-15:
        var_d = np.var(d, ddof=1)
    if var_d <= 1e-15:
        # d constante y diferente de 0: diferencia perfectamente sistemática
        # -> el test es máximamente significativo
        direction = np.sign(d_bar) if d_bar != 0 else 1.0
        return float(direction * np.inf), 0.0

    # --- Estadístico DM base ---
    dm_raw = d_bar / np.sqrt(var_d / T)

    # --- Corrección HLN (Harvey, Leybourne & Newbold 1997) ---
    # IMPORTANTE: h es el horizonte del forecast (Paso), NO len(serie).
    # Si se pasa h = T, el numerador (T+1-2T+T(T-1)/T) = 0 -> DM = 0 -> p = 1.
    hln_num = T + 1.0 - 2.0 * h + h * (h - 1.0) / T
    hln_num = max(hln_num, 1.0)   # mínimo: equivalente a no corregir (h=1 aprox)
    dm_stat = dm_raw * np.sqrt(hln_num / T)

    p_val = 2.0 * stats.t.sf(abs(dm_stat), df=T - 1)
    return float(dm_stat), float(p_val)


# ------------------------------------------------------------------ #
# FUNCIÓN PRINCIPAL
# ------------------------------------------------------------------ #
def run_dm_analysis(suffix: str = ''):
    scenario_filter = ESCENARIOS[suffix]
    label           = LABEL[suffix]

    # ── Filtrar datos ───────────────────────────────────────────────
    data = df.copy() if scenario_filter is None else \
           df[df['ESCENARIO'] == scenario_filter].copy()

    # ── Grupos: (Config, Dist, Var) [+ ESCENARIO si es general] ────
    # El Paso se usa como eje temporal DENTRO de cada grupo.
    group_keys = ['Config', 'Dist', 'Var'] if scenario_filter else \
                 ['ESCENARIO', 'Config', 'Dist', 'Var']

    groups     = data[group_keys].drop_duplicates().reset_index(drop=True)
    n_groups   = len(groups)

    model_pairs  = list(combinations(model_cols, 2))
    n_pairs      = len(model_pairs)
    pair_names   = [f"{a} vs {b}" for a, b in model_pairs]

    alpha            = 0.05
    alpha_bonferroni = alpha / n_pairs

    print(f"\n{'='*65}")
    print(f"  DM TEST — {label}")
    print(f"{'='*65}")
    print(f"  Grupos (Config × Dist × Var): {n_groups}")
    print(f"  Pasos por grupo             : {data['Paso'].nunique()}  (serie temporal)")
    print(f"  Pares de modelos            : {n_pairs}")
    print(f"  α Bonferroni                : {alpha_bonferroni:.6f}")
    print(f"  Tamaño tabla de resultados  : {n_groups} × {n_pairs}")

    # ── PASO 1 — Tabla detallada: n_groups × n_pairs ───────────────
    # Columnas: group_keys + [DM_stat, P_valor, Mean_A, Mean_B]  × n_pairs
    records = []

    for _, grp in groups.iterrows():
        # Filtrar las filas del grupo y ordenar por Paso
        mask = pd.Series([True] * len(data), index=data.index)
        for k in group_keys:
            mask &= (data[k] == grp[k])
        grp_data = data[mask].sort_values('Paso')

        row = {k: grp[k] for k in group_keys}

        for (ma, mb), pname in zip(model_pairs, pair_names):
            ea = grp_data[ma].values
            eb = grp_data[mb].values

            # h=1: cada elemento de la serie es un horizonte distinto (Paso 1..12).
            # No es un forecast h-step-ahead del mismo origen, sino un
            # forecast 1-step-ahead repetido en distintos horizontes.
            # Pasar h=len(ea)=12 anularía el factor HLN -> DM=0 -> p=1.
            dm_stat, p_val = dm_hln(ea, eb, h=1)

            row[f'DM_stat | {pname}']  = round(dm_stat, 6) if not np.isnan(dm_stat) else np.nan
            row[f'P_valor | {pname}']  = round(p_val,   6) if not np.isnan(p_val)   else np.nan
            row[f'Sig_Bonf | {pname}'] = ('SÍ' if (not np.isnan(p_val) and p_val < alpha_bonferroni)
                                          else 'NO')
            row[f'Mean_A | {pname}']   = round(float(ea.mean()), 6)
            row[f'Mean_B | {pname}']   = round(float(eb.mean()), 6)
            row[f'A<B | {pname}']      = 'SÍ' if ea.mean() < eb.mean() else 'NO'

        records.append(row)

    detail_df = pd.DataFrame(records)

    # ── PASO 2 — Tabla de p-valores limpia (n_groups × n_pairs) ────
    pval_cols = [f'P_valor | {p}' for p in pair_names]
    dm_cols   = [f'DM_stat | {p}' for p in pair_names]
    sig_cols  = [f'Sig_Bonf | {p}' for p in pair_names]
    mA_cols   = [f'Mean_A | {p}' for p in pair_names]
    mB_cols   = [f'Mean_B | {p}' for p in pair_names]
    ab_cols   = [f'A<B | {p}' for p in pair_names]

    # ── PASO 3 — Matrices resumen NxN ──────────────────────────────
    n_mod = len(model_cols)
    sig_pct_mat  = np.zeros((n_mod, n_mod))   # % grupos donde p < α_bonf
    mean_pct_mat = np.zeros((n_mod, n_mod))   # % grupos donde mean_fila < mean_col

    for (ma, mb), pname in zip(model_pairs, pair_names):
        ia, ib = model_cols.index(ma), model_cols.index(mb)

        pvals  = detail_df[f'P_valor | {pname}'].dropna()
        meansA = detail_df[f'Mean_A | {pname}']
        meansB = detail_df[f'Mean_B | {pname}']

        n_val   = len(pvals)
        n_sig   = (pvals < alpha_bonferroni).sum()
        pct_sig = 100.0 * n_sig / n_val if n_val > 0 else 0.0

        n_A_lt_B = (meansA < meansB).sum()
        n_B_lt_A = (meansB < meansA).sum()
        pct_A_lt_B = 100.0 * n_A_lt_B / len(meansA)
        pct_B_lt_A = 100.0 * n_B_lt_A / len(meansB)

        # Sig es simétrica
        sig_pct_mat[ia, ib] = pct_sig
        sig_pct_mat[ib, ia] = pct_sig

        # % fila < columna (asimétrica):  [ia,ib] = A<B,  [ib,ia] = B<A
        mean_pct_mat[ia, ib] = pct_A_lt_B
        mean_pct_mat[ib, ia] = pct_B_lt_A

    sig_df  = pd.DataFrame(sig_pct_mat,  index=model_cols, columns=model_cols)
    mean_df = pd.DataFrame(mean_pct_mat, index=model_cols, columns=model_cols)

    # Diagonal: NaN (no aplica)
    np.fill_diagonal(sig_pct_mat,  np.nan)
    np.fill_diagonal(mean_pct_mat, np.nan)

    # ── PASO 4 — Heatmap combinado ──────────────────────────────────
    annot = np.empty((n_mod, n_mod), dtype=object)
    for i in range(n_mod):
        for j in range(n_mod):
            if i == j:
                annot[i, j] = f'{data[model_cols[i]].mean():.4f}'
            else:
                annot[i, j] = (
                    f'Sig: {sig_pct_mat[i,j]:.0f}%\n'
                    f'F<C: {mean_pct_mat[i,j]:.0f}%'
                )

    fig, ax = plt.subplots(figsize=(max(14, n_mod * 2), max(12, n_mod * 1.8)))

    # Base del heatmap = % Significancia
    heat_data = pd.DataFrame(sig_pct_mat, index=model_cols, columns=model_cols)

    sns.heatmap(
        heat_data, annot=annot, fmt='', cmap='RdYlGn',
        center=50, vmin=0, vmax=100, ax=ax,
        cbar_kws={'label': '% Tests Significativos (Bonferroni)'},
        linewidths=0.6, linecolor='gray',
        annot_kws={'fontsize': 8}
    )

    ax.set_title(
        f'Matriz DM — {label}\n'
        f'Sig: % grupos (de {n_groups}) con p < α Bonf ({alpha_bonferroni:.5f})  |  '
        f'F<C: % grupos con media fila < media columna',
        fontsize=12, fontweight='bold', pad=15
    )
    ax.set_xlabel('Modelo (Columna C)', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo (Fila F)',    fontsize=11, fontweight='bold')
    ax.tick_params(axis='x', rotation=45, labelsize=9)
    ax.tick_params(axis='y', rotation=0,  labelsize=9)

    plt.tight_layout()
    png_path = output_dir / f'6.4{suffix}_matriz_DM.png'
    plt.savefig(png_path, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"  ✓ Heatmap guardado → {png_path.name}")

    # ── PASO 5 — Excel con todas las hojas ─────────────────────────
    excel_path = output_dir / f'6.4{suffix}_dm_resultados.xlsx'

    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:

        # Hoja 0: Metodología
        pd.DataFrame({
            'Parámetro': [
                'Análisis', 'Alpha nominal', 'Alpha Bonferroni',
                'Modelos comparados', 'Pares totales',
                'Grupos (Config×Dist×Var)', 'Pasos por grupo (serie DM)',
                'Tabla de resultados', 'Test', 'Corrección varianza', 'Corrección estadístico'
            ],
            'Valor': [
                label, alpha, round(alpha_bonferroni, 8),
                len(model_cols), n_pairs,
                n_groups, data['Paso'].nunique(),
                f'{n_groups} filas × {n_pairs} pares',
                'Diebold-Mariano', 'Newey-West Bartlett (BW = floor(T^1/3))',
                'HLN 1997 — t(T-1)'
            ]
        }).to_excel(writer, sheet_name='0_Metodologia', index=False)

        # Hoja 1: P-valores  (n_grupos × n_pares)
        detail_df[group_keys + pval_cols].to_excel(
            writer, sheet_name='1_PValores', index=False)

        # Hoja 2: Estadísticos DM
        detail_df[group_keys + dm_cols].to_excel(
            writer, sheet_name='2_DM_Stats', index=False)

        # Hoja 3: Significancia (SÍ/NO)
        detail_df[group_keys + sig_cols].to_excel(
            writer, sheet_name='3_Significancia_SiNo', index=False)

        # Hoja 4: Medias A y B
        detail_df[group_keys + mA_cols + mB_cols].to_excel(
            writer, sheet_name='4_Medias_A_B', index=False)

        # Hoja 5: A<B (SÍ/NO)
        detail_df[group_keys + ab_cols].to_excel(
            writer, sheet_name='5_A_menor_B', index=False)

        # Hoja 6: Tabla completa
        detail_df.to_excel(writer, sheet_name='6_Tabla_Completa', index=False)

        # Hoja 7: Matriz % Significancia
        sig_df.to_excel(writer, sheet_name='7_Matriz_Pct_Significancia')

        # Hoja 8: Matriz % F<C
        mean_df.to_excel(writer, sheet_name='8_Matriz_Pct_FilaMenusCol')

        # Hoja 9: Resumen por modelo
        resumen = []
        for m in model_cols:
            i = model_cols.index(m)
            off_diag_sig  = [sig_pct_mat[i,j]  for j in range(n_mod) if j != i and not np.isnan(sig_pct_mat[i,j])]
            off_diag_mean = [mean_pct_mat[i,j] for j in range(n_mod) if j != i and not np.isnan(mean_pct_mat[i,j])]
            resumen.append({
                'Modelo'                    : m,
                'ECRPS_promedio'            : round(data[m].mean(), 6),
                'Sig_promedio_%'            : round(np.mean(off_diag_sig),  2),
                'F<C_promedio_%'            : round(np.mean(off_diag_mean), 2),
                'F<C_mediana_%'             : round(np.median(off_diag_mean), 2),
            })
        pd.DataFrame(resumen).sort_values('ECRPS_promedio').to_excel(
            writer, sheet_name='9_Resumen_por_Modelo', index=False)

    print(f"  ✓ Excel guardado    → {excel_path.name}")
    print(f"    Hoja 1: {n_groups} filas × {n_pairs} pares (p-valores)")
    print(f"    Hojas 7-8: matrices {n_mod}×{n_mod}")

    return detail_df, sig_df, mean_df


# ------------------------------------------------------------------ #
# EJECUTAR PARA TODOS LOS ESCENARIOS
# ------------------------------------------------------------------ #
for suf in ['', '.a', '.b', '.c']:
    run_dm_analysis(suf)

print("\n" + "="*65)
print("  PROCESO COMPLETADO")
print("="*65)


  DM TEST — GENERAL
  Grupos (Config × Dist × Var): 420
  Pasos por grupo             : 12  (serie temporal)
  Pares de modelos            : 6
  α Bonferroni                : 0.008333
  Tamaño tabla de resultados  : 420 × 6
  ✓ Heatmap guardado → 6.4_matriz_DM.png
  ✓ Excel guardado    → 6.4_dm_resultados.xlsx
    Hoja 1: 420 filas × 6 pares (p-valores)
    Hojas 7-8: matrices 4×4

  DM TEST — ARMA
  Grupos (Config × Dist × Var): 140
  Pasos por grupo             : 12  (serie temporal)
  Pares de modelos            : 6
  α Bonferroni                : 0.008333
  Tamaño tabla de resultados  : 140 × 6
  ✓ Heatmap guardado → 6.4.a_matriz_DM.png
  ✓ Excel guardado    → 6.4.a_dm_resultados.xlsx
    Hoja 1: 140 filas × 6 pares (p-valores)
    Hojas 7-8: matrices 4×4

  DM TEST — ARIMA
  Grupos (Config × Dist × Var): 140
  Pasos por grupo             : 12  (serie temporal)
  Pares de modelos            : 6
  α Bonferroni                : 0.008333
  Tamaño tabla de resultados  : 140 × 6
  ✓ He

# Revisión uso DM

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# ============================================================================
# SIMULACIÓN DE DATOS SIMPLIFICADA
# ============================================================================
# Supongamos solo 2 modelos, 2 configs, 2 distribuciones, 2 varianzas, 3 pasos
# Total: 2×2×2×3 = 24 observaciones por modelo

np.random.seed(42)

configs = [1, 2]
dists = ['Normal', 'T-Student']
vars_val = [0.5, 1.0]
pasos = [1, 2, 3]

data = []
for config in configs:
    for dist in dists:
        for var in vars_val:
            for paso in pasos:
                # Modelo A: mejor en general, pero peor con Config=2 + Var=1.0
                if config == 2 and var == 1.0:
                    ecrps_A = np.random.normal(0.8, 0.1)  # Peor
                else:
                    ecrps_A = np.random.normal(0.4, 0.05)  # Mejor
                
                # Modelo B: más estable, rendimiento medio
                ecrps_B = np.random.normal(0.5, 0.05)
                
                data.append({
                    'Config': config,
                    'Dist': dist,
                    'Var': var,
                    'Paso': paso,
                    'Modelo_A': ecrps_A,
                    'Modelo_B': ecrps_B
                })

df = pd.DataFrame(data)
print("="*80)
print("DATOS SIMULADOS (primeras 12 filas)")
print("="*80)
print(df.head(12))
print(f"\nTotal observaciones: {len(df)}")

# ============================================================================
# ENFOQUE ACTUAL: UN SOLO TEST D-M CON TODAS LAS OBSERVACIONES
# ============================================================================
print("\n" + "="*80)
print("ENFOQUE 1: TEST D-M GLOBAL (COMO ESTÁ EN TU CÓDIGO)")
print("="*80)

errors_A = df['Modelo_A'].values
errors_B = df['Modelo_B'].values
diferencias = errors_A - errors_B

print(f"\nDiferencias individuales (primeras 12):")
print(diferencias[:12])
print(f"\nMedia de diferencias: {np.mean(diferencias):.4f}")
print(f"Desv. Std de diferencias: {np.std(diferencias, ddof=1):.4f}")

# Test D-M simple
d_bar = np.mean(diferencias)
T = len(diferencias)
se = np.std(diferencias, ddof=1) / np.sqrt(T)
dm_stat = d_bar / se
p_value_dm = 2 * (1 - stats.norm.cdf(abs(dm_stat)))

print(f"\nEstadístico D-M: {dm_stat:.4f}")
print(f"P-valor: {p_value_dm:.4f}")
print(f"Conclusión: {'Modelo A significativamente diferente de B' if p_value_dm < 0.05 else 'No hay diferencia significativa'}")

# ============================================================================
# ENFOQUE ALTERNATIVO 1: TESTS D-M ESTRATIFICADOS
# ============================================================================
print("\n" + "="*80)
print("ENFOQUE 2: TESTS D-M ESTRATIFICADOS POR CONDICIÓN")
print("="*80)

print("\nTests por Config:")
for config in configs:
    subset = df[df['Config'] == config]
    dif = subset['Modelo_A'].values - subset['Modelo_B'].values
    d_bar = np.mean(dif)
    se = np.std(dif, ddof=1) / np.sqrt(len(dif))
    dm_stat = d_bar / se
    p_val = 2 * (1 - stats.norm.cdf(abs(dm_stat)))
    print(f"  Config {config}: media_dif={d_bar:.4f}, DM={dm_stat:.4f}, p={p_val:.4f}")

print("\nTests por Varianza:")
for var in vars_val:
    subset = df[df['Var'] == var]
    dif = subset['Modelo_A'].values - subset['Modelo_B'].values
    d_bar = np.mean(dif)
    se = np.std(dif, ddof=1) / np.sqrt(len(dif))
    dm_stat = d_bar / se
    p_val = 2 * (1 - stats.norm.cdf(abs(dm_stat)))
    print(f"  Var {var}: media_dif={d_bar:.4f}, DM={dm_stat:.4f}, p={p_val:.4f}")

print("\nTests por Config × Var (combinación crítica):")
for config in configs:
    for var in vars_val:
        subset = df[(df['Config'] == config) & (df['Var'] == var)]
        dif = subset['Modelo_A'].values - subset['Modelo_B'].values
        d_bar = np.mean(dif)
        se = np.std(dif, ddof=1) / np.sqrt(len(dif))
        dm_stat = d_bar / se if se > 0 else 0
        p_val = 2 * (1 - stats.norm.cdf(abs(dm_stat))) if se > 0 else 1.0
        print(f"  Config={config}, Var={var}: media_dif={d_bar:.4f}, DM={dm_stat:.4f}, p={p_val:.4f}")

# ============================================================================
# ENFOQUE ALTERNATIVO 2: ANOVA SOBRE LAS DIFERENCIAS
# ============================================================================
print("\n" + "="*80)
print("ENFOQUE 3: ANOVA MULTIFACTORIAL SOBRE DIFERENCIAS")
print("="*80)

df['Diferencia'] = df['Modelo_A'] - df['Modelo_B']

# ANOVA de efectos principales
from scipy.stats import f_oneway

print("\nEfecto de Config:")
grupos_config = [df[df['Config'] == c]['Diferencia'].values for c in configs]
F_stat, p_val = f_oneway(*grupos_config)
print(f"  F={F_stat:.4f}, p={p_val:.4f}")

print("\nEfecto de Var:")
grupos_var = [df[df['Var'] == v]['Diferencia'].values for v in vars_val]
F_stat, p_val = f_oneway(*grupos_var)
print(f"  F={F_stat:.4f}, p={p_val:.4f}")

print("\nEfecto de Dist:")
grupos_dist = [df[df['Dist'] == d]['Diferencia'].values for d in dists]
F_stat, p_val = f_oneway(*grupos_dist)
print(f"  F={F_stat:.4f}, p={p_val:.4f}")

# Interacción Config × Var
print("\nEfecto de interacción Config × Var:")
grupos_interaccion = []
for config in configs:
    for var in vars_val:
        subset = df[(df['Config'] == config) & (df['Var'] == var)]
        grupos_interaccion.append(subset['Diferencia'].values)
F_stat, p_val = f_oneway(*grupos_interaccion)
print(f"  F={F_stat:.4f}, p={p_val:.4f}")

# ============================================================================
# RESUMEN VISUAL
# ============================================================================
print("\n" + "="*80)
print("RESUMEN: MEDIAS DE DIFERENCIAS POR CONDICIÓN")
print("="*80)

pivot = df.pivot_table(values='Diferencia', 
                       index=['Config', 'Var'], 
                       columns='Dist', 
                       aggfunc='mean')
print("\nTabla: Diferencia promedio (Modelo_A - Modelo_B)")
print(pivot.round(4))
print("\nInterpretación:")
print("  - Valores negativos: Modelo A mejor (menor ECRPS)")
print("  - Valores positivos: Modelo B mejor")
print("  - Nota: Config=2 + Var=1.0 tiene diferencias positivas (A peor que B)")

# ============================================================================
# CONCLUSIÓN COMPARATIVA
# ============================================================================
print("\n" + "="*80)
print("CONCLUSIONES COMPARATIVAS")
print("="*80)

print("\n1. TEST D-M GLOBAL (tu código actual):")
print(f"   → Conclusión: Modelo A {'mejor' if d_bar < 0 else 'peor'} que B en promedio")
print(f"   → Problema: NO detecta que en Config=2+Var=1.0 la situación se invierte")

print("\n2. TESTS D-M ESTRATIFICADOS:")
print("   → Conclusión: La superioridad de A depende de las condiciones")
print("   → Ventaja: Identifica dónde cada modelo es mejor")
print("   → Problema: Múltiples tests, ajuste de Bonferroni muy conservador")

print("\n3. ANOVA SOBRE DIFERENCIAS:")
print("   → Conclusión: Hay efectos significativos de Config, Var, y su interacción")
print("   → Ventaja: Cuantifica qué factores afectan las diferencias")
print("   → Limitación: No hace comparación pairwise directa entre modelos")

print("\n" + "="*80)
print("RECOMENDACIÓN PARA TU ANÁLISIS")
print("="*80)
print("""
Deberías combinar los tres enfoques:

1. Test D-M global → Responde: "¿Quién es mejor en general?"
2. Tests D-M por escenario → Responde: "¿Esa conclusión se mantiene en todos los escenarios?"
3. ANOVA sobre diferencias → Responde: "¿Qué factores explican las diferencias?"

Esto te da una historia completa:
- Modelo A es mejor en promedio (D-M global)
- PERO esa ventaja desaparece cuando Config=2 y Var=1.0 (D-M estratificado)
- Los factores Config y Var tienen efectos significativos (ANOVA)
""")

DATOS SIMULADOS (primeras 12 filas)
    Config       Dist  Var  Paso  Modelo_A  Modelo_B
0        1     Normal  0.5     1  0.424836  0.493087
1        1     Normal  0.5     2  0.432384  0.576151
2        1     Normal  0.5     3  0.388292  0.488293
3        1     Normal  1.0     1  0.478961  0.538372
4        1     Normal  1.0     2  0.376526  0.527128
5        1     Normal  1.0     3  0.376829  0.476714
6        1  T-Student  0.5     1  0.412098  0.404336
7        1  T-Student  0.5     2  0.313754  0.471886
8        1  T-Student  0.5     3  0.349358  0.515712
9        1  T-Student  1.0     1  0.354599  0.429385
10       1  T-Student  1.0     2  0.473282  0.488711
11       1  T-Student  1.0     3  0.403376  0.428763

Total observaciones: 24

ENFOQUE 1: TEST D-M GLOBAL (COMO ESTÁ EN TU CÓDIGO)

Diferencias individuales (primeras 12):
[-0.06825108 -0.14376707 -0.10000082 -0.0594111  -0.15060172 -0.0998844
  0.00776213 -0.15813152 -0.16635392 -0.07478602 -0.01542875 -0.02538618]

Media de 